## Загрузка библиотек

In [1]:
#basic
import pandas as pd
from typing import Dict
import numpy as np
import re

#error
import warnings
warnings.filterwarnings('ignore')

## Загрузка данных

#### Напишем функцию, с помощью которой будем загружать данные

> Так как у нас таблицы из разных источников, то и форматирование у них разное, в том числе начало строк с информацией по блогерам можем быть на 10 строчки, а название столбцов у разных источников отличаться. Чтобы привести к единому формату, сделаем функции и в укажет ней ключи, по которым обрежем таблицу снизу и сверху

In [2]:
def read_csv_from_dict(data_dict: Dict[str, str], column_number: int, header_keyword: str, footer_keyword: str) -> pd.DataFrame:
    
    
    """
    Функция принимает как параметр словарь data_dict, в котором ключ - название DataFrame, а объект - ссылка на источник,
    и в дальнейшем мы обрезаем DataFrame с помощью параметров column_number, header_keyword, footer_keyword.
    
    ПАРАМЕТРЫ:
    - data_dict (DataFrame) - словарь, в котором ключ - это будущее название датафрейма, объект - 
    ссылка на страницу таблицы, из которой берём данные.
    - column_number (int) - номер столбца, по которому определяем границы DataFrame.
    - header_keyword (str) - ключ, по которому мы обрезаем таблице сверху.
    - footer_keyword - ключ, по которому мы обрезаем таблице снизу.
    
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    df: DataFrame, название и источник которого указан в словаре data_dict и который обрезан по нашим параметрам header_keyword и footer_keyword
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    didenok_df = read_csv_from_dict(didenok_data_dict, 0, didenok_header_keyword, didenok_footer_keyword)
    """
    
    for name, url in data_dict.items():
        url = url.replace('/edit#gid=', '/export?format=csv&gid=')
        df = pd.read_csv(url)

# В pandas метод .iloc используется для доступа к элементам в DataFrame или Series по их целочисленным позициям.
# Индексация начинается с нуля, поэтому .iloc[n] обращается к элементу с позицией n-1.        
    
        header_index = df.loc[df.iloc[:, column_number].str.strip().isin(header_keyword)].index[0]
        
#Используем globals() для создания новой переменной с именем, которое хранится в переменной name,
# и присваиваем этой переменной значение, полученное в результате операции df.set_axis(df.iloc[header_index], axis='columns').

        globals()[name] = df.set_axis(df.iloc[header_index], axis='columns')

        if footer_keyword:
            footer_index = df.loc[df.iloc[:, column_number].str.strip().isin(footer_keyword)].index[0] - 2
            globals()[name] = globals()[name].iloc[header_index + 1:footer_index]
        else:
            globals()[name] = globals()[name].iloc[header_index + 1:]
            
    return df

In [3]:
print(read_csv_from_dict.__doc__)


    Функция принимает как параметр словарь data_dict, в котором ключ - название DataFrame, а объект - ссылка на источник,
    и в дальнейшем мы обрезаем DataFrame с помощью параметров column_number, header_keyword, footer_keyword.
    
    ПАРАМЕТРЫ:
    - data_dict (DataFrame) - словарь, в котором ключ - это будущее название датафрейма, объект - 
    ссылка на страницу таблицы, из которой берём данные.
    - column_number (int) - номер столбца, по которому определяем границы DataFrame.
    - header_keyword (str) - ключ, по которому мы обрезаем таблице сверху.
    - footer_keyword - ключ, по которому мы обрезаем таблице снизу.
    
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    df: DataFrame, название и источник которого указан в словаре data_dict и который обрезан по нашим параметрам header_keyword и footer_keyword
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    didenok_df = read_csv_from_dict(didenok_data_dict, 0, didenok_header_keyword, didenok_footer_keyword)
    


> Стоит отметить, что так как исходные данные у нас лежат на Google-таблицах, то помимо разного формата, мы ещё столнулись с тем, что часть ссылок сделаны гиперссылками в виде текста, которые Pandas и специальные сторонние библиотеки переводят текст. Поэтому чтобы не разводить лишний код, мы сделали определённые преобразования в Google-таблицах, а именно:
>* Через функцию IMPORTRANGE копируем данные из разных Google-таблиц в единую.
>* Напишем функцию для App Script на специальном языке google script (gs), которая преобразовывает гиперссылки в ссылки и выводит их в отдельный столбец (пример функции ниже):
>
><em>function GetURL(input) {  
  var myFormula = SpreadsheetApp.getActiveRange().getFormula();  
  var myAddress = myFormula.replace('=GetURL(','').replace(')','');   
  var myRange = SpreadsheetApp.getActiveSheet().getRange(myAddress);  
  return myRange.getRichTextValue().getLinkUrl();  
 };</em>  
> 
>**За счёт этого мы**:
>* Получим единую Google-таблицу, в которой видим все предварительные данные, в том числе если будут какие-то изменения в первоначальных данным, то мы это увидим, что позволит нам избежать ошибки.
>* Решим проблему с гиперссылками.

### Блогеры от Didenok Team

In [4]:
didenok_data_dict = {'didenok_instagram':'https://docs.google.com/spreadsheets/d/19eW6Uv1qFeOCms8X63rTN3FJwIgnBbLZ8vhIfW18NLI/edit#gid=0',  
                     'didenok_youtube':'https://docs.google.com/spreadsheets/d/19eW6Uv1qFeOCms8X63rTN3FJwIgnBbLZ8vhIfW18NLI/edit#gid=1530136015',
                     'didenok_telegram':'https://docs.google.com/spreadsheets/d/19eW6Uv1qFeOCms8X63rTN3FJwIgnBbLZ8vhIfW18NLI/edit#gid=1663343566',
                     'didenok_vk_zen_show':'https://docs.google.com/spreadsheets/d/19eW6Uv1qFeOCms8X63rTN3FJwIgnBbLZ8vhIfW18NLI/edit#gid=1247151005',
                     'didenok_vk_group':'https://docs.google.com/spreadsheets/d/19eW6Uv1qFeOCms8X63rTN3FJwIgnBbLZ8vhIfW18NLI/edit#gid=1346604925',
#                      'didenok_vk_ls':'https://docs.google.com/spreadsheets/d/19eW6Uv1qFeOCms8X63rTN3FJwIgnBbLZ8vhIfW18NLI/edit#gid=1735507243',
                     'didenok_twitch':'https://docs.google.com/spreadsheets/d/19eW6Uv1qFeOCms8X63rTN3FJwIgnBbLZ8vhIfW18NLI/edit#gid=765599759'
                    }

In [5]:
didenok_header_keyword = ['Блогер', 'Канал']
didenok_footer_keyword = ['Примечание']

In [6]:
didenok_df = read_csv_from_dict(didenok_data_dict,
                                0,
                                didenok_header_keyword,
                                didenok_footer_keyword)

> Проверим результат

In [7]:
didenok_instagram.head()

7,Блогер,Ссылка,Категория/\nТематика блогера,Вид сотрудничества**,Количество подписчиков,Средний охват фотопоста*,Средний охват 1 сторис*,Среднее кол-во просмотров Reels*,Среднее кол-во лайков*,Среднее кол-во комментариев*,"ERR, %",Стоимость за фотопост (руб.) до НДС,Стоимость за 1 сторис до 15 сек (руб.) до НДС,Стоимость за серию из 3х сториc до 45 сек (руб.) до НДС,Стоимость за Reels (руб.) до НДС,CPU (фото),CPU (сторис),CPV \n(Reels),Статистика,Статистика
8,Катя Клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,П,10 100 000,250 000,150 000,200 000,50 000,100,"20,04%",р.375 000,р.335 000,р.530 000,р.820 000,"р.1,5","р.2,2","р.4,1",Статистика,https://drive.google.com/drive/folders/1J26W2R...
9,Влад Бумага А4,https://www.instagram.com/a4omg/,челленджи,П,7 100 000,900 000,400 000,1 000 000,60 000,700,"6,74%",р.875 000,р.440 000,р.625 000,р.1 875 000,"р.1,0","р.1,1","р.1,9",-,NaN
10,Катя Адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,Э,6 300 000,700 000,260 000,600 000,30 000,100,"4,30%",р.445 000,р.305 000,р.625 000,р.685 000,"р.0,6","р.1,2","р.1,1",Статистика,https://drive.google.com/drive/u/1/folders/11S...
11,Амина Тендерлибае,https://www.instagram.com/tenderlybae/,твич/лайфстайл,Э,3 900 000,600 000,300 000,650 000,110 000,150,"18,36%",р.695 000,р.265 000,р.625 000,р.835 000,"р.1,2","р.0,9","р.1,3",Статистика,https://drive.google.com/drive/u/1/folders/1M3...
12,VikiShow,https://www.instagram.com/vikishow_official/,лайфстайл/дети/подростки,Э,2 500 000,70 000,30 000,80 000,15 000,200,"21,71%",р.120 000,р.60 000,р.135 000,р.155 000,"р.1,7","р.2,0","р.1,9",Статистика,https://drive.google.com/drive/folders/1haDuDs...


### Блогеры от VK

In [8]:
vk_data_dict = {'vk_bloggers':\
                'https://docs.google.com/spreadsheets/d/19eW6Uv1qFeOCms8X63rTN3FJwIgnBbLZ8vhIfW18NLI/edit#gid=1569694027'
               }

In [9]:
vk_header_keyword = ['Сообщество']
vk_footer_keyword = []

In [10]:
vk_df = read_csv_from_dict(vk_data_dict,
                           0,
                           vk_header_keyword,
                           vk_footer_keyword
                          )

> Проверим результат

In [11]:
vk_bloggers.head()

,Сообщество,Шоу,KPI (стартов просмотра на 1 эпизод на 30-е сутки после релиза),Стоимость интеграции 1-й слот до НДС,Стоимость интеграции 2-й слот до НДС,Продакт плейсмент (30 сек) до НДС,Стоимость рекламной заставки (пре-ролл до 20 сек) до НДС,Стоимость рекламной заставки (мид-ролл до 20 сек) до НДС,Статистика,Контакты менеджера по продажам,Ссылка,Статистика ссылка
1,Азамат Мусагалиев,Большое шоу,2 600 000,р.3 500 000,р.3 200 000,р.2 000 000,р.3 000 000,р.2 800 000,Статистика,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...
2,NaN,Это про меня,600 000,р.700 000,по запросу,р.300 000,р.300 000,р.200 000,NaN,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",NaN,NaN
3,NaN,Я себя знаю,2 000 000,р.2 800 000,р.2 500 000,по запросу,р.2 000 000,р.1 800 000,NaN,"Марина Ванюшкина, TG: @MarinaVan, m.vanyushkin...",NaN,NaN
4,NaN,Шоу памяти Азамата Мусагалиева NEW,1 500 000,р.2 500 000,по запросу,по запросу,р.2 000 000,по запросу,NaN,"Магда Булавина, TG: @Portenato, m.bulavina@vk....",NaN,NaN
5,Пикабу,Чебатков читает Пикабу,1 000 000,р.1 350 000,по запросу,по запросу,р.1 000 000,по запросу,Статистика,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/pikabu,https://drive.google.com/drive/u/1/folders/1Aq...


### Блогеры от BlogoSphere

In [12]:
blogosphere_data_dict = {'blogosphere_youtube':'https://docs.google.com/spreadsheets/d/19eW6Uv1qFeOCms8X63rTN3FJwIgnBbLZ8vhIfW18NLI/edit#gid=871143129',
                        'blogosphere_tg_vk_zen':'https://docs.google.com/spreadsheets/d/19eW6Uv1qFeOCms8X63rTN3FJwIgnBbLZ8vhIfW18NLI/edit#gid=292459457'
                        }

In [13]:
blogosphere_header_keyword = ['Название канала', 'Имя блогера']
blogosphere_footer_keyword = []

In [14]:
blogosphere_df = read_csv_from_dict(blogosphere_data_dict,
                                    1,
                                    blogosphere_header_keyword,
                                    blogosphere_footer_keyword)

> Проверим результат

In [15]:
blogosphere_youtube.head()

5,NaN,Название канала,Шоу,Подписчики,Среднее кол-во просмотров на видео,Статистика аудитории,Стандартные форматы,NaN,NaN,NaN,...,NaN,NaN,NaN,Пример рекламы 1,Пример рекламы 2,Пример рекламы 3,Среднее CPV,NaN,Ссылка,Статистика ссылка
6,NaN,NaN,NaN,NaN,NaN,NaN,Формат 1,NaN,Формат 2,NaN,...,NaN,Формат 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,Описание,Цена* от,Описание,Цена* от,...,Цена* от,Описание,Цена* от,NaN,NaN,NaN,Стандартный Формат 1,Стандартный Формат 2,NaN,NaN
8,Э,NaN,"Осторожно, Собчак",NaN,1 000 000,NaN,1 слот,1 310 000 ₽,2 слот,960 000 ₽,...,1 310 000 ₽,Product Placement,1 470 000 ₽,Eduson,Экоплант,Травелата,"1,31₽","0,96₽",NaN,NaN
9,NaN,Ксения Собчак,Осторожно: новости!,3 430 000,550 000,Подробнее,1 слот,760 000 ₽,2 Слот,510 000 ₽,...,NaN,NaN,NaN,Astons,CGPods,SkillFactory,"1,38₽","0,93₽",https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...
10,NaN,NaN,Красота требует КЭШ,NaN,800 000,NaN,1 слот,960 000 ₽,2 Слот,860 000 ₽,...,NaN,NaN,NaN,Золотое яблоко,4FORMS,FOREO,"1,20₽","1,08₽",NaN,NaN


### Блогеры от Insight People

In [16]:
insight_data_dict = {'insight_instagram':'https://docs.google.com/spreadsheets/d/1pmm_Kena9NjqCzc7OpSJ-QlG1xqxiwxJ/edit#gid=101311564',
                     'insight_threads':'https://docs.google.com/spreadsheets/d/1pmm_Kena9NjqCzc7OpSJ-QlG1xqxiwxJ/edit#gid=328762867',
                     'insight_telegram':'https://docs.google.com/spreadsheets/d/1pmm_Kena9NjqCzc7OpSJ-QlG1xqxiwxJ/edit#gid=967469171',
                     'insight_vk_ls':'https://docs.google.com/spreadsheets/d/1pmm_Kena9NjqCzc7OpSJ-QlG1xqxiwxJ/edit#gid=726736292',
                     'insight_vk_group':'https://docs.google.com/spreadsheets/d/1pmm_Kena9NjqCzc7OpSJ-QlG1xqxiwxJ/edit#gid=696480148',
                     'insight_youtube':'https://docs.google.com/spreadsheets/d/1pmm_Kena9NjqCzc7OpSJ-QlG1xqxiwxJ/edit#gid=1815862158',
                     'insight_zen':'https://docs.google.com/spreadsheets/d/1pmm_Kena9NjqCzc7OpSJ-QlG1xqxiwxJ/edit#gid=640897440',
                     'insight_likee':'https://docs.google.com/spreadsheets/d/1pmm_Kena9NjqCzc7OpSJ-QlG1xqxiwxJ/edit#gid=952598266',
                     'insight_yappy':'https://docs.google.com/spreadsheets/d/1pmm_Kena9NjqCzc7OpSJ-QlG1xqxiwxJ/edit#gid=1606329803',
                     'insight_rutube':'https://docs.google.com/spreadsheets/d/1pmm_Kena9NjqCzc7OpSJ-QlG1xqxiwxJ/edit#gid=1626713104',
#                      'insight_ok':'https://docs.google.com/spreadsheets/d/1pmm_Kena9NjqCzc7OpSJ-QlG1xqxiwxJ/edit#gid=282211806'
                    }

In [17]:
insight_header_keyword = ['Город']
insight_footer_keyword = []

In [18]:
insight_df = read_csv_from_dict(insight_data_dict,
                                0,
                                insight_header_keyword,
                                insight_footer_keyword)

> Проверим результат

In [19]:
insight_rutube.head()

3,Город,ФИО,Ссылка,Количество подписчиков,Средние просмотры за месяц,Стоимость преролла,Стоимость интеграции (до 1 минуты),"CPV, преролл","CPV, интеграция"
4,Тверь,Аркатов Станислав Александрович,https://rutube.ru/channel/23684374/,Фудблогер,118 300,25 030,"р.50 060,00","р.0,75","р.2,00"
5,Краснодар,Солодов Юрий Юрьевич,https://rutube.ru/channel/23722477/,"Лайфстайл, влог",24 500,47 000,"р.82 250,00","р.0,50","р.1,75"
6,Казань,Ахмадуллин Наиль Радикович,https://rutube.ru/channel/23694775/,"Путешествия, лайфстайл",18 500,27 524,под запрос,-,-
7,Воронеж,Рязанова Татьяна Алексеевна,https://rutube.ru/channel/19363597/,"Бьюти, лайфстайл",16 500,50 066,"р.87 615,00","р.0,75","р.2,00"
8,Воронеж,Лаврова Валерия Владимировна,https://rutube.ru/channel/26070201/,Художник,15 600,40 063,"р.70 115,00","р.0,75","р.2,00"


### Блогеры от NextUp

In [20]:
nextup_data_dict = {'nextup_instagram':'https://docs.google.com/spreadsheets/d/19eW6Uv1qFeOCms8X63rTN3FJwIgnBbLZ8vhIfW18NLI/edit#gid=2054537359',
                    'nextup_telegram':'https://docs.google.com/spreadsheets/d/19eW6Uv1qFeOCms8X63rTN3FJwIgnBbLZ8vhIfW18NLI/edit#gid=166181969',
                    'nextup_youtube':'https://docs.google.com/spreadsheets/d/19eW6Uv1qFeOCms8X63rTN3FJwIgnBbLZ8vhIfW18NLI/edit#gid=557975443',
                    'nextup_twitch':'https://docs.google.com/spreadsheets/d/19eW6Uv1qFeOCms8X63rTN3FJwIgnBbLZ8vhIfW18NLI/edit#gid=696092184',
                    'nextup_vk':'https://docs.google.com/spreadsheets/d/19eW6Uv1qFeOCms8X63rTN3FJwIgnBbLZ8vhIfW18NLI/edit#gid=1463715721'
                   }

In [21]:
nextup_header_keyword = ['Имя блогера']
nextup_footer_keyword = []

In [22]:
nextup_df = read_csv_from_dict(nextup_data_dict,
                               2,
                               nextup_header_keyword,
                               nextup_footer_keyword
                              )

> Проверим результат

In [23]:
nextup_vk.head()

5,NaN,NaN,Имя блогера,Ссылка,Кол-во подписчиков,Статистика,ЦА по городам,ЦА по возрасту,ЦА по полу,Охват поста за 30 дней,Охват VK клип за 30 дней,Стоимость поста,CPV пост,Стоимость VK клип,CPV Клип,Статистика ссылка
6,NaN,П,Hardplay,https://vk.com/hp_legion,407 000,NaN,NaN,NaN,NaN,10 000,—,72 000 ₽,"7,20 ₽",—,—,NaN
7,NaN,П,Paradeev1ch,https://vk.com/paradeev1ch,414 000,NaN,NaN,NaN,NaN,45 000,-,78 500 ₽,NaN,71 500 ₽,NaN,NaN
8,NaN,П,Маш Милаш,https://vk.com/milash,92 000,NaN,NaN,NaN,NaN,NaN,NaN,79 000 ₽,NaN,71 500 ₽,NaN,NaN
9,NaN,Э,Ира (inst_rinna),https://vk.com/inst_rinnaaaa,113 000,статистика,Москва – 26%\nСПб – 11%\nКраснодар – 4%,"13–17 лет – 17,5%\n18–24 года – 15%\n25–34 год...",Мужской – 68%\nЖенский – 32%,60 000,40 000,72 000 ₽,"1,20 ₽",72 000 ₽,"1,80 ₽",https://drive.google.com/drive/folders/1-3rUVS...
10,NaN,П,Ярослав (Дилблин),https://vk.com/dilblinnn,87 000,статистика,Москва – 21%\nСПб – 11%\nУфа – 4%,13–17 лет – 35%\n18–24 года – 17%\n25–34 года ...,Мужской – 34%\nЖенский – 66%,100 000,–,80 000 ₽,"0,80 ₽",–,–,https://drive.google.com/drive/folders/15UUvEu...


### Блогеры от AvtorMedia

In [24]:
avtormedia_data_dict = {'avtormedia_youtube':'https://docs.google.com/spreadsheets/d/19eW6Uv1qFeOCms8X63rTN3FJwIgnBbLZ8vhIfW18NLI/edit#gid=1144280133',
                        'avtormedia_instagram':'https://docs.google.com/spreadsheets/d/19eW6Uv1qFeOCms8X63rTN3FJwIgnBbLZ8vhIfW18NLI/edit#gid=1039116533',
                        'avtormedia_telegram':'https://docs.google.com/spreadsheets/d/19eW6Uv1qFeOCms8X63rTN3FJwIgnBbLZ8vhIfW18NLI/edit#gid=940855010',
                        #'avtormedia_tiktok':''
                        'avtormedia_vk':'https://docs.google.com/spreadsheets/d/19eW6Uv1qFeOCms8X63rTN3FJwIgnBbLZ8vhIfW18NLI/edit#gid=829776315',
                        'avtormedia_zen':'https://docs.google.com/spreadsheets/d/19eW6Uv1qFeOCms8X63rTN3FJwIgnBbLZ8vhIfW18NLI/edit#gid=1150975360',
                        'avtormedia_podcast':'https://docs.google.com/spreadsheets/d/19eW6Uv1qFeOCms8X63rTN3FJwIgnBbLZ8vhIfW18NLI/edit#gid=483098911'
                       }

In [25]:
avtormedia_header_keyword = ['Блогер']
avtormedia_footer_keyword = []

In [26]:
avtormedia_df = read_csv_from_dict(avtormedia_data_dict,
                                   0,
                                   avtormedia_header_keyword,
                                   avtormedia_footer_keyword
                                  )

> Проверим результат

In [27]:
avtormedia_telegram.head()

,Блогер,Тематика,Подписчики,Материалы\nи статистика,Средние просмотры,Короткое видео,Пост,Спешл,Цена,Спешл,...,Пакетное предложение,Цена,Охват,CPV,Пакетное предложение,Цена,Охват,CPV,Ссылка,Статистика ссылка
1,Илья Варламов @varlamov_news,новости,1 200 000,Ссылка,470 000,833 333 ₽,450 000 ₽,Сторис,500 000 ₽,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://t.me/varlamov_news,https://drive.google.com/drive/u/1/folders/165...
2,Mamix,эксперименты,337 000,Ссылка,261 000,NaN,425 531 ₽,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://t.me/its_mamix,https://drive.google.com/drive/u/1/folders/12S...
3,Илья Варламов @varlamov,урбанистика\nобщество\nполитика,329 000,Ссылка,212 000,888 888 ₽,600 000 ₽,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://t.me/varlamov,https://drive.google.com/drive/u/1/folders/165...
4,Венедиктов,общество\nполитика,223 000,Ссылка,131 000,NaN,250 000 ₽,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://t.me/aavst2022,https://drive.google.com/drive/folders/1UtGUA_...
5,Даша Козловская,творчество\nбьюти\nлайфстайл,186 400,Ссылка,81 000,430 000 ₽,350 000 ₽,NaN,NaN,NaN,...,Фото-пост в Инст + пост в Telegram,1 402 009 ₽,362 000,"3,87",NaN,NaN,NaN,NaN,https://t.me/dashakozlouskayatg,https://drive.google.com/drive/u/1/folders/1cW...


### Блогеры от PLATFORMA.COMMUNITY

In [28]:
sheet_url = 'https://docs.google.com/spreadsheets/d/19eW6Uv1qFeOCms8X63rTN3FJwIgnBbLZ8vhIfW18NLI/edit#gid=356038320'
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
platformacommunity_youtube = pd.read_csv(csv_export_url, header=1)

sheet_url = 'https://docs.google.com/spreadsheets/d/1yWxdd0IlIBWc2XXhVXV9waLnQDNztMEtiMkgJvjZQlw/edit#gid=1230416951'
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
platformacommunity_instagram = pd.read_csv(csv_export_url, header=0)

sheet_url = 'https://docs.google.com/spreadsheets/d/19eW6Uv1qFeOCms8X63rTN3FJwIgnBbLZ8vhIfW18NLI/edit#gid=1919761800'
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
platformacommunity_telegram = pd.read_csv(csv_export_url, header=0)

In [29]:
platformacommunity_youtube.head()

,Вид контракта,Канал,Проект\nВедущий\nФормат,Тематика,Подписчики,Средние просмотры для рекламных интеграций,Статистика,"Pre-roll / OLV, от, руб., до учета налогов","CPV, pre-roll, от, руб., до учета налогов","MID-roll / OLV, от, руб., до учета налогов","Интеграция 1-ый слот, от, руб., до учета налогов","CPV, 1 слот, от, руб., до учета налогов","Интеграция 2-ой слот, от, руб., до учета налогов","CPV, 2 слот, от, руб., до учета налогов","Продакт плейсмент, от, руб., до учета налогов","Размещение в сообществе, от, руб., до учета налогов",Пример,Ссылка,Статистика ссылка
0,Э,Уральские Пельмени,Официальный канал шоу «Уральские Пельмени»,Специальные сборки номеров под рекламодателя,"2 550 000,00","2 000 000,00",Статистика,"1 200 000,00","0,60",NaN,"2 500 000,00","1,25",NaN,NaN,NaN,NaN,Тинькофф,https://www.youtube.com/c/ProPelmeny/featured,https://drive.google.com/drive/folders/1vQIyBd...
1,NaN,NaN,NaN,Эксклюзивные номера,NaN,"1 100 000,00",NaN,"850 000,00","0,77",NaN,NaN,"0,00",NaN,NaN,NaN,NaN,Пятерочка,NaN,NaN
2,NaN,NaN,NaN,Регулярные выкладки,NaN,"700 000,00",NaN,"600 000,00","0,86",NaN,NaN,NaN,NaN,NaN,NaN,NaN,HAVAL,NaN,NaN
3,NaN,NaN,NaN,"""ЕДЕМ С ПЕЛЬМЕНЯМИ"" travel шоу про гастроли и ...",NaN,"300 000,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"800 000,00",NaN,Орматек,NaN,NaN
4,NaN,ВПИСКА,Коля Редькин,Интервью,"1 830 000,00","2 500 000,00",Статистика,"1 500 000,00",NaN,по запросу,по запросу,#VALUE!,NaN,"0,00",по запросу,NaN,NaN,https://www.youtube.com/@VPISKA/featured,https://drive.google.com/drive/folders/1KxM-Mc...


### Блогер ТОПЛЕС

In [30]:
sheet_url = 'https://docs.google.com/spreadsheets/d/1FCGQrwnAbPBh3ZlnmERyl6ZHg1Id4muJFf1Dik5hmjw/edit#gid=0'
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
toples_telegram = pd.read_csv(csv_export_url, header=0)

sheet_url = 'https://docs.google.com/spreadsheets/d/1FCGQrwnAbPBh3ZlnmERyl6ZHg1Id4muJFf1Dik5hmjw/edit#gid=1286799283'
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
toples_instagram = pd.read_csv(csv_export_url, header=0)

sheet_url = 'https://docs.google.com/spreadsheets/d/1FCGQrwnAbPBh3ZlnmERyl6ZHg1Id4muJFf1Dik5hmjw/edit#gid=1864371720'
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
toples_vk = pd.read_csv(csv_export_url, header=0)

In [31]:
toples_telegram.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Telegram-канал Топлес,NaN,NaN,NaN,NaN
2,NaN,https://t.me/toplesofficial,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Подписчики: 500К,NaN,NaN,NaN,NaN
5,NaN,Охват поста: 120-130К,NaN,NaN,NaN,Условия актуальны до 30.11.2023
6,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,Формат публикации,NaN,NaN,Подробности формата,Стоимость (от)
8,NaN,1,Текстовый пост\n\nМатериалы клиента,- текст от клиента\n- визуал от клиента (по не...,"Материалы клиента, при необходимости адаптируе...",р.85 000
9,NaN,2,Текстовый пост\n\nПишем сами по брифу клиента,- текст по брифу\n- визуал от клиента (по необ...,Текст пишем в стиле канала по брифу клиента. Т...,р.100 000


## Подготовка данных


#### Функция, с помощью которой мы предварительно подготовим DataFrame для дальнейшей работы

In [32]:
def pretraining_df(df: pd.DataFrame,
                   drop_keywords: str,
                   drop_word: str,
                   upd_columns: str) -> pd.DataFrame:

    """
    Функция принимает DataFrame - df, с которым мы делаем преобразования:
    • Редактируем названия столбцов: приводим к нижнему регистру, убираем лишние пробелы до и после текста, удаляем перенос \n и сноски *.
    • Удаляем столбцы по ключевым словам drop_keywords. В коде учтено, что их может и не быть.
    • Удаляем ключевые слова drop_word из названия столбцов. В коде учтено, что их может и не быть.
    • Столбцы upd_columns, в которых содержится текстовая информация для дальнейшего объединения разных DataFrame,
    приводим к единому формату по примеру первого пункта, а также заполняем NaN с помощью 

    
    ПАРАМЕТРЫ:
    df (DataFrame) - DataFrame, в котором мы будем вносить изменения
    drop_keywords (str) - список ключевых слов для удаления столбцов
    drop_word (str) - слово для удаления из названий столбцов
    upd_columns (str) - список имен столбцов для приведения к нижнему регистру и удаления символов
    
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    df - DataFrame, в котором мы делали преобразования
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    key_word = [] # Список ключевых слов для поиска столбцов и их удаления
    drop_word = ' до ндс' # Список ключевых слов для их удаления из названия столбцов
    upd_columns = ['сообщество', 'шоу'] # Список столбцов с именами блогеров для приведения к единому формату
    upd_columns = [column.lower() for column in upd_columns]

    vk_bloggers = pretraining_df(vk_bloggers, key_word, drop_word, upd_columns)
    
    """
    
    drop_keywords = [keywords.lower() for keywords in drop_keywords] #
    drop_keywords = [keyword.strip() for keyword in drop_keywords]
    drop_word = drop_word.lower()
    upd_columns = [column.lower() for column in upd_columns] #
    
    
    df.columns = df.columns.str.strip()\
                            .str.lower()\
                            .str.replace('/\n', ', ')\
                            .str.replace('\n', ', ')\
                            .str.replace('*', '')
    
# Найдём столбцы по ключевым словам и их удалим их, так как в DataFrame у нас встречаются столбцы,
# которые не несут уникальной информации, например cpm, cpv и тд,
# и которые мы можем вычислить точнее через математические операции.


    if drop_keywords:
        columns_to_drop = [column for column in df.columns if isinstance(column, str) and any(keyword in column for keyword in drop_keywords)]
        df.drop(columns = columns_to_drop, axis=1, inplace=True)

        
    
# Удалим ключевые слова из названия столбцов
    if drop_word:
        df.columns = df.columns.str.replace(drop_word, '', regex=False)


# Столбцы, в которых содержится информация для дальнейшего объединения, приведём к единому формату
    for upd_column in upd_columns:
        df[upd_column] = df[upd_column].fillna('').apply(lambda x: str(x).strip().lower()\
                                                                         .replace('/\n', ', ')\
                                                                         .replace('\n', ', ')\
                                                                         .replace('*', '')\
                                                             if isinstance(x, str) else x)
        # Заменим пустую строку на NaN
        df[upd_column] = df[upd_column].replace('', np.nan)
    
    return df

## Подготовка данных по блогерам от Didenok Team

#### Имеющиеся датафреймы:  

* didenok_instagram  
* didenok_youtube  
* didenok_telegram  
* didenok_vk_zen_show  
* didenok_vk_group  
<!-- * didenok_vk_ls   -->
* didenok_twitch  

> Сделаем список DataFrame, чтобы в дальнейшем работать с ним

In [33]:
didenok_df = [didenok_instagram,
              didenok_youtube,
              didenok_telegram,
              didenok_vk_zen_show,
              didenok_vk_group,
#               didenok_vk_ls,
              didenok_twitch]

> Так как до предобработки названий столбцов DataFrame название столбца с именем блогера может быть как "Блогер", так и "Канал", учтём это в цикле ниже для запуска функции pretraining_df

In [34]:
key_word = ['вид сотрудничества', 'cpv', 'cpu', 'err']  # Список ключевых слов для поиска столбцов и их удаления
drop_word = ' (руб.) до ндс'  # Список ключевых слов для их удаления из названия столбцов

for i, df in enumerate(didenok_df):
    if 'Блогер' in df.columns:
        upd_columns = ['блогер']  # Столбец с именами блогеров для приведения к единому формату
        pretraining_df(df, key_word, drop_word, upd_columns)
    elif 'Канал' in df.columns:
        upd_columns = ['канал']  # Столбец с именами блогеров для приведения к единому формату
        pretraining_df(df, key_word, drop_word, upd_columns)
    else:
        print(f'Не удалось найти столбец "Блогер" или "Канал" для преобразования DataFrame {i}') 

> Проверим результат

In [35]:
didenok_youtube.head()

7,канал,ссылка,"категория, тематика канала",количество подписчиков,среднее кол-во просмотров,среднее кол-во просмотров shorts,среднее кол-во лайков,среднее кол-во комментариев,стоимость за интеграцию до 1 мин,стоимость за продакт плейсмент,стоимость за преролл до 30 сек,стоимость за мидролл до 30 сек,стоимость за shorts до 1 мин,статистика,статистика ссылка
8,влад бумага а4,https://www.youtube.com/@A4a4a4a4,юмор/челленджи/развлечения,47 100 000,8 000 000,-,350 000,30 000,р.4 375 000,-,р.1 625 000,р.1 625 000,-,Статистика,https://drive.google.com/drive/u/1/folders/1Z-...
9,влад глент,https://www.youtube.com/@GLENTYOUTUBE,юмор/челленджи/гейминг,16 300 000,1 800 000,-,70 000,4 000,р.1 250 000,-,р.690 000,р.690 000,-,Статистика,https://drive.google.com/drive/folders/1N165vD...
10,watchme,https://www.youtube.com/@WatchMeOff,дети/подростки,11 900 000,90 000,-,1 000,-,р.200 000,р.145 000,р.125 000,р.125 000,-,Статистика,https://yadi.sk/d/BYF3WbphVjtT-Q
11,vikishow,https://www.youtube.com/@VikiShow,дети/подростки,11 200 000,1 500 000,200 000,50 000,4 000,р.700 000,р.440 000,р.440 000,р.350 000,р.440 000,Статистика,https://drive.google.com/drive/folders/1_CHbok...
12,fixeye,https://www.youtube.com/@FixEye,гейминг,10 300 000,400 000,-,30 000,500,р.350 000,р.240 000,р.240 000,р.210 000,-,Статистика,https://drive.google.com/drive/folders/1VTEv65...


### Подготовка таблицы didenok_instagram

In [36]:
[col for col in didenok_instagram.columns]

['блогер',
 'ссылка',
 'категория, тематика блогера',
 'количество подписчиков',
 'средний охват фотопоста',
 'средний охват 1 сторис',
 'среднее кол-во просмотров reels',
 'среднее кол-во лайков',
 'среднее кол-во комментариев',
 'стоимость за фотопост',
 'стоимость за 1 сторис до 15 сек',
 'стоимость за серию из 3х сториc до 45 сек',
 'стоимость за reels',
 'статистика',
 'статистика']

In [37]:
bufer_columns = ['блогер',
                 'ссылка',
                 'тематика',
                 'количество подписчиков',
                 'среднее кол-во просмотров фотопоста',
                 'среднее кол-во просмотров 1 сторис',
                 'среднее кол-во просмотров reels / видеопоста',
                 'среднее кол-во лайков',
                 'среднее кол-во комментариев',
                 'стоимость фотопост',
                 'стоимость 1 сторис до 15 сек',
                 'стоимость серия из 3х сторис до 45 сек',
                 'стоимость reels / видеопост',
                 'статистика удалить',
                 'статистика']

In [38]:
didenok_instagram.columns = bufer_columns

In [39]:
didenok_instagram.head()

,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров фотопоста,среднее кол-во просмотров 1 сторис,среднее кол-во просмотров reels / видеопоста,среднее кол-во лайков,среднее кол-во комментариев,стоимость фотопост,стоимость 1 сторис до 15 сек,стоимость серия из 3х сторис до 45 сек,стоимость reels / видеопост,статистика удалить,статистика
8,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,100,р.375 000,р.335 000,р.530 000,р.820 000,Статистика,https://drive.google.com/drive/folders/1J26W2R...
9,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,700,р.875 000,р.440 000,р.625 000,р.1 875 000,-,NaN
10,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,100,р.445 000,р.305 000,р.625 000,р.685 000,Статистика,https://drive.google.com/drive/u/1/folders/11S...
11,амина тендерлибае,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,150,р.695 000,р.265 000,р.625 000,р.835 000,Статистика,https://drive.google.com/drive/u/1/folders/1M3...
12,vikishow,https://www.instagram.com/vikishow_official/,лайфстайл/дети/подростки,2 500 000,70 000,30 000,80 000,15 000,200,р.120 000,р.60 000,р.135 000,р.155 000,Статистика,https://drive.google.com/drive/folders/1haDuDs...


> Удалим столбец со статистикой, в котором не высветились гиперссылки

In [40]:
didenok_instagram.drop(columns = ['статистика удалить'], axis=1, inplace=True)

> Удалим первую строку, которая не показывает что-то по блогерам, а являестся побочной строкой названия столбцом.  
> <i>[Она то появляется, то исчезает, поэтому оставим как удаление пропуска]</i>

In [41]:
didenok_instagram.drop(didenok_instagram[didenok_instagram['блогер'] == ''].index, inplace=True)

In [42]:
didenok_instagram.head()

,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров фотопоста,среднее кол-во просмотров 1 сторис,среднее кол-во просмотров reels / видеопоста,среднее кол-во лайков,среднее кол-во комментариев,стоимость фотопост,стоимость 1 сторис до 15 сек,стоимость серия из 3х сторис до 45 сек,стоимость reels / видеопост,статистика
8,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,100,р.375 000,р.335 000,р.530 000,р.820 000,https://drive.google.com/drive/folders/1J26W2R...
9,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,700,р.875 000,р.440 000,р.625 000,р.1 875 000,NaN
10,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,100,р.445 000,р.305 000,р.625 000,р.685 000,https://drive.google.com/drive/u/1/folders/11S...
11,амина тендерлибае,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,150,р.695 000,р.265 000,р.625 000,р.835 000,https://drive.google.com/drive/u/1/folders/1M3...
12,vikishow,https://www.instagram.com/vikishow_official/,лайфстайл/дети/подростки,2 500 000,70 000,30 000,80 000,15 000,200,р.120 000,р.60 000,р.135 000,р.155 000,https://drive.google.com/drive/folders/1haDuDs...


> Добавим указание на название соц.сети в название столбцов,чтобы в дальнейшем правильно объединять и искать нужные столбцы, в том числе в боте

##### Функция для указания названия соц.сети в названии столбцов

In [43]:
def type_social_network(df: pd.DataFrame, social_network: str) -> pd.DataFrame:
    
    
    """
    Функция принимает DataFrame - df, в название столбцов которого мы добавим название соц сети social_network
    
    ПАРАМЕТРЫ:
    df - DataFrame, в котором делаем преобразования.
    social_network - название соц. сети, которое мы добавим в название столбцов
    
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    df (DataFrame) с обновлёнными названиями столбцов
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    type_social_network(didenok_instagram, 'Instagram')
    """
    
    bufer_lst = []
    
    for element in df.columns.tolist():
        bufer_lst.append(f'{social_network} ' + element)
        
    df.columns = bufer_lst
    
    return df

In [44]:
type_social_network(didenok_instagram, 'Instagram').head()

,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,Instagram среднее кол-во комментариев,Instagram стоимость фотопост,Instagram стоимость 1 сторис до 15 сек,Instagram стоимость серия из 3х сторис до 45 сек,Instagram стоимость reels / видеопост,Instagram статистика
8,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,100,р.375 000,р.335 000,р.530 000,р.820 000,https://drive.google.com/drive/folders/1J26W2R...
9,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,700,р.875 000,р.440 000,р.625 000,р.1 875 000,NaN
10,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,100,р.445 000,р.305 000,р.625 000,р.685 000,https://drive.google.com/drive/u/1/folders/11S...
11,амина тендерлибае,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,150,р.695 000,р.265 000,р.625 000,р.835 000,https://drive.google.com/drive/u/1/folders/1M3...
12,vikishow,https://www.instagram.com/vikishow_official/,лайфстайл/дети/подростки,2 500 000,70 000,30 000,80 000,15 000,200,р.120 000,р.60 000,р.135 000,р.155 000,https://drive.google.com/drive/folders/1haDuDs...


### Подготовка таблицы didenok_youtube

In [45]:
didenok_youtube.head()

7,канал,ссылка,"категория, тематика канала",количество подписчиков,среднее кол-во просмотров,среднее кол-во просмотров shorts,среднее кол-во лайков,среднее кол-во комментариев,стоимость за интеграцию до 1 мин,стоимость за продакт плейсмент,стоимость за преролл до 30 сек,стоимость за мидролл до 30 сек,стоимость за shorts до 1 мин,статистика,статистика ссылка
8,влад бумага а4,https://www.youtube.com/@A4a4a4a4,юмор/челленджи/развлечения,47 100 000,8 000 000,-,350 000,30 000,р.4 375 000,-,р.1 625 000,р.1 625 000,-,Статистика,https://drive.google.com/drive/u/1/folders/1Z-...
9,влад глент,https://www.youtube.com/@GLENTYOUTUBE,юмор/челленджи/гейминг,16 300 000,1 800 000,-,70 000,4 000,р.1 250 000,-,р.690 000,р.690 000,-,Статистика,https://drive.google.com/drive/folders/1N165vD...
10,watchme,https://www.youtube.com/@WatchMeOff,дети/подростки,11 900 000,90 000,-,1 000,-,р.200 000,р.145 000,р.125 000,р.125 000,-,Статистика,https://yadi.sk/d/BYF3WbphVjtT-Q
11,vikishow,https://www.youtube.com/@VikiShow,дети/подростки,11 200 000,1 500 000,200 000,50 000,4 000,р.700 000,р.440 000,р.440 000,р.350 000,р.440 000,Статистика,https://drive.google.com/drive/folders/1_CHbok...
12,fixeye,https://www.youtube.com/@FixEye,гейминг,10 300 000,400 000,-,30 000,500,р.350 000,р.240 000,р.240 000,р.210 000,-,Статистика,https://drive.google.com/drive/folders/1VTEv65...


> Переименуем столбцы

In [46]:
[col for col in didenok_youtube.columns]

['канал',
 'ссылка',
 'категория, тематика канала',
 'количество подписчиков',
 'среднее кол-во просмотров',
 'среднее кол-во просмотров shorts',
 'среднее кол-во лайков',
 'среднее кол-во комментариев',
 'стоимость за интеграцию до 1 мин',
 'стоимость за продакт плейсмент',
 'стоимость за преролл до 30 сек',
 'стоимость за мидролл до 30 сек',
 'стоимость за shorts до 1 мин',
 'статистика',
 'статистика ссылка']

> У нас есть столбцы с возрастом и полу в процентном соотношении - объединим их в единый столбец.  
> <i>[Данные столбцы то появляются, то исчезают, поэтому на будущее оставим код]</i>

In [47]:
try:
    bufer_columns = ['блогер',
                     'ссылка',
                     'тематика',
                     'количество подписчиков',
                     'среднее кол-во просмотров',
                     'среднее кол-во просмотров shorts',
                     'среднее кол-во лайков',
                     'среднее кол-во комментариев',
                     'стоимость интеграция 1 слот',
                     'стоимость продакт плейсмент',
                     'стоимость преролл до 30 сек',
                     'стоимость мидролл до 30 сек',
                     'стоимость shorts до 1 мин',
                     'статистика удалить',
                     'м',
                     'ж',
                     '13-17',
                     '18-24',
                     '25-34',
                     '35-44',
                     '45-54',
                     '55+',
                     'ца гео рф',
                     'статистика ссылка']
    
    didenok_youtube.columns = bufer_columns
    
    didenok_youtube['ца по возрасту'] = didenok_youtube.apply(lambda row:
                                                              f"13-17 - {row['13-17']}, " +
                                                              f"18-24 - {row['18-24']}, " +
                                                              f"25-34 - {row['25-34']}, " +
                                                              f"35-44 - {row['35-44']}, " +
                                                              f"45-54 - {row['45-54']}" +
                                                              f"55+ - {row['55+']}", axis=1)
    
    didenok_youtube.drop(columns=['13-17',
                              '18-24',
                              '25-34',
                              '35-44',
                              '45-54',
                              '55+'], axis=1, inplace=True)
    didenok_youtube['ца по полу'] = didenok_youtube.apply(lambda row:
    f"Мужской - {row['м']}, " +
    f"Женской - {row['ж']}", axis=1)
    didenok_youtube.drop(columns = ['м', 'ж'], axis=1, inplace=True)
    
    
except:
    bufer_columns = ['блогер',
                     'ссылка',
                     'тематика',
                     'количество подписчиков',
                     'среднее кол-во просмотров видео',
                     'среднее кол-во просмотров shorts',
                     'среднее кол-во лайков',
                     'среднее кол-во комментариев',
                     'стоимость интеграция 1 слот',
                     'стоимость продакт плейсмент',
                     'стоимость преролл до 30 сек',
                     'стоимость мидролл до 30 сек',
                     'стоимость shorts до 1 мин',
                     'статистика удалить',
                     'статистика ссылка']
    
    didenok_youtube.columns = bufer_columns

In [48]:
didenok_youtube.drop(columns = ['статистика удалить'], axis=1, inplace=True)
didenok_youtube.rename(columns = {'статистика ссылка':'статистика'}, inplace=True)

In [49]:
didenok_youtube.head()

,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров видео,среднее кол-во просмотров shorts,среднее кол-во лайков,среднее кол-во комментариев,стоимость интеграция 1 слот,стоимость продакт плейсмент,стоимость преролл до 30 сек,стоимость мидролл до 30 сек,стоимость shorts до 1 мин,статистика
8,влад бумага а4,https://www.youtube.com/@A4a4a4a4,юмор/челленджи/развлечения,47 100 000,8 000 000,-,350 000,30 000,р.4 375 000,-,р.1 625 000,р.1 625 000,-,https://drive.google.com/drive/u/1/folders/1Z-...
9,влад глент,https://www.youtube.com/@GLENTYOUTUBE,юмор/челленджи/гейминг,16 300 000,1 800 000,-,70 000,4 000,р.1 250 000,-,р.690 000,р.690 000,-,https://drive.google.com/drive/folders/1N165vD...
10,watchme,https://www.youtube.com/@WatchMeOff,дети/подростки,11 900 000,90 000,-,1 000,-,р.200 000,р.145 000,р.125 000,р.125 000,-,https://yadi.sk/d/BYF3WbphVjtT-Q
11,vikishow,https://www.youtube.com/@VikiShow,дети/подростки,11 200 000,1 500 000,200 000,50 000,4 000,р.700 000,р.440 000,р.440 000,р.350 000,р.440 000,https://drive.google.com/drive/folders/1_CHbok...
12,fixeye,https://www.youtube.com/@FixEye,гейминг,10 300 000,400 000,-,30 000,500,р.350 000,р.240 000,р.240 000,р.210 000,-,https://drive.google.com/drive/folders/1VTEv65...


> Удалим пустые строки

In [50]:
didenok_youtube.drop(didenok_youtube[didenok_youtube['блогер'] == ''].index, inplace=True)

In [51]:
didenok_youtube.head()

,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров видео,среднее кол-во просмотров shorts,среднее кол-во лайков,среднее кол-во комментариев,стоимость интеграция 1 слот,стоимость продакт плейсмент,стоимость преролл до 30 сек,стоимость мидролл до 30 сек,стоимость shorts до 1 мин,статистика
8,влад бумага а4,https://www.youtube.com/@A4a4a4a4,юмор/челленджи/развлечения,47 100 000,8 000 000,-,350 000,30 000,р.4 375 000,-,р.1 625 000,р.1 625 000,-,https://drive.google.com/drive/u/1/folders/1Z-...
9,влад глент,https://www.youtube.com/@GLENTYOUTUBE,юмор/челленджи/гейминг,16 300 000,1 800 000,-,70 000,4 000,р.1 250 000,-,р.690 000,р.690 000,-,https://drive.google.com/drive/folders/1N165vD...
10,watchme,https://www.youtube.com/@WatchMeOff,дети/подростки,11 900 000,90 000,-,1 000,-,р.200 000,р.145 000,р.125 000,р.125 000,-,https://yadi.sk/d/BYF3WbphVjtT-Q
11,vikishow,https://www.youtube.com/@VikiShow,дети/подростки,11 200 000,1 500 000,200 000,50 000,4 000,р.700 000,р.440 000,р.440 000,р.350 000,р.440 000,https://drive.google.com/drive/folders/1_CHbok...
12,fixeye,https://www.youtube.com/@FixEye,гейминг,10 300 000,400 000,-,30 000,500,р.350 000,р.240 000,р.240 000,р.210 000,-,https://drive.google.com/drive/folders/1VTEv65...


> У нас есть проблема в том, что часть по одному и тому же блогеру выведены в отдельные строки как 1 и 2 слот

In [52]:
didenok_youtube.loc[(didenok_youtube['блогер'].str.contains('1 слот')) | (didenok_youtube['блогер'].str.contains('2 слот'))]

,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров видео,среднее кол-во просмотров shorts,среднее кол-во лайков,среднее кол-во комментариев,стоимость интеграция 1 слот,стоимость продакт плейсмент,стоимость преролл до 30 сек,стоимость мидролл до 30 сек,стоимость shorts до 1 мин,статистика
14,катя адушкина (1 слот),https://www.youtube.com/@KatyaAdushkinaa,лайфстайл,5 390 000,500 000,4 000,20 000,500,р.640 000,р.400 000,р.440 000,р.370 000,р.250 000,https://drive.google.com/drive/folders/1aSmonK...
15,катя адушкина (2 слот),https://www.youtube.com/@KatyaAdushkinaa,лайфстайл,5 390 000,500 000,4 000,20 000,500,р.490 000,-,-,-,-,https://drive.google.com/drive/folders/1aSmonK...
16,supercrastan (1 слот),https://www.youtube.com/user/SuperCrastan,научпоп/DIY/шоу,3 430 000,750 000,40 000,7 000,500,р.980 000,-,р.735 000,р.615 000,р.735 000,https://drive.google.com/drive/folders/1dVEwSt...
17,supercrastan (2 слот),https://www.youtube.com/user/SuperCrastan,научпоп/DIY/шоу,3 430 000,750 000,40 000,7 000,500,р.860 000,-,-,-,-,https://drive.google.com/drive/folders/1dVEwSt...
21,контакты (1 слот),https://www.youtube.com/@shastoon.channel,шоу/юмор/развлечения,2 560 000,450 000,-,20 000,1 000,р.950 000,-,р.500 000,р.400 000,-,https://drive.google.com/drive/u/1/folders/1dz...
22,контакты (2 слот),https://www.youtube.com/@shastoon.channel,шоу/юмор/развлечения,2 560 000,450 000,-,20 000,1 000,р.800 000,-,р.500 000,р.400 000,-,NaN
23,цитаты (1 слот),https://www.youtube.com/@shastoon.channel,шоу/юмор/развлечения,2 560 000,100 000,-,7 000,500,р.650 000,-,р.400 000,р.300 000,-,NaN
24,цитаты (2 слот),https://www.youtube.com/@shastoon.channel,шоу/юмор/развлечения,2 560 000,100 000,-,7 000,500,р.500 000,-,р.400 000,р.300 000,-,NaN
25,не игры (1 слот),https://www.youtube.com/@shastoon.channel,шоу/юмор/развлечения,2 560 000,500 000,-,30 000,1 000,р.950 000,-,р.500 000,р.400 000,-,NaN
26,не игры (2 слот),https://www.youtube.com/@shastoon.channel,шоу/юмор/развлечения,2 560 000,500 000,-,30 000,1 000,р.800 000,-,р.500 000,р.400 000,-,NaN


> Исправим данную проблему, переведя данные цены в отдельный столбец и объединив строки

In [53]:
def new_position_in_columns(df: pd.DataFrame,
                            columns_with_name: str,
                            columns_with_position: str) -> pd.DataFrame:
    
    """
    ОПИСАНИЕ РАБОТЫ:
        Функция выполняет определенные преобразования в DataFrame, связанные с именами блогеров,
        их позициями и стоимостью за интеграцию, а именно объединяем 1 и 2 слот
        по одному и тому же блогеры в единую строку

    ПАРАМЕТРЫ:
        - df (pd.DataFrame): DataFrame, в котором выполняются преобразования.
        - columns_with_name (str): Название столбца, содержащего имена блогеров.
        - columns_with_position (str): Название столбца, содержащего информацию о позиции блогера и стоимости за интеграцию.

    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
        - pd.DataFrame: Измененный DataFrame после выполнения преобразований.

    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
        didenok_youtube = new_position_in_columns(didenok_youtube, 'блогер', 'стоимость за интеграцию до 1 мин')
    """
    
    df[['блогер upd', 'слот']] = df[columns_with_name].str.extract(r'(.+)\s?\((\d+ слот)?\)')
    df['блогер upd'] = df['блогер upd'].fillna(df[columns_with_name])
    
    rows_to_delete = []

# Итерируемся по строкам DataFrame
    for index, row in df.iterrows():
        if '2 слот' in row['блогер']:
            blogger_name = row['блогер'].split(' (')[0]
        # Проверяем, есть ли строка с '1 слот' для этого блогера
            if any(df[(df['блогер'] != row['блогер']) & (df['блогер'].str.contains(blogger_name))]['блогер'].str.contains('1 слот')):
            # Находим индексы строк с совпадающим именем блогера
                matching_indices = df[df['блогер'].str.contains(blogger_name)].index
            # Обновляем данные в столбце 'стоимость за интеграцию 2 слот до 1 мин'
                df.loc[matching_indices, f'{columns_with_position} 2 слот'] = row[columns_with_position]
            # Добавляем индекс строки для удаления
                rows_to_delete.append(index)

# Удаляем строки с 2 слотом
    df.drop(rows_to_delete, inplace=True)
    
    df.drop(columns=['слот', 'блогер'], inplace=True)

# Переименовываем столбец 'блогер upd' в 'блогер'
    df.rename(columns={'блогер upd': 'блогер'}, inplace=True)

# Отчисти имена блогеров от лишних пробелов по бокам имени
    df['блогер'] = df['блогер'].str.strip()

    df.reset_index(drop=True, inplace=True)
    
    return df

In [54]:
didenok_youtube = new_position_in_columns(didenok_youtube, 'блогер', 'стоимость интеграция 1 слот')

In [55]:
# Проверим результат

didenok_youtube.loc[(didenok_youtube['блогер'].str.contains('1 слот')) | (didenok_youtube['блогер'].str.contains('2 слот'))]

,ссылка,тематика,количество подписчиков,среднее кол-во просмотров видео,среднее кол-во просмотров shorts,среднее кол-во лайков,среднее кол-во комментариев,стоимость интеграция 1 слот,стоимость продакт плейсмент,стоимость преролл до 30 сек,стоимость мидролл до 30 сек,стоимость shorts до 1 мин,статистика,блогер,стоимость интеграция 1 слот 2 слот


In [56]:
didenok_youtube.rename(columns = {'стоимость интеграция 1 слот 2 слот':'стоимость интеграция 2 слот'
                                  }, inplace=True)

### Нулевое объединение

> Сейчас мы объединим не два DataFrame из источников выше, а сделаем первый DataFrame, который станет основой для будущих объединений.  
> Перед объединением исправим неточность в наименовании блогера.

In [57]:
didenok_instagram.loc\
    [didenok_instagram['Instagram блогер'] == 'амина тендерлибае', 'Instagram блогер']  = 'амина tenderlybae'

didenok_instagram.loc\
    [didenok_instagram['Instagram блогер'] == 'helenavi', 'Instagram блогер']  = 'helenavi и gopronin'

> У нас есть два столбца, которые нам отдельно важны - это Никнейм блогера и его Тематика. Создадим единый DataFrame с такими столбцами, которые будем заполнять по мере объединения столбцов

In [58]:
bufer = pd.DataFrame(columns=['блогер','тематика'])
bufer

,блогер,тематика


> Объединим два DataFrame по вертикали с помощью метода concat (добавляет строки из didenok_instagram в bufer, пропуски в bufer в дальнейшем заполним)

In [59]:
bufer = pd.concat([bufer, didenok_instagram], ignore_index=True)

In [60]:
bufer.head()

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,Instagram среднее кол-во комментариев,Instagram стоимость фотопост,Instagram стоимость 1 сторис до 15 сек,Instagram стоимость серия из 3х сторис до 45 сек,Instagram стоимость reels / видеопост,Instagram статистика
0,NaN,NaN,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,100,р.375 000,р.335 000,р.530 000,р.820 000,https://drive.google.com/drive/folders/1J26W2R...
1,NaN,NaN,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,700,р.875 000,р.440 000,р.625 000,р.1 875 000,NaN
2,NaN,NaN,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,100,р.445 000,р.305 000,р.625 000,р.685 000,https://drive.google.com/drive/u/1/folders/11S...
3,NaN,NaN,амина tenderlybae,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,150,р.695 000,р.265 000,р.625 000,р.835 000,https://drive.google.com/drive/u/1/folders/1M3...
4,NaN,NaN,vikishow,https://www.instagram.com/vikishow_official/,лайфстайл/дети/подростки,2 500 000,70 000,30 000,80 000,15 000,200,р.120 000,р.60 000,р.135 000,р.155 000,https://drive.google.com/drive/folders/1haDuDs...


> Те пропуски в данных двух столбцах, что мы имеем сейчас, мы будем заполнять с помощью похожих столбцов из присоединённых датафреймов.  
  
> Сделаем для этого отдельную функцию

#### Функция, с помощью которой мы заполним Имя и Тематику блогера в едином DataFrame

In [61]:
def bufer_name_category(bufer: pd.DataFrame, social_network: str) -> pd.DataFrame:
    
    """
    Функция принимает DataFrame - bufer, столбцы 'блогер' и 'тематика' которого мы заполняем с помощью данных из новых колонок
    '{social_network} блогер' и '{social_network} тематика'.
    Учитываем, что столбец 'тематика' не всегда есть в присоединяемом столбце

    
    ПАРАМЕТРЫ:
    bufer (DataFrame) - единый DataFrame, к которому мы присоединяем другие DataFrame
    social_network (str) - название соц сети, по которому мы будем искать вновь присоединённые колонки
    
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    DataFrame - bufer, в котором мы заполнили столбцы 'блогер' и 'тематика'
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    bufer = bufer_name_category(bufer, 'Instagram')
    """
    
    bufer['блогер'] = bufer['блогер'].fillna(bufer[f'{social_network} блогер'])
    
    # Учтём, что столбец 'тематика' не всегда есть в присоединяемом столбце

    if f'{social_network} тематика' in bufer.columns:
        bufer['тематика'] = bufer['тематика'].fillna(bufer[f'{social_network} тематика'])
    
    return bufer

In [62]:
bufer = bufer_name_category(bufer, 'Instagram')

In [63]:
bufer.head()

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,Instagram среднее кол-во комментариев,Instagram стоимость фотопост,Instagram стоимость 1 сторис до 15 сек,Instagram стоимость серия из 3х сторис до 45 сек,Instagram стоимость reels / видеопост,Instagram статистика
0,катя клэп,лайфстайл/бьюти,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,100,р.375 000,р.335 000,р.530 000,р.820 000,https://drive.google.com/drive/folders/1J26W2R...
1,влад бумага а4,челленджи,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,700,р.875 000,р.440 000,р.625 000,р.1 875 000,NaN
2,катя адушкина,музыка/лайфстайл/танцы,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,100,р.445 000,р.305 000,р.625 000,р.685 000,https://drive.google.com/drive/u/1/folders/11S...
3,амина tenderlybae,твич/лайфстайл,амина tenderlybae,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,150,р.695 000,р.265 000,р.625 000,р.835 000,https://drive.google.com/drive/u/1/folders/1M3...
4,vikishow,лайфстайл/дети/подростки,vikishow,https://www.instagram.com/vikishow_official/,лайфстайл/дети/подростки,2 500 000,70 000,30 000,80 000,15 000,200,р.120 000,р.60 000,р.135 000,р.155 000,https://drive.google.com/drive/folders/1haDuDs...


### Первое объединение

#### Функция для объединения DataFrame

> Так как мы будем объединять DataFrame, в которых как могут быть общие столбцы, так и нет, то для корректной работы merge мы напишем специальную функцию, которая будет как указывать в названии новых DataFrame название соц. сети, объединять по общим столбцам, а если их нет, то по имени блогеров. Также отдельно мы будем выводить количество дубликатов в объединённом DataFrame

In [64]:
def merge_bufer_df(bufer: pd.DataFrame,
                   df: pd.DataFrame,
                   social_network: str) -> pd.DataFrame:
    
    """
    Функция добавляет название соц сети в название столбцов с помощью ключа social_network,
    далее делает объединение по двум сценариям:
    1) У нас есть общие столбцы с учётом добавления названия соц сети - тогда объединяет по ним с помощью merge.
    2) У нас нет общих столбов - тогда объединяет по имени блогера из bufer['блогер'] и нового DataFrame.
    
    Далее функция заполняет столбцы 'блогер' и 'тематика' в едином DataFrame с учётом того,
    что 'тематика' не всегда есть в присоединяемом DataFrame.
    
    ПАРАМЕТРЫ:
    - bufer (DataFrame): наш единый DataFrame, к которому мы присоединяем новый DataFrame (df)
    - df (DataFrame): DataFrame, который мы будем присоединять
    - social_network (str): Ключ-название соц сети, по которому мы будем искать столбцы с именем и тематикой блогеров, кототорые присоединили

    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    merged_bufer: единый DataFrame, в который мы объединили два DataFrame
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    bufer = merge_bufer_df(bufer, didenok_youtube, 'YouTube')

    """
    
    list_columns_no_social_network = []
    
    for element in df.columns.tolist():
        list_columns_no_social_network.append(f'{social_network} ' + element)
        
    df.columns = list_columns_no_social_network
    
    common_columns = bufer.columns.intersection(df.columns)
    
# В Pandas нельзя использовать if для проверки истинности объекта Index, поскольку он считается неоднозначным.
# Вместо этого мы будем использовать if not common_columns.empty: для проверки, является ли common_columns пустым или нет.
    
    if not common_columns.empty:
        merged_bufer = bufer.merge(df,
                                   left_on = common_columns.tolist(),
                                   right_on = common_columns.tolist(),
                                   how = 'outer')
    
    else:
        merged_bufer = bufer.merge(df, left_on='блогер', right_on=f'{social_network} блогер', how='outer')

    
    merged_bufer['блогер'] = merged_bufer['блогер'].fillna(merged_bufer[f'{social_network} блогер'])

# Учтём, что столбец 'тематика' не всегда есть в присоединяемом столбце
    if f'{social_network} тематика' in merged_bufer.columns:
        merged_bufer['тематика'] = merged_bufer['тематика'].fillna(merged_bufer[f'{social_network} тематика'])
        
        
    duplicates_merged_bufer = merged_bufer[merged_bufer.duplicated('блогер', keep=False)]

    # Чтобы посчитать количество дубликатов, можно использовать len() или метод .shape
    count_duplicates = len(duplicates_merged_bufer)  # или duplicates.shape[0]

    print("Количество дубликатов:", count_duplicates)
    
    return merged_bufer

In [65]:
bufer = merge_bufer_df(bufer, didenok_youtube, 'YouTube')

Количество дубликатов: 0


In [66]:
bufer.head()

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,YouTube среднее кол-во лайков,YouTube среднее кол-во комментариев,YouTube стоимость интеграция 1 слот,YouTube стоимость продакт плейсмент,YouTube стоимость преролл до 30 сек,YouTube стоимость мидролл до 30 сек,YouTube стоимость shorts до 1 мин,YouTube статистика,YouTube блогер,YouTube стоимость интеграция 2 слот
0,катя клэп,лайфстайл/бьюти,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,...,60 000,1 000,р.1 330 000,-,-,-,-,https://drive.google.com/drive/folders/1LU4q-1...,катя клэп,NaN
1,влад бумага а4,челленджи,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,...,350 000,30 000,р.4 375 000,-,р.1 625 000,р.1 625 000,-,https://drive.google.com/drive/u/1/folders/1Z-...,влад бумага а4,NaN
2,катя адушкина,музыка/лайфстайл/танцы,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,...,20 000,500,р.640 000,р.400 000,р.440 000,р.370 000,р.250 000,https://drive.google.com/drive/folders/1aSmonK...,катя адушкина,р.490 000
3,амина tenderlybae,твич/лайфстайл,амина tenderlybae,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,...,26 000,500,р.560 000,р.280 000,р.250 000,р.225 000,-,https://drive.google.com/drive/folders/13YIWAD...,амина tenderlybae,NaN
4,vikishow,лайфстайл/дети/подростки,vikishow,https://www.instagram.com/vikishow_official/,лайфстайл/дети/подростки,2 500 000,70 000,30 000,80 000,15 000,...,50 000,4 000,р.700 000,р.440 000,р.440 000,р.350 000,р.440 000,https://drive.google.com/drive/folders/1_CHbok...,vikishow,NaN


> На данный момент мы не будем удалять дублирующие столбцы с именем и категорией блогеров из просоединённых столбцов, так как они нам понадобятся в дальнейшем для бота и для исследования

### Подготовка таблицы didenok_telegram

In [67]:
didenok_telegram.head()

7,блогер,ссылка,"категория, тематика блогера",количество подписчиков,среднее кол-во просмотров поста,среднее кол-во реакций и комментариев,стоимость за фотопост + текст,стоимость за видеопост + текст,стоимость за кружочек до 60 сек,стоимость за текстовое сообщение без фото,статистика,статистика ссылка
8,влад бумага а4,https://t.me/A4omg,лайфстайл/гейминг,753 000,500 000,100 000,р.1 000 000,р.1 500 000,р.875 000,р.625 000,Статистика,https://tgstat.ru/channel/@A4omg/stat
9,влад глент,https://t.me/vladglenttg,техно/лайфстайл,515 000,270 000,15 000,р.440 000,р.565 000,р.565 000,р.440 000,Статистика,https://tgstat.ru/channel/@vladglenttg/stat
10,антон шастун,https://t.me/shastoonchannel,лайфстайл/юмор,273 000,200 000,15 000,р.645 000,р.645 000,р.500 000,-,Статистика,https://drive.google.com/drive/folders/14xxWvL...
11,амина тендерлибае,https://t.me/plsbetenderly,твич/лайфстайл,256 000,80 000,3 000,р.240 000,р.390 000,р.310 000,р.170 000,Статистика,https://drive.google.com/drive/folders/1Gg5LC6...
12,катя клэп,https://t.me/KateClappTelegram,лайфстайл/бьюти,151 000,95 000,5 000,р.285 000,р.650 000,р.395 000,-,-,NaN


In [68]:
didenok_telegram.drop(columns=['статистика'], axis=1, inplace=True)

In [69]:
didenok_telegram.columns

Index(['блогер', 'ссылка', 'категория, тематика блогера',
       'количество подписчиков', 'среднее кол-во просмотров поста',
       'среднее кол-во реакций и комментариев',
       'стоимость за фотопост + текст', 'стоимость за видеопост + текст',
       'стоимость за кружочек до 60 сек',
       'стоимость за текстовое сообщение без фото', 'статистика ссылка'],
      dtype='object', name=7)

In [70]:
didenok_telegram.rename(columns = {'категория, тематика блогера':'тематика',
                                   'статистика ссылка':'статистика',
                                   'среднее кол-во реакций и комментариев':'средняя вовлечённость',
                                   'стоимость за фотопост + текст':'стоимость фотопост',
                                   'стоимость за видеопост + текст':'стоимость видеопост',
                                   'стоимость за кружочек до 60 сек':'стоимость кружочек до 60 сек',
                                   'стоимость за текстовое сообщение без фото':'стоимость текстовое сообщение без фото'
                                  }, inplace=True)

### Второе объединение

> Исправим неправильное наименование блогеров

In [71]:
didenok_telegram.loc\
    [didenok_telegram['блогер'] == 'амина тендерлибае', 'блогер'] = 'амина tenderlybae'

didenok_telegram.loc\
    [didenok_telegram['блогер'] == 'володя ххл', 'блогер'] = 'володя xxl'

didenok_telegram.loc\
    [didenok_telegram['блогер'] == 'маша омг', 'блогер'] = 'мария омг'

didenok_telegram.loc\
    [didenok_telegram['блогер'] == 'илья бёрн', 'блогер'] = 'илья берн'

> Объединяем

In [72]:
bufer = merge_bufer_df(bufer, didenok_telegram, 'Telegram')

Количество дубликатов: 0


In [73]:
bufer.tail()

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Telegram ссылка,Telegram тематика,Telegram количество подписчиков,Telegram среднее кол-во просмотров поста,Telegram средняя вовлечённость,Telegram стоимость фотопост,Telegram стоимость видеопост,Telegram стоимость кружочек до 60 сек,Telegram стоимость текстовое сообщение без фото,Telegram статистика
69,позавчерашние новости,новости/интервью,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,чай с жасмин,религия/психология,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,dimixer live,новости/события,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,несусвет,интервью/шоу,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,маша нарциссизм,лайфстайл/бьюти,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,https://t.me/notmasha1,лайфстайл/бьюти,37 000,16 000,500,р.35 000,р.50 000,р.35 000,р.20 000,https://tgstat.ru/channel/@notmasha1/stat


### Подготовка таблицы didenok_vk_zen_show

> Мы видим, что помимо рекламы в VK, в таблице также есть строки с рекламой в Дзен Шоу.  
> Сделаем отдельный датафрейм для них, а из основного датафрейма удалим их.

In [74]:
didenok_vk_zen_show.head()

6,канал,ссылка,"категория, тематика канала",количество подписчиков,среднее кол-во просмотров ролика,стоимость за интеграцию до 1 мин,стоимость за продакт плейсмент,стоимость за преролл до 30 сек,стоимость за мидролл до 30 сек,статистика,статистика ссылка
7,громкий вопрос (1 слот),https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 600 000,р.1 900 000,р.1 000 000,р.1 000 000,р.800 000,Статистика,https://drive.google.com/drive/u/1/folders/1nx...
8,громкий вопрос (2 слот),https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 600 000,р.1 600 000,NaN,NaN,NaN,NaN,NaN
9,шоу истории (1 слот),https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 200 000,р.1 500 000,р.850 000,р.850 000,р.650 000,NaN,NaN
10,шоу истории (2 слот),https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 200 000,р.1 200 000,NaN,NaN,NaN,NaN,NaN
11,что делаем? куда идём? (1 слот),https://vk.com/improcom,шоу/юмор/тревел,460 000,3 000 000,р.3 000 000,р.2 500 000,р.1 700 000,р.1 300 000,NaN,NaN


> Заполним столбец "Статистика"

In [75]:
didenok_vk_zen_show[['статистика ссылка']] = \
            didenok_vk_zen_show[['статистика ссылка']].fillna(method='ffill')

In [76]:
didenok_vk_zen_show.drop(columns = ['статистика'], axis=1, inplace=True)
didenok_vk_zen_show.rename(columns = {'статистика ссылка':'статистика'}, inplace=True)

In [77]:
didenok_vk_zen_show.rename(columns = {'канал':'блогер',
                                      'категория, тематика блогера':'тематика',
                                      'стоимость за интеграцию до 1 мин':'стоимость интеграция до 1 мин',
                                      'стоимость за продакт плейсмент':'стоимость продакт плейсмент',
                                      'стоимость за преролл до 30 сек':'стоимость преролл до 30 сек',
                                      'стоимость за мидролл до 30 сек':'стоимость мидролл до 30 сек'
                                     }, inplace=True)

> Переведём позиции со строк "2 слот" в отдельный столбец

In [78]:
didenok_vk_zen_show = new_position_in_columns(didenok_vk_zen_show, 'блогер', 'стоимость интеграция до 1 мин')
didenok_vk_zen_show.rename(columns = {'стоимость интеграция до 1 мин 2 слот':'стоимость интеграция 2 слот'
                                  }, inplace = True)

> Используем функцию .str.contains() для поиска подстроки в строках столбца DataFrame, а именно 'https://dzen.ru' в ссылке

In [79]:
# didenok_zen_show = didenok_vk_zen_show[didenok_vk_zen_show['ссылка'].str.contains('https://dzen.ru', na=False)]

In [80]:
# didenok_zen_show

> Используем ту же логику, что и выше, но теперь исключим строки, где в ссылке встречается 'https://dzen.ru', то есть оставим только 'https://vk.com/'

In [81]:
#Инверсия условия ~ здесь означает "не содержит", то есть выбираются строки,
#которые не содержат 'https://dzen.ru' в столбце 'ссылка'.
#Таким образом, didenok_vk_show будет содержать строки, которые не соответствуют заданному условию.

# didenok_vk_show = didenok_vk_zen_show[~didenok_vk_zen_show['ссылка'].str.contains('https://dzen.ru', na=False)]

> Переименуем столбцы, чтобы подвести их под единый принятый формат

In [82]:
# didenok_vk_show.columns

In [83]:
# didenok_vk_show.rename(columns = {'канал':'блогер',
#                                   'категория, тематика блогера':'тематика',
#                                   'стоимость за интеграцию до 1 мин':'стоимость интеграция до 1 мин',
#                                   'стоимость за продакт плейсмент':'стоимость продакт плейсмент',
#                                   'стоимость за преролл до 30 сек':'стоимость преролл до 30 сек',
#                                   'стоимость за мидролл до 30 сек':'стоимость мидролл до 30 сек'
#                                  }, inplace = True)

# didenok_zen_show.rename(columns = {'канал':'блогер',
#                                   'категория, тематика блогера':'тематика',
#                                   'стоимость за интеграцию до 1 мин':'стоимость интеграция до 1 мин',
#                                   'стоимость за продакт плейсмент':'стоимость продакт плейсмент',
#                                   'стоимость за преролл до 30 сек':'стоимость преролл до 30 сек',
#                                   'стоимость за мидролл до 30 сек':'стоимость мидролл до 30 сек'
#                                  }, inplace = True)

In [84]:
# didenok_zen_show = new_position_in_columns(didenok_zen_show, 'блогер', 'стоимость интеграция до 1 мин')

In [85]:
# didenok_zen_show.rename(columns = {'стоимость интеграция до 1 мин 2 слот':'стоимость интеграция 2 слот'
#                                   }, inplace = True)

In [86]:
# Проверим, что получилось
# didenok_zen_show.head()

In [87]:
# didenok_vk_show = new_position_in_columns(didenok_vk_show, 'блогер', 'стоимость интеграция до 1 мин')

In [88]:
# didenok_vk_show.rename(columns = {'стоимость интеграция до 1 мин 2 слот':'стоимость интеграция 2 слот'
#                                   }, inplace = True)


In [89]:
#Проверим, что получилось
didenok_vk_zen_show.head()

6,ссылка,"категория, тематика канала",количество подписчиков,среднее кол-во просмотров ролика,стоимость интеграция до 1 мин,стоимость продакт плейсмент,стоимость преролл до 30 сек,стоимость мидролл до 30 сек,статистика,блогер,стоимость интеграция 2 слот
0,https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 600 000,р.1 900 000,р.1 000 000,р.1 000 000,р.800 000,https://drive.google.com/drive/u/1/folders/1nx...,громкий вопрос,р.1 600 000
1,https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 200 000,р.1 500 000,р.850 000,р.850 000,р.650 000,https://drive.google.com/drive/u/1/folders/1nx...,шоу истории,р.1 200 000
2,https://vk.com/improcom,шоу/юмор/тревел,460 000,3 000 000,р.3 000 000,р.2 500 000,р.1 700 000,р.1 300 000,https://drive.google.com/drive/u/1/folders/1nx...,что делаем? куда идём?,NaN
3,https://vk.com/improcom,шоу/юмор/тревел,460 000,3 000 000,р.2 700 000,NaN,NaN,NaN,https://drive.google.com/drive/u/1/folders/1nx...,что делаем? куда идём?,NaN
4,https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 000 000,р.1 600 000,р.900 000,р.900 000,р.700 000,https://drive.google.com/drive/u/1/folders/1nx...,тейблтайм,р.1 300 000


> У нас получилась интересная ситуация: у блогера "что делаем? куда идём?" не сработала функция.  
> Сделаем проверку:

In [90]:
string1 = didenok_vk_zen_show.loc[2, 'блогер']
string2 = didenok_vk_zen_show.loc[3, 'блогер']

print(repr(string1))
print(repr(string2))

'что делаем? куда идём?'
'что делаем? куда идём?'


In [91]:
# Затем вы можете сравнить их
if repr(string1) == repr(string2):
    print("Строки идентичны")
else:
    print("Строки различаются")

Строки идентичны


> Объединим данные строки:

In [92]:
indexes_to_remember = []

# Проверяем условие и сохраняем индексы в список
for index, row in didenok_vk_zen_show.iterrows():
    if row['блогер'] == 'что делаем? куда идём?':
        indexes_to_remember.append(index)

# Получаем последний и первый индексы
last_index = indexes_to_remember[-1]
first_index = indexes_to_remember[0]

# Заменяем значение в столбце 'стоимость за интеграцию до 1 мин 2 слот' в строке с первым индексом
didenok_vk_zen_show.at[first_index, 'стоимость интеграция 2 слот'] = \
            didenok_vk_zen_show.at[last_index, 'стоимость интеграция до 1 мин']

didenok_vk_zen_show = didenok_vk_zen_show.drop(last_index)

In [93]:
didenok_vk_zen_show.head()

6,ссылка,"категория, тематика канала",количество подписчиков,среднее кол-во просмотров ролика,стоимость интеграция до 1 мин,стоимость продакт плейсмент,стоимость преролл до 30 сек,стоимость мидролл до 30 сек,статистика,блогер,стоимость интеграция 2 слот
0,https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 600 000,р.1 900 000,р.1 000 000,р.1 000 000,р.800 000,https://drive.google.com/drive/u/1/folders/1nx...,громкий вопрос,р.1 600 000
1,https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 200 000,р.1 500 000,р.850 000,р.850 000,р.650 000,https://drive.google.com/drive/u/1/folders/1nx...,шоу истории,р.1 200 000
2,https://vk.com/improcom,шоу/юмор/тревел,460 000,3 000 000,р.3 000 000,р.2 500 000,р.1 700 000,р.1 300 000,https://drive.google.com/drive/u/1/folders/1nx...,что делаем? куда идём?,р.2 700 000
4,https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 000 000,р.1 600 000,р.900 000,р.900 000,р.700 000,https://drive.google.com/drive/u/1/folders/1nx...,тейблтайм,р.1 300 000
5,https://vk.com/shastoon.channel,шоу/юмор/развлечения,358 000,1 600 000,р.1 900 000,р.1 000 000,р.1 000 000,р.800 000,https://drive.google.com/drive/u/1/folders/1US...,контакты,р.1 600 000


### Третье объединение

In [94]:
# didenok_zen_show.rename(columns = {'категория, тематика канала':'тематика'}, inplace=True)

In [95]:
# bufer = merge_bufer_df(bufer, didenok_zen_show, 'Дзен Шоу')

In [96]:
# bufer.tail()

### Четвёртое объединение

In [97]:
didenok_vk_zen_show.rename(columns = {'категория, тематика канала':'тематика'}, inplace=True)

In [98]:
bufer = merge_bufer_df(bufer, didenok_vk_zen_show, 'VK Видео')

Количество дубликатов: 0


In [99]:
bufer.tail()

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,VK Видео тематика,VK Видео количество подписчиков,VK Видео среднее кол-во просмотров ролика,VK Видео стоимость интеграция до 1 мин,VK Видео стоимость продакт плейсмент,VK Видео стоимость преролл до 30 сек,VK Видео стоимость мидролл до 30 сек,VK Видео статистика,VK Видео блогер,VK Видео стоимость интеграция 2 слот
72,несусвет,интервью/шоу,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,маша нарциссизм,лайфстайл/бьюти,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,что делаем? куда идём?,шоу/юмор/тревел,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,шоу/юмор/тревел,460 000,3 000 000,р.3 000 000,р.2 500 000,р.1 700 000,р.1 300 000,https://drive.google.com/drive/u/1/folders/1nx...,что делаем? куда идём?,р.2 700 000
75,тейблтайм,шоу/юмор/развлечения,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,шоу/юмор/развлечения,460 000,1 000 000,р.1 600 000,р.900 000,р.900 000,р.700 000,https://drive.google.com/drive/u/1/folders/1nx...,тейблтайм,р.1 300 000
76,jam,шоу/юмор/развлечения,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,шоу/юмор/развлечения,61 000,700 000,р.900 000,р.500 000,р.500 000,р.400 000,https://drive.google.com/drive/u/1/folders/1sk...,jam,р.700 000


### Подготовка таблицы didenok_vk_group

In [100]:
didenok_vk_group.head()

7,блогер,ссылка,"категория, тематика блогера",количество подписчиков,среднее кол-во просмотров поста,среднее кол-во просмотров вк клипа,среднее кол-во лайков,среднее кол-во комментариев,стоимость за фотопост,стоимость за видеопост,стоимость за вк клип,статистика,статистика ссылка
8,влад бумага а4,https://vk.com/a4,лайфстайл/гейминг,1 540 000,200 000,250 000,6 000,180,р.750 000,-,р.1 625 000,обновляем,NaN
9,амина тендерлибае,https://vk.com/tenderlybae,твич/лайфстайл,739 000,250 000,80 000,10 000,50,р.210 000,р.300 000,р.300 000,Статистика,https://drive.google.com/drive/folders/12PNvkW...
10,катя адушкина,https://vk.com/katyaadushkina,музыка/лайфстайл/танцы,1 000 000,250 000,150 000,4 000,10,р.155 000,р.185 000,р.185 000,Статистика,https://drive.google.com/drive/folders/1MnkkeU...
11,даня крастер,https://vk.com/bnbproduction,DIY,275 000,30 000,30 000,200,10,р.200 000,р.370 000,р.370 000,Статистика,https://drive.google.com/drive/folders/1dVEwSt...
12,vikishow,https://vk.com/vikishow,лайфстайл/дети/подростки,478 000,70 000,50 000,2 000,20,р.70 000,р.115 000,р.115 000,Статистика,https://drive.google.com/drive/folders/14MQXd3...


In [101]:
didenok_vk_group.drop(columns=['статистика'], axis=1, inplace=True)

In [102]:
didenok_vk_group.columns

Index(['блогер', 'ссылка', 'категория, тематика блогера',
       'количество подписчиков', 'среднее кол-во просмотров поста',
       'среднее кол-во просмотров вк клипа', 'среднее кол-во лайков',
       'среднее кол-во комментариев', 'стоимость за фотопост',
       'стоимость за видеопост', 'стоимость за вк клип', 'статистика ссылка'],
      dtype='object', name=7)

In [103]:
didenok_vk_group.rename(columns = {'категория, тематика блогера':'тематика',
                                   'статистика ссылка':'статистика',
                                   'стоимость за фотопост':'стоимость фотопост',
                                   'стоимость за видеопост':'стоимость видеопост',
                                   'стоимость за вк клип':'стоимость вк клип'
                                  }, inplace=True)

### Пятое объединение

> Исправим одну неточность в имени блогера

In [104]:
didenok_vk_group.loc\
    [didenok_vk_group['блогер'] == 'амина тендерлибае', 'блогер'] = 'амина tenderlybae'

In [105]:
bufer = merge_bufer_df(bufer, didenok_vk_group, 'VK группа')

Количество дубликатов: 0


In [106]:
bufer.head()

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,VK группа тематика,VK группа количество подписчиков,VK группа среднее кол-во просмотров поста,VK группа среднее кол-во просмотров вк клипа,VK группа среднее кол-во лайков,VK группа среднее кол-во комментариев,VK группа стоимость фотопост,VK группа стоимость видеопост,VK группа стоимость вк клип,VK группа статистика
0,катя клэп,лайфстайл/бьюти,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,влад бумага а4,челленджи,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,...,лайфстайл/гейминг,1 540 000,200 000,250 000,6 000,180,р.750 000,-,р.1 625 000,NaN
2,катя адушкина,музыка/лайфстайл/танцы,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,...,музыка/лайфстайл/танцы,1 000 000,250 000,150 000,4 000,10,р.155 000,р.185 000,р.185 000,https://drive.google.com/drive/folders/1MnkkeU...
3,амина tenderlybae,твич/лайфстайл,амина tenderlybae,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,...,твич/лайфстайл,739 000,250 000,80 000,10 000,50,р.210 000,р.300 000,р.300 000,https://drive.google.com/drive/folders/12PNvkW...
4,vikishow,лайфстайл/дети/подростки,vikishow,https://www.instagram.com/vikishow_official/,лайфстайл/дети/подростки,2 500 000,70 000,30 000,80 000,15 000,...,лайфстайл/дети/подростки,478 000,70 000,50 000,2 000,20,р.70 000,р.115 000,р.115 000,https://drive.google.com/drive/folders/14MQXd3...


### Подготовка таблицы didenok_vk_ls

In [107]:
# didenok_vk_ls
# didenok_vk_ls.drop(columns=['статистика'], axis=1, inplace= True)
# didenok_vk_ls.rename(columns = {'категория, тематика блогера':'тематика',
#                                'статистика ссылка':'статистика'}, inplace = True)

# ### Шестое объединение
# bufer = merge_bufer_df(bufer, didenok_vk_ls, 'VK личная страница')
# bufer.tail()

### Подготовка таблицы didenok_twitch

In [108]:
didenok_twitch.head()

7,блогер,сcылка,категория,количество подписчиков,среднее кол-во зрителей,"уникальные зрители , (за 30 дней)",статистика,баннер + ссылка в описании 1 мес,оверлей использование логотипа товара/бренда в процессе трансляции,нативное голосовое упоминание рекламируемого товара в процессе трансляции,ссылка в чат бот 1 стрим,демонстрация-обзор товара на трансляции,статистика ссылка
8,амина tenderlybae,twitch.tv/tenderlybae,Just Chatting / IRL / Multigaming,1 300 000,6 500,546 000,Статистика,р.950 000,р.240 000,р.160 000,р.160 000,р.715 000,https://disk.yandex.ru/d/UoxOK0U47TB6rA


In [109]:
didenok_twitch.drop(columns=['статистика'], axis=1, inplace=True)

In [110]:
didenok_twitch.rename(columns = {'сcылка':'ссылка',
                                 'уникальные зрители , (за 30 дней)':'уникальные зрители (30 дней)',
                                 'баннер + ссылка в описании 1 мес':'стоимость за баннер + ссылка в описании 1 мес',
                                 'оверлей использование логотипа товара/бренда в процессе трансляции':'стоимость оверлей логотипа товара/бренда в процессе трансляции',
                                 'нативное голосовое упоминание рекламируемого товара в процессе трансляции':'стоимость нативное голосовое упоминание товара в трансляции',
                                 'ссылка в чат бот 1 стрим ':'стоимость ссылка в чат-боте на 1 стрим',
                                 'демонстрация-обзор товара на трансляции':'стоимость демонстрация/обзор товара на трансляции',
                                 'статистика':'статистика',
                                 'категория':'тематика',
                                 'статистика ссылка':'статистика'
                                }, inplace=True)

### Седьмое объединение

In [111]:
bufer = merge_bufer_df(bufer, didenok_twitch, 'Twitch')

Количество дубликатов: 0


In [112]:
bufer.head()

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Twitch тематика,Twitch количество подписчиков,Twitch среднее кол-во зрителей,Twitch уникальные зрители (30 дней),Twitch стоимость за баннер + ссылка в описании 1 мес,Twitch стоимость оверлей логотипа товара/бренда в процессе трансляции,Twitch стоимость нативное голосовое упоминание товара в трансляции,Twitch стоимость ссылка в чат-боте на 1 стрим,Twitch стоимость демонстрация/обзор товара на трансляции,Twitch статистика
0,катя клэп,лайфстайл/бьюти,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,влад бумага а4,челленджи,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,катя адушкина,музыка/лайфстайл/танцы,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,амина tenderlybae,твич/лайфстайл,амина tenderlybae,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,...,Just Chatting / IRL / Multigaming,1 300 000,6 500,546 000,р.950 000,р.240 000,р.160 000,р.160 000,р.715 000,https://disk.yandex.ru/d/UoxOK0U47TB6rA
4,vikishow,лайфстайл/дети/подростки,vikishow,https://www.instagram.com/vikishow_official/,лайфстайл/дети/подростки,2 500 000,70 000,30 000,80 000,15 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


> Мы объединили все таблицы DIDENOK, осталось добавить два столбца: налог и контакты менеджера

In [113]:
nalog = 'НДС (20%)'
manager = 'info@didenokteam.com'

In [114]:
bufer['налог'] = nalog
bufer['контакты менеджера'] = manager

In [115]:
bufer.head()

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Twitch среднее кол-во зрителей,Twitch уникальные зрители (30 дней),Twitch стоимость за баннер + ссылка в описании 1 мес,Twitch стоимость оверлей логотипа товара/бренда в процессе трансляции,Twitch стоимость нативное голосовое упоминание товара в трансляции,Twitch стоимость ссылка в чат-боте на 1 стрим,Twitch стоимость демонстрация/обзор товара на трансляции,Twitch статистика,налог,контакты менеджера
0,катя клэп,лайфстайл/бьюти,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,НДС (20%),info@didenokteam.com
1,влад бумага а4,челленджи,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,НДС (20%),info@didenokteam.com
2,катя адушкина,музыка/лайфстайл/танцы,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,НДС (20%),info@didenokteam.com
3,амина tenderlybae,твич/лайфстайл,амина tenderlybae,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,...,6 500,546 000,р.950 000,р.240 000,р.160 000,р.160 000,р.715 000,https://disk.yandex.ru/d/UoxOK0U47TB6rA,НДС (20%),info@didenokteam.com
4,vikishow,лайфстайл/дети/подростки,vikishow,https://www.instagram.com/vikishow_official/,лайфстайл/дети/подростки,2 500 000,70 000,30 000,80 000,15 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,НДС (20%),info@didenokteam.com


## Подготовка данных по блогерам от VK

In [116]:
vk_bloggers.head(7)

,Сообщество,Шоу,KPI (стартов просмотра на 1 эпизод на 30-е сутки после релиза),Стоимость интеграции 1-й слот до НДС,Стоимость интеграции 2-й слот до НДС,Продакт плейсмент (30 сек) до НДС,Стоимость рекламной заставки (пре-ролл до 20 сек) до НДС,Стоимость рекламной заставки (мид-ролл до 20 сек) до НДС,Статистика,Контакты менеджера по продажам,Ссылка,Статистика ссылка
1,Азамат Мусагалиев,Большое шоу,2 600 000,р.3 500 000,р.3 200 000,р.2 000 000,р.3 000 000,р.2 800 000,Статистика,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...
2,NaN,Это про меня,600 000,р.700 000,по запросу,р.300 000,р.300 000,р.200 000,NaN,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",NaN,NaN
3,NaN,Я себя знаю,2 000 000,р.2 800 000,р.2 500 000,по запросу,р.2 000 000,р.1 800 000,NaN,"Марина Ванюшкина, TG: @MarinaVan, m.vanyushkin...",NaN,NaN
4,NaN,Шоу памяти Азамата Мусагалиева NEW,1 500 000,р.2 500 000,по запросу,по запросу,р.2 000 000,по запросу,NaN,"Магда Булавина, TG: @Portenato, m.bulavina@vk....",NaN,NaN
5,Пикабу,Чебатков читает Пикабу,1 000 000,р.1 350 000,по запросу,по запросу,р.1 000 000,по запросу,Статистика,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/pikabu,https://drive.google.com/drive/u/1/folders/1Aq...
6,Женский форум,Женский форум,1 000 000,р.1 800 000,р.1 500 000,р.600 000,р.1 000 000,нет,Статистика,"Марина Ванюшкина, TG: @MarinaVan, m.vanyushkin...",https://vk.com/zhenforum,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%96...
7,Варвара Щербакова по вашим интересам!,В гостях у Вари,1 000 000,р.1 500 000,р.1 200 000,р.500 000,р.850 000,нет,Статистика,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/varjauletela_group,https://drive.google.com/drive/u/1/folders/1Q1...


> Сделаем предобработку DataFrame

In [117]:
key_word = [] # Список ключевых слов для поиска столбцов и их удаления
drop_word = ' до НДС' # Список ключевых слов для их удаления из названия столбцов
upd_columns = ['Сообщество', 'Шоу'] # Список столбцов с именами блогеров для приведения к единому формату

In [118]:
vk_bloggers = pretraining_df(vk_bloggers, key_word, drop_word, upd_columns)

> Удалим сразу не нужный нам столбец "Статистика"

In [119]:
vk_bloggers.columns

Index(['сообщество', 'шоу',
       'kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза)',
       'стоимость интеграции 1-й слот', 'стоимость интеграции 2-й слот',
       'продакт плейсмент (30 сек)',
       'стоимость рекламной заставки (пре-ролл до 20 сек)',
       'стоимость рекламной заставки (мид-ролл до 20 сек)', 'статистика',
       'контакты менеджера по продажам', 'ссылка', 'статистика ссылка'],
      dtype='object', name=0)

In [120]:
vk_bloggers.drop(columns=['статистика'], axis=1, inplace=True)

vk_bloggers.rename(columns = {'статистика ссылка':'статистика',
                              'стоимость интеграции 1-й слот':'стоимость интеграция 1 слот',
                              'стоимость интеграции 2-й слот':'стоимость интеграция 2 слот',
                              'продакт плейсмент (30 сек)':'стоимость продакт плейсмент',
                              'стоимость рекламной заставки (пре-ролл до 20 сек)':'стоимость преролл до 20 сек',
                              'стоимость рекламной заставки (мид-ролл до 20 сек)':'стоимость мидролл до 20 сек'
                             }, inplace=True)

In [121]:
vk_bloggers.head()

,сообщество,шоу,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика
1,азамат мусагалиев,большое шоу,2 600 000,р.3 500 000,р.3 200 000,р.2 000 000,р.3 000 000,р.2 800 000,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...
2,NaN,это про меня,600 000,р.700 000,по запросу,р.300 000,р.300 000,р.200 000,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",NaN,NaN
3,NaN,я себя знаю,2 000 000,р.2 800 000,р.2 500 000,по запросу,р.2 000 000,р.1 800 000,"Марина Ванюшкина, TG: @MarinaVan, m.vanyushkin...",NaN,NaN
4,NaN,шоу памяти азамата мусагалиева new,1 500 000,р.2 500 000,по запросу,по запросу,р.2 000 000,по запросу,"Магда Булавина, TG: @Portenato, m.bulavina@vk....",NaN,NaN
5,пикабу,чебатков читает пикабу,1 000 000,р.1 350 000,по запросу,по запросу,р.1 000 000,по запросу,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/pikabu,https://drive.google.com/drive/u/1/folders/1Aq...


> Нам необходимо заполнить данные - так как при часть ячеек в исходной таблице объеденены, то правильное значение ставиться в верхнюю строку, а остальные становятся NaN, и мы заполним значения "сверху вниз". 
> Мы заполним столбцы 'Сообщество', 'Ссылка на канал ', 'Ссылка' и 'Шоу'.  
> Столбец 'Ссылка на канал' то появляется, то исчезает, так что учтём и его.

> К сожалению, мы не сможем сразу заполнить столбец 'Статистика ссылка' и 'ссылка', так как у некоторых каналов нет Статистики и ссылки, и если мы заполним сверху вниз, то у нас получится неверные ссылки - отдельно решим эту проблему.

> В функции pretraining_df мы используем методы .strip(), .lower() и .replace() непосредственно на значениях NaN, которые мы заполнили через .fillna(''). Если не заполнить пропуски, то Python пытается их выполнить, что может привести к неожиданным результатам <em>[В данном проекте, например, символ * из значения NaN был заменен на "8" методом .replace("*")]</em>.
Поэтому надо превратить строки обратно в NaN, чтобы сделать заполнение.

In [122]:
vk_bloggers['ссылка'] = vk_bloggers['ссылка'].replace('', np.nan)
vk_bloggers['контакты менеджера по продажам'] = vk_bloggers['контакты менеджера по продажам'].replace('', np.nan)

In [123]:
try:
    vk_bloggers[['сообщество', 'ссылка', 'шоу', 'контакты менеджера по продажам']] = \
            vk_bloggers[['сообщество', 'ссылка', 'шоу', 'контакты менеджера по продажам']].fillna(method='ffill')
    
except:
    vk_bloggers[['сообщество', 'шоу', 'контакты менеджера по продажам']] = \
            vk_bloggers[['сообщество', 'шоу', 'контакты менеджера по продажам']].fillna(method='ffill')

> Чтобы заполнить столбец 'статистика', сделаем групировку по столбцу 'сообщество', так как у них одинаковые каналы, а значит и одинаковая статистика, выберем только столбец 'статистика' для работы и заполним с помощью .transform('first'), но учтём, что в столбце "ссылка" у нас есть исключение

In [124]:
vk_bloggers['статистика'] = vk_bloggers.groupby('сообщество')['статистика'].transform('first')

In [125]:
vk_bloggers['ссылка'] = vk_bloggers.groupby('сообщество')['ссылка']\
                        .transform(lambda group: group.iloc[0] if 'ок video' not in group.name else group)

In [126]:
# Проверяем
vk_bloggers.head()

,сообщество,шоу,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика
1,азамат мусагалиев,большое шоу,2 600 000,р.3 500 000,р.3 200 000,р.2 000 000,р.3 000 000,р.2 800 000,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...
2,азамат мусагалиев,это про меня,600 000,р.700 000,по запросу,р.300 000,р.300 000,р.200 000,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...
3,азамат мусагалиев,я себя знаю,2 000 000,р.2 800 000,р.2 500 000,по запросу,р.2 000 000,р.1 800 000,"Марина Ванюшкина, TG: @MarinaVan, m.vanyushkin...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...
4,азамат мусагалиев,шоу памяти азамата мусагалиева new,1 500 000,р.2 500 000,по запросу,по запросу,р.2 000 000,по запросу,"Магда Булавина, TG: @Portenato, m.bulavina@vk....",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...
5,пикабу,чебатков читает пикабу,1 000 000,р.1 350 000,по запросу,по запросу,р.1 000 000,по запросу,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/pikabu,https://drive.google.com/drive/u/1/folders/1Aq...


> В основной таблице bufer название блогера или его шоу указываются в едином столбце "блогер".  
> В таблице vk_bloggers имя блогера и его шоу разделены по разным столбцам.  
> Мы объединим их, чтобы в дальшейшем мы могли использовать поиск в боте

In [127]:
vk_bloggers['блогер'] = vk_bloggers['сообщество'] + ' - ' + vk_bloggers['шоу']
vk_bloggers.drop(columns=['сообщество', 'шоу'], axis=1 , inplace=True)

In [128]:
#Проверяем результат
vk_bloggers.head()

,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика,блогер
1,2 600 000,р.3 500 000,р.3 200 000,р.2 000 000,р.3 000 000,р.2 800 000,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...,азамат мусагалиев - большое шоу
2,600 000,р.700 000,по запросу,р.300 000,р.300 000,р.200 000,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...,азамат мусагалиев - это про меня
3,2 000 000,р.2 800 000,р.2 500 000,по запросу,р.2 000 000,р.1 800 000,"Марина Ванюшкина, TG: @MarinaVan, m.vanyushkin...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...,азамат мусагалиев - я себя знаю
4,1 500 000,р.2 500 000,по запросу,по запросу,р.2 000 000,по запросу,"Магда Булавина, TG: @Portenato, m.bulavina@vk....",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...,азамат мусагалиев - шоу памяти азамата мусагал...
5,1 000 000,р.1 350 000,по запросу,по запросу,р.1 000 000,по запросу,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/pikabu,https://drive.google.com/drive/u/1/folders/1Aq...,пикабу - чебатков читает пикабу


> Мы видим, что помимо рекламы в VK, в таблице также есть строки с рекламой в Одноклассниках.  
> Сделаем отдельный датафрейм для них, а из основного датафрейма удалим их.

In [129]:
ok_bloggers = vk_bloggers[vk_bloggers['блогер'].str.contains('ок video')]

In [130]:
ok_bloggers.head()

,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика,блогер
85,5 000 000,р.1 150 000,по запросу,нет,р.800 000,нет,"Анна Ипатова, TG: @Anna_ipp, a.ipatova@vk.team",https://ok.ru/oknutye,None,ок video - в мире домашних животных
86,5 000 000,р.700 000,по запросу,по запросу,нет,нет,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://ok.ru/travelok,None,"ок video - ок, мама, поехали!"
87,2 500 000,р.800 000,р.700 000,по запросу,р.500 000,р.400 000,Алексей Леднев - alexey.lednev@vk.team,https://ok.ru/travelok,None,ок video - психология кино
88,5 000 000,р.1 100 000,по запросу,по запросу,р.500 000,нет,Алексей Леднев - alexey.lednev@vk.team,https://ok.ru/oknutye,None,ок video - полтора спортсмена
89,5 000 000,р.1 100 000,по запросу,р.800 000,р.500 000,нет,Алексей Леднев - alexey.lednev@vk.team,https://ok.ru/oknutye,None,ок video - шеф против любителя


In [131]:
# Проверим на дубликаты:

duplicates = ok_bloggers[ok_bloggers.duplicated('блогер', keep=False)]
duplicates

,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика,блогер


> Сделаем DataFrame с блогерами из VK, удалив строки с блогерами из Одноклассников из исходного DataFrame

In [132]:
vk_bloggers = vk_bloggers[~vk_bloggers['блогер'].str.contains('ок video')]

In [133]:
vk_bloggers.head()

,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика,блогер
1,2 600 000,р.3 500 000,р.3 200 000,р.2 000 000,р.3 000 000,р.2 800 000,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...,азамат мусагалиев - большое шоу
2,600 000,р.700 000,по запросу,р.300 000,р.300 000,р.200 000,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...,азамат мусагалиев - это про меня
3,2 000 000,р.2 800 000,р.2 500 000,по запросу,р.2 000 000,р.1 800 000,"Марина Ванюшкина, TG: @MarinaVan, m.vanyushkin...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...,азамат мусагалиев - я себя знаю
4,1 500 000,р.2 500 000,по запросу,по запросу,р.2 000 000,по запросу,"Магда Булавина, TG: @Portenato, m.bulavina@vk....",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...,азамат мусагалиев - шоу памяти азамата мусагал...
5,1 000 000,р.1 350 000,по запросу,по запросу,р.1 000 000,по запросу,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/pikabu,https://drive.google.com/drive/u/1/folders/1Aq...,пикабу - чебатков читает пикабу


In [134]:
# Проверим на дубликаты

duplicates = vk_bloggers[vk_bloggers.duplicated('блогер', keep=False)]
duplicates

,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика,блогер
27,600 000,Интеграция с участием актеров: 350 000р,по запросу,по запросу,р.350 000,р.250 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - 9 панчей
28,NaN,Интеграция с участием блогера: 550 000р,Интеграция с участием блогера: 450 000р,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - 9 панчей
29,700 000,Интеграция с участием актеров: 600 000р,по запросу,по запросу,р.450 000,р.350 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - арена
30,NaN,Интеграция с участием блогера: 750 000р,Интеграция с участием блогера: 650 000р,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - арена
32,1 600 000,Интеграция с участием актеров: 1 150 000р,Интеграция с участием актеров: 1 050 000р,по запросу,р.1 000 000,р.800 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - кто твой подписчик
33,NaN,Интеграция с участием блогера: 1 550 000р,Интеграция с участием блогера: 1 350 000р,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - кто твой подписчик
36,1 600 000,Интеграция с участием актеров: 1 150 000р,Интеграция с участием актеров: 1 050 000р,по запросу,р.1 000 000,р.800 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - удиви меня
37,NaN,Интеграция с участием блогера: 1 450 000р,Интеграция с участием блогера: 1 250 000р,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - удиви меня
38,700 000,Интеграция с участием актеров: 650 000р,по запросу,по запросу,р.450 000,р.350 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - сходка
39,NaN,Интеграция с участием блогера: 1 250 000р,Интеграция с участием блогера: 1 150 000р,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - сходка


> Как мы видим, часть ячеек заполнено как тестовыми, так и числовыми значениями "Интеграция с участием блогера: 650 000р", то есть указана и позиция (базово она у нас указана в столбце), и цены (базово они у нас указаны в строках). Нам надо перевести данные значения в отдельные столбцы, объединив похожие позиции

In [135]:
def custom_split_text(df: pd.DataFrame, column1, column2) -> pd.DataFrame:
    
    """
    Функция форматирует информацию по строкам, создавая новые столбцы на основе содержимого 
    ячеек в указанных столбцах. Основные шаги включают в себя извлечение числовых значений и
    текстовых описаний из указанных столбцов, их дальнейшую обработку и создание новых столбцов
    с отформатированными данными, где мы выделяем позицию, указанную в ячейках, и цену на данную позицию.
    В результате, функция предназначена для структурирования данных и улучшения их читаемости.
    
    
    ПАРАМЕТРЫ:
    - df (pd.DataFrame): Исходный DataFrame, содержащий данные.
    - column1 (str): Название первого столбца, из которого нужно извлечь информацию.
    - column2 (str): Название второго столбца, из которого нужно извлечь информацию.


    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    - filtered_df (pandas DataFrame): Новый DataFrame, содержащий обработанные данные, включая извлеченные числа и описания.

    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    filtered_df (pandas DataFrame): Новый DataFrame, содержащий обработанные данные,
    включая извлеченные числа и описания.
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    filtered_df = custom_split_text(vk_bloggers, 'стоимость интеграция 1 слот', 'стоимость интеграция 2 слот')

    """
    
    duplicates_filter = df[df.duplicated('блогер', keep=False)]

    filtered_df = duplicates_filter[['блогер', 
                                     column1,
                                     column2]]    


    for index in filtered_df.index:
        text = filtered_df.at[index, column1]

        # Поиск чисел в тексте
        numbers = re.findall(r'\d[\d\s]+', text)

        if numbers:
            # Извлечение первого найденного числа
            number = re.sub(r'[^0-9\s]', '', numbers[0])
            # Извлечение описания (текста без чисел)
            description = re.sub(r'[0-9]', '', text).strip()
            # Сохраняем числа и описание
            filtered_df.at[index, 'Позиция 1'] = description            
            filtered_df.at[index, 'Цена 1'] = number
        else:
            # Если нет чисел, сохраняем текст в столбце 'Позиция 1' и пустую строку в столбце 'цена 1'
            filtered_df.at[index, 'Позиция 1'] = text
            filtered_df.at[index, 'Цена 1'] = ''
            
    for index in filtered_df.index:
        text = filtered_df.at[index, column2]

        # Поиск чисел в тексте
        numbers = re.findall(r'\d[\d\s]+', text)

        if numbers:
            # Извлечение первого найденного числа
            number = re.sub(r'[^0-9\s]', '', numbers[0])
            # Извлечение описания (текста без чисел)
            description = re.sub(r'[0-9]', '', text).strip()
            # Сохраняем числа и описание
            filtered_df.at[index, 'Цена 2'] = number
            filtered_df.at[index, 'Позиция 2'] = description
        else:
            # Если нет чисел, сохраняем текст в столбце 'Позиция 1' и пустую строку в столбце 'цена 1'
            filtered_df.at[index, 'Позиция 2'] = text
            filtered_df.at[index, 'Цена 2'] = ''            
            
    for i in range(1, 3):
            # Удаление "р" и символов ".:()" из столбцов 'Позиция'
        filtered_df[f'Позиция {i}'] = filtered_df[f'Позиция {i}'].str.replace(r'\bр\.?\b', '', regex=True)
        filtered_df[f'Позиция {i}'] = filtered_df[f'Позиция {i}'].str.replace(r'[.:()]', '', regex=True)
            # Замена пустых строк на NaN
        filtered_df[f'Позиция {i}'] = filtered_df[f'Позиция {i}'].replace('', np.nan)
        filtered_df[f'Позиция {i}'] = filtered_df[f'Позиция {i}'].replace('по запросу', np.nan)    

        filtered_df[f'Цена {i}'] = filtered_df[f'Цена {i}'].replace('', np.nan)    
    
    return filtered_df

> Выведем новые значения в отдельный DataFrame

In [136]:
filtered_df = custom_split_text(vk_bloggers, 'стоимость интеграция 1 слот', 'стоимость интеграция 2 слот')

In [137]:
filtered_df.head()

,блогер,стоимость интеграция 1 слот,стоимость интеграция 2 слот,Позиция 1,Цена 1,Позиция 2,Цена 2
27,bubblegunshow - 9 панчей,Интеграция с участием актеров: 350 000р,по запросу,Интеграция с участием актеров,350 000,NaN,NaN
28,bubblegunshow - 9 панчей,Интеграция с участием блогера: 550 000р,Интеграция с участием блогера: 450 000р,Интеграция с участием блогера,550 000,Интеграция с участием блогера,450 000
29,bubblegunshow - арена,Интеграция с участием актеров: 600 000р,по запросу,Интеграция с участием актеров,600 000,NaN,NaN
30,bubblegunshow - арена,Интеграция с участием блогера: 750 000р,Интеграция с участием блогера: 650 000р,Интеграция с участием блогера,750 000,Интеграция с участием блогера,650 000
32,bubblegunshow - кто твой подписчик,Интеграция с участием актеров: 1 150 000р,Интеграция с участием актеров: 1 050 000р,Интеграция с участием актеров,1 150 000,Интеграция с участием актеров,1 050 000


> Сохраним индексы на будущее

In [138]:
indexes_to_filtered_df = list(filtered_df.index)

In [139]:
unique_position = filtered_df['блогер'].str.strip().unique()
len(unique_position)

10

In [140]:
filtered_df.head()

,блогер,стоимость интеграция 1 слот,стоимость интеграция 2 слот,Позиция 1,Цена 1,Позиция 2,Цена 2
27,bubblegunshow - 9 панчей,Интеграция с участием актеров: 350 000р,по запросу,Интеграция с участием актеров,350 000,NaN,NaN
28,bubblegunshow - 9 панчей,Интеграция с участием блогера: 550 000р,Интеграция с участием блогера: 450 000р,Интеграция с участием блогера,550 000,Интеграция с участием блогера,450 000
29,bubblegunshow - арена,Интеграция с участием актеров: 600 000р,по запросу,Интеграция с участием актеров,600 000,NaN,NaN
30,bubblegunshow - арена,Интеграция с участием блогера: 750 000р,Интеграция с участием блогера: 650 000р,Интеграция с участием блогера,750 000,Интеграция с участием блогера,650 000
32,bubblegunshow - кто твой подписчик,Интеграция с участием актеров: 1 150 000р,Интеграция с участием актеров: 1 050 000р,Интеграция с участием актеров,1 150 000,Интеграция с участием актеров,1 050 000


In [141]:
unique_position = filtered_df['Позиция 1'].str.strip().unique()
unique_position

array(['Интеграция с участием актеров', 'Интеграция с участием блогера',
       'Интеграция с участием комиков из выпуска',
       'Интеграция с участием Сергея Орлова', nan,
       'Интеграция с участием ведущих', 'Пушной, Крастер, Старый',
       'Мамикс', 'интеграция до  сек', 'устник до  сек'], dtype=object)

In [142]:
filtered_df.columns

Index(['блогер', 'стоимость интеграция 1 слот', 'стоимость интеграция 2 слот',
       'Позиция 1', 'Цена 1', 'Позиция 2', 'Цена 2'],
      dtype='object', name=0)

> Отчистим столбец с ценами от букв и сторонних символов, оставив только цифры

In [143]:
filtered_df['Позиция 1'] = filtered_df['Позиция 1'].fillna('')
filtered_df['Цена 1'].fillna(0, inplace=True)
# Уберем неразрывные пробелы из значений столбца 'Цена 1'
filtered_df['Цена 1'] = filtered_df['Цена 1'].str.replace('\xa0', '')
filtered_df['Цена 1'] = filtered_df['Цена 1'].str.replace(' ', '')
# Теперь преобразуем значения в числа с плавающей точкой
filtered_df['Цена 1'] = filtered_df['Цена 1'].astype(float)


filtered_df['Позиция 2'] = filtered_df['Позиция 2'].fillna('')
filtered_df['Цена 2'].fillna(0, inplace=True)
# Уберем неразрывные пробелы из значений столбца 'Цена 1'
filtered_df['Цена 2'] = filtered_df['Цена 2'].str.replace('\xa0', '')
filtered_df['Цена 2'] = filtered_df['Цена 2'].str.replace(' ', '')
# Теперь преобразуем значения в числа с плавающей точкой
filtered_df['Цена 2'] = filtered_df['Цена 2'].astype(float)


filtered_df['Позиция 1'] = filtered_df['Позиция 1'].str.strip()
filtered_df['Позиция 2'] = filtered_df['Позиция 2'].str.strip()

In [144]:
filtered_df.head()

,блогер,стоимость интеграция 1 слот,стоимость интеграция 2 слот,Позиция 1,Цена 1,Позиция 2,Цена 2
27,bubblegunshow - 9 панчей,Интеграция с участием актеров: 350 000р,по запросу,Интеграция с участием актеров,350000.0,,NaN
28,bubblegunshow - 9 панчей,Интеграция с участием блогера: 550 000р,Интеграция с участием блогера: 450 000р,Интеграция с участием блогера,550000.0,Интеграция с участием блогера,450000.0
29,bubblegunshow - арена,Интеграция с участием актеров: 600 000р,по запросу,Интеграция с участием актеров,600000.0,,NaN
30,bubblegunshow - арена,Интеграция с участием блогера: 750 000р,Интеграция с участием блогера: 650 000р,Интеграция с участием блогера,750000.0,Интеграция с участием блогера,650000.0
32,bubblegunshow - кто твой подписчик,Интеграция с участием актеров: 1 150 000р,Интеграция с участием актеров: 1 050 000р,Интеграция с участием актеров,1150000.0,Интеграция с участием актеров,1050000.0


In [145]:
def get_column_values_filtered_df(df: pd.DataFrame,
                                  key: int,
                                  column_with_position: str,
                                  column_with_price: str,
                                  column_list: str) -> pd.DataFrame:
    
    """
    АРГУМЕНТЫ ФУНКЦИИ:
    df - DataFrame, в котором делаем преобразования;
    key - Ключ в виде int, который нужно добавить к названию, чтобы в дальнейшем объединить несколько столбцов с одинаковой позицией;
    column_with_position - Столбец, в котором указаны позиции;
    column_with_price - Столбец, в котором указаны цены на позиции;
    
    
    ОПИСАНИЕ ВЫПОЛНЕНИЯ ФУНКЦИИ С ПРИМЕРОМ:
    
    Создаём список с уникальными значениями позиций, приведя их к единому формату, используя .strip(), .str.lower(), .str.replace:
    spisok = avtormedia_youtube_actual['Спешл'].str.strip()\
                                               .str.lower()\
                                               .str.replace('/\n', ', ')\
                                               .str.replace('\n', ', ')\
                                               .str.replace('*', '')\
                                               .unique()
    
    #Большинство значений строк в столбце column_with_position - NaN, у которых нет определённых цен.
    #Мы удалим данные позиции из списка, чтобы не создавать для них столбцы и/или не получать ошибку
    spisok = spisok[~pd.isna(spisok)]
        
        
    # Цикл for проходит по списку уникальных позиций и вводит их названия:
    for i in range(len(unique_position)):
    
    # Делаем уникальный столбец, название которого соотвествует уникальной позиции, заполняя его значениями соответсвующей цены:
        df[f"{unique_position[i]}_{key}"] = df[column_with_price].\
        where(df[column_with_position] == unique_position[i], other='')
    
    
    Удаляем исходные столбцы, чтобы не было повтора:
    df.drop(columns=[column_with_price, column_with_position], axis=1, inplace= True)
    
    
    Возвращаем обновлённый DataFrame
    return df    
                
    """
#     column_list = []

    unique_position = df[column_with_position].str.strip()\
                                              .unique()
    
    unique_position = unique_position[~pd.isna(unique_position)]
    
    for i in range(len(unique_position)):
        
        new_column_name = f'{unique_position[i]}_{key} слот'
        df[new_column_name] = \
                df[column_with_price].where(df[column_with_position] == unique_position[i], other='')

            
        # Добавляем имема новых столбцов в список, чтобы в дальнейшем работать с ними
        column_list.append(new_column_name) 
        
        # Заменим пустую строку на NaN
        df[new_column_name] = df[new_column_name].replace('', np.nan)
            
    df.drop(columns=[column_with_price, column_with_position], axis=1, inplace=True)
    
    return df

In [146]:
column_list = []

for key in range(1, 3):
    column_with_position = f'Позиция {key}'
    price_column = f'Цена {key}'
    filtered_df_new = get_column_values_filtered_df(filtered_df,
                                                    key,
                                                    column_with_position,
                                                    price_column,
                                                    column_list)

In [147]:
filtered_df_new.head()

,блогер,стоимость интеграция 1 слот,стоимость интеграция 2 слот,Интеграция с участием актеров_1 слот,Интеграция с участием блогера_1 слот,Интеграция с участием комиков из выпуска_1 слот,Интеграция с участием Сергея Орлова_1 слот,_1 слот,Интеграция с участием ведущих_1 слот,"Пушной, Крастер, Старый_1 слот",...,устник до сек_1 слот,_2 слот,Интеграция с участием блогера_2 слот,Интеграция с участием актеров_2 слот,Интеграция с участием комиков из выпуска_2 слот,Интеграция с участием Сергея Орлова_2 слот,Интеграция с участием ведущих_2 слот,-_2 слот,"Пушной, Крастер, Старый_2 слот",нет_2 слот
27,bubblegunshow - 9 панчей,Интеграция с участием актеров: 350 000р,по запросу,350000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,bubblegunshow - 9 панчей,Интеграция с участием блогера: 550 000р,Интеграция с участием блогера: 450 000р,NaN,550000.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,450000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,bubblegunshow - арена,Интеграция с участием актеров: 600 000р,по запросу,600000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,bubblegunshow - арена,Интеграция с участием блогера: 750 000р,Интеграция с участием блогера: 650 000р,NaN,750000.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,650000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,bubblegunshow - кто твой подписчик,Интеграция с участием актеров: 1 150 000р,Интеграция с участием актеров: 1 050 000р,1150000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1050000.0,NaN,NaN,NaN,NaN,NaN,NaN


In [148]:
filtered_df_new.columns

Index(['блогер', 'стоимость интеграция 1 слот', 'стоимость интеграция 2 слот',
       'Интеграция с участием актеров_1 слот',
       'Интеграция с участием блогера_1 слот',
       'Интеграция с участием комиков из выпуска_1 слот',
       'Интеграция с участием Сергея Орлова_1 слот', '_1 слот',
       'Интеграция с участием ведущих_1 слот',
       'Пушной, Крастер, Старый_1 слот', 'Мамикс_1 слот',
       'интеграция до  сек_1 слот', 'устник до  сек_1 слот', '_2 слот',
       'Интеграция с участием блогера_2 слот',
       'Интеграция с участием актеров_2 слот',
       'Интеграция с участием комиков из выпуска_2 слот',
       'Интеграция с участием Сергея Орлова_2 слот',
       'Интеграция с участием ведущих_2 слот', '-_2 слот',
       'Пушной, Крастер, Старый_2 слот', 'нет_2 слот'],
      dtype='object', name=0)

> Удалим столбцы, не несущие дополнительной информации

In [149]:
filtered_df_new.drop(columns=['стоимость интеграция 1 слот',
                              'стоимость интеграция 2 слот',
                              '_1 слот',
                              '_2 слот',
                              '-_2 слот',
                              'нет_2 слот'], axis=1, inplace=True)

In [150]:
filtered_df_new.head()

,блогер,Интеграция с участием актеров_1 слот,Интеграция с участием блогера_1 слот,Интеграция с участием комиков из выпуска_1 слот,Интеграция с участием Сергея Орлова_1 слот,Интеграция с участием ведущих_1 слот,"Пушной, Крастер, Старый_1 слот",Мамикс_1 слот,интеграция до сек_1 слот,устник до сек_1 слот,Интеграция с участием блогера_2 слот,Интеграция с участием актеров_2 слот,Интеграция с участием комиков из выпуска_2 слот,Интеграция с участием Сергея Орлова_2 слот,Интеграция с участием ведущих_2 слот,"Пушной, Крастер, Старый_2 слот"
27,bubblegunshow - 9 панчей,350000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,bubblegunshow - 9 панчей,NaN,550000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,450000.0,NaN,NaN,NaN,NaN,NaN
29,bubblegunshow - арена,600000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,bubblegunshow - арена,NaN,750000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,650000.0,NaN,NaN,NaN,NaN,NaN
32,bubblegunshow - кто твой подписчик,1150000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1050000.0,NaN,NaN,NaN,NaN


In [151]:
filtered_df_new.reset_index(inplace=True)
filtered_df_new.head()

,index,блогер,Интеграция с участием актеров_1 слот,Интеграция с участием блогера_1 слот,Интеграция с участием комиков из выпуска_1 слот,Интеграция с участием Сергея Орлова_1 слот,Интеграция с участием ведущих_1 слот,"Пушной, Крастер, Старый_1 слот",Мамикс_1 слот,интеграция до сек_1 слот,устник до сек_1 слот,Интеграция с участием блогера_2 слот,Интеграция с участием актеров_2 слот,Интеграция с участием комиков из выпуска_2 слот,Интеграция с участием Сергея Орлова_2 слот,Интеграция с участием ведущих_2 слот,"Пушной, Крастер, Старый_2 слот"
0,27,bubblegunshow - 9 панчей,350000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28,bubblegunshow - 9 панчей,NaN,550000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,450000.0,NaN,NaN,NaN,NaN,NaN
2,29,bubblegunshow - арена,600000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30,bubblegunshow - арена,NaN,750000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,650000.0,NaN,NaN,NaN,NaN,NaN
4,32,bubblegunshow - кто твой подписчик,1150000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1050000.0,NaN,NaN,NaN,NaN


> Сделаем два DataFrame с чётными и нечётными индексами, так как в них дублируются строки с одними и теми же блогерами

In [152]:
filtered_df_1 = filtered_df_new[filtered_df_new.index % 2 == 0]
filtered_df_2 = filtered_df_new[filtered_df_new.index % 2 != 0]

In [153]:
indexes_to_filtered_df_1 = filtered_df_1['index'].tolist()
indexes_to_filtered_df_1

[27, 29, 32, 36, 38, 42, 55, 58, 67, 71]

In [154]:
indexes_to_filtered_df_2 = filtered_df_2['index'].tolist()
indexes_to_filtered_df_2

[28, 30, 33, 37, 39, 43, 56, 59, 68, 72]

> С помощью merge объединим два получившихся DataFrame, чтобы не было дубликатов по блогерам

In [155]:
filtered_df_merge = pd.merge(filtered_df_1, filtered_df_2, on='блогер', how='outer')

In [156]:
filtered_df_merge.columns

Index(['index_x', 'блогер', 'Интеграция с участием актеров_1 слот_x',
       'Интеграция с участием блогера_1 слот_x',
       'Интеграция с участием комиков из выпуска_1 слот_x',
       'Интеграция с участием Сергея Орлова_1 слот_x',
       'Интеграция с участием ведущих_1 слот_x',
       'Пушной, Крастер, Старый_1 слот_x', 'Мамикс_1 слот_x',
       'интеграция до  сек_1 слот_x', 'устник до  сек_1 слот_x',
       'Интеграция с участием блогера_2 слот_x',
       'Интеграция с участием актеров_2 слот_x',
       'Интеграция с участием комиков из выпуска_2 слот_x',
       'Интеграция с участием Сергея Орлова_2 слот_x',
       'Интеграция с участием ведущих_2 слот_x',
       'Пушной, Крастер, Старый_2 слот_x', 'index_y',
       'Интеграция с участием актеров_1 слот_y',
       'Интеграция с участием блогера_1 слот_y',
       'Интеграция с участием комиков из выпуска_1 слот_y',
       'Интеграция с участием Сергея Орлова_1 слот_y',
       'Интеграция с участием ведущих_1 слот_y',
       'Пушн

> Удалим пустные столбцы

In [157]:
columns_with_all_nan = filtered_df_merge.columns[filtered_df_merge.isna().all()].tolist()
filtered_df_merge.drop(columns=columns_with_all_nan, axis=1, inplace=True)

In [158]:
filtered_df_merge.head()

,index_x,блогер,Интеграция с участием актеров_1 слот_x,Интеграция с участием комиков из выпуска_1 слот_x,"Пушной, Крастер, Старый_1 слот_x",интеграция до сек_1 слот_x,Интеграция с участием актеров_2 слот_x,Интеграция с участием комиков из выпуска_2 слот_x,"Пушной, Крастер, Старый_2 слот_x",index_y,Интеграция с участием блогера_1 слот_y,Интеграция с участием Сергея Орлова_1 слот_y,Интеграция с участием ведущих_1 слот_y,Мамикс_1 слот_y,устник до сек_1 слот_y,Интеграция с участием блогера_2 слот_y,Интеграция с участием Сергея Орлова_2 слот_y,Интеграция с участием ведущих_2 слот_y
0,27,bubblegunshow - 9 панчей,350000.0,NaN,NaN,NaN,NaN,NaN,NaN,28,550000.0,NaN,NaN,NaN,NaN,450000.0,NaN,NaN
1,29,bubblegunshow - арена,600000.0,NaN,NaN,NaN,NaN,NaN,NaN,30,750000.0,NaN,NaN,NaN,NaN,650000.0,NaN,NaN
2,32,bubblegunshow - кто твой подписчик,1150000.0,NaN,NaN,NaN,1050000.0,NaN,NaN,33,1550000.0,NaN,NaN,NaN,NaN,1350000.0,NaN,NaN
3,36,bubblegunshow - удиви меня,1150000.0,NaN,NaN,NaN,1050000.0,NaN,NaN,37,1450000.0,NaN,NaN,NaN,NaN,1250000.0,NaN,NaN
4,38,bubblegunshow - сходка,650000.0,NaN,NaN,NaN,NaN,NaN,NaN,39,1250000.0,NaN,NaN,NaN,NaN,1150000.0,NaN,NaN


> Удалим столбцы со старыми индексами

In [159]:
filtered_df_merge.drop(columns=['index_x', 'index_y'], axis=1, inplace=True)

In [160]:
filtered_df_merge.head()

,блогер,Интеграция с участием актеров_1 слот_x,Интеграция с участием комиков из выпуска_1 слот_x,"Пушной, Крастер, Старый_1 слот_x",интеграция до сек_1 слот_x,Интеграция с участием актеров_2 слот_x,Интеграция с участием комиков из выпуска_2 слот_x,"Пушной, Крастер, Старый_2 слот_x",Интеграция с участием блогера_1 слот_y,Интеграция с участием Сергея Орлова_1 слот_y,Интеграция с участием ведущих_1 слот_y,Мамикс_1 слот_y,устник до сек_1 слот_y,Интеграция с участием блогера_2 слот_y,Интеграция с участием Сергея Орлова_2 слот_y,Интеграция с участием ведущих_2 слот_y
0,bubblegunshow - 9 панчей,350000.0,NaN,NaN,NaN,NaN,NaN,NaN,550000.0,NaN,NaN,NaN,NaN,450000.0,NaN,NaN
1,bubblegunshow - арена,600000.0,NaN,NaN,NaN,NaN,NaN,NaN,750000.0,NaN,NaN,NaN,NaN,650000.0,NaN,NaN
2,bubblegunshow - кто твой подписчик,1150000.0,NaN,NaN,NaN,1050000.0,NaN,NaN,1550000.0,NaN,NaN,NaN,NaN,1350000.0,NaN,NaN
3,bubblegunshow - удиви меня,1150000.0,NaN,NaN,NaN,1050000.0,NaN,NaN,1450000.0,NaN,NaN,NaN,NaN,1250000.0,NaN,NaN
4,bubblegunshow - сходка,650000.0,NaN,NaN,NaN,NaN,NaN,NaN,1250000.0,NaN,NaN,NaN,NaN,1150000.0,NaN,NaN


> Объединим получившийся DataFrame с исходным по столбцу "блогер"

In [161]:
vk_bloggers = vk_bloggers.merge(filtered_df_merge, on='блогер', how='outer')

In [162]:
indexes_to_filtered_df_2

[28, 30, 33, 37, 39, 43, 56, 59, 68, 72]

In [163]:
indexes_to_filtered_df_1

[27, 29, 32, 36, 38, 42, 55, 58, 67, 71]

In [164]:
# Проверим на дубликаты

duplicates = vk_bloggers[vk_bloggers.duplicated('блогер', keep=False)]
duplicates.head()

,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика,блогер,...,Интеграция с участием комиков из выпуска_2 слот_x,"Пушной, Крастер, Старый_2 слот_x",Интеграция с участием блогера_1 слот_y,Интеграция с участием Сергея Орлова_1 слот_y,Интеграция с участием ведущих_1 слот_y,Мамикс_1 слот_y,устник до сек_1 слот_y,Интеграция с участием блогера_2 слот_y,Интеграция с участием Сергея Орлова_2 слот_y,Интеграция с участием ведущих_2 слот_y
26,600 000,Интеграция с участием актеров: 350 000р,по запросу,по запросу,р.350 000,р.250 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - 9 панчей,...,NaN,NaN,550000.0,NaN,NaN,NaN,NaN,450000.0,NaN,NaN
27,NaN,Интеграция с участием блогера: 550 000р,Интеграция с участием блогера: 450 000р,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - 9 панчей,...,NaN,NaN,550000.0,NaN,NaN,NaN,NaN,450000.0,NaN,NaN
28,700 000,Интеграция с участием актеров: 600 000р,по запросу,по запросу,р.450 000,р.350 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - арена,...,NaN,NaN,750000.0,NaN,NaN,NaN,NaN,650000.0,NaN,NaN
29,NaN,Интеграция с участием блогера: 750 000р,Интеграция с участием блогера: 650 000р,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - арена,...,NaN,NaN,750000.0,NaN,NaN,NaN,NaN,650000.0,NaN,NaN
31,1 600 000,Интеграция с участием актеров: 1 150 000р,Интеграция с участием актеров: 1 050 000р,по запросу,р.1 000 000,р.800 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - кто твой подписчик,...,NaN,NaN,1550000.0,NaN,NaN,NaN,NaN,1350000.0,NaN,NaN


> Отчистим столбцы 'стоимость интеграции 1-й слот', 'стоимость интеграции 2-й слот' от тех значений, которые мы перевели в отдельные столбцы

In [165]:
# Создаем паттерн для поиска строк, содержащих 'интеграция', 'устник' и '('
pattern = r'интеграция|устник|\(.*\)'  # Это регулярное выражение будет искать любое из указанных слов или '('

# Определите список слотов, которые вы хотите обработать
slots = ['стоимость интеграция 1 слот', 'стоимость интеграция 2 слот']

for slot_number, slot_column in enumerate(slots, start=1):
    
    for i in range (1,3):
        vk_bloggers[f'стоимость интеграция {i} слот'] = vk_bloggers[f'стоимость интеграция {i} слот'].str.lower()
    
    # Применяем фильтрацию с использованием регулярного выражения
    filtered_duplicates = vk_bloggers[slot_column].str.contains(pattern, case=False)

    # Извлекаем значения в виде списка
    duplicates_position = vk_bloggers[filtered_duplicates][slot_column].tolist()

    # Заменяем найденные значения на np.nan
    for i in range(len(duplicates_position)):
        vk_bloggers[slot_column] = vk_bloggers[slot_column].replace(duplicates_position[i], np.nan)

In [166]:
duplicates = vk_bloggers[vk_bloggers.duplicated('блогер', keep=False)]
duplicates.head()

,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика,блогер,...,Интеграция с участием комиков из выпуска_2 слот_x,"Пушной, Крастер, Старый_2 слот_x",Интеграция с участием блогера_1 слот_y,Интеграция с участием Сергея Орлова_1 слот_y,Интеграция с участием ведущих_1 слот_y,Мамикс_1 слот_y,устник до сек_1 слот_y,Интеграция с участием блогера_2 слот_y,Интеграция с участием Сергея Орлова_2 слот_y,Интеграция с участием ведущих_2 слот_y
26,600 000,NaN,по запросу,по запросу,р.350 000,р.250 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - 9 панчей,...,NaN,NaN,550000.0,NaN,NaN,NaN,NaN,450000.0,NaN,NaN
27,NaN,NaN,NaN,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - 9 панчей,...,NaN,NaN,550000.0,NaN,NaN,NaN,NaN,450000.0,NaN,NaN
28,700 000,NaN,по запросу,по запросу,р.450 000,р.350 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - арена,...,NaN,NaN,750000.0,NaN,NaN,NaN,NaN,650000.0,NaN,NaN
29,NaN,NaN,NaN,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - арена,...,NaN,NaN,750000.0,NaN,NaN,NaN,NaN,650000.0,NaN,NaN
31,1 600 000,NaN,NaN,по запросу,р.1 000 000,р.800 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - кто твой подписчик,...,NaN,NaN,1550000.0,NaN,NaN,NaN,NaN,1350000.0,NaN,NaN


> Удалим дубликаты из основного DataFrame vk_bloggers по индексам

In [167]:
indexes_to_duplicates = duplicates[duplicates.index % 2 == 0]
indexes_to_duplicates = indexes_to_duplicates.index.tolist()
indexes_to_duplicates

[26, 28, 32, 36, 38, 42, 54, 58, 66, 70]

In [168]:
vk_bloggers.drop(indexes_to_duplicates, inplace=True)

> Отлично, мы избавились от дубликатов и от неправильных значений в столбцe с ценами на интеграцию!

In [169]:
# Проверим на дубликаты

duplicates = vk_bloggers[vk_bloggers.duplicated('блогер', keep=False)]
duplicates

,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика,блогер,...,Интеграция с участием комиков из выпуска_2 слот_x,"Пушной, Крастер, Старый_2 слот_x",Интеграция с участием блогера_1 слот_y,Интеграция с участием Сергея Орлова_1 слот_y,Интеграция с участием ведущих_1 слот_y,Мамикс_1 слот_y,устник до сек_1 слот_y,Интеграция с участием блогера_2 слот_y,Интеграция с участием Сергея Орлова_2 слот_y,Интеграция с участием ведущих_2 слот_y


> В тоже время мы сохранили все значение

In [170]:
vk_bloggers.loc[vk_bloggers['блогер'] == 'vk video - кошмарный сон а4']

,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика,блогер,...,Интеграция с участием комиков из выпуска_2 слот_x,"Пушной, Крастер, Старый_2 слот_x",Интеграция с участием блогера_1 слот_y,Интеграция с участием Сергея Орлова_1 слот_y,Интеграция с участием ведущих_1 слот_y,Мамикс_1 слот_y,устник до сек_1 слот_y,Интеграция с участием блогера_2 слот_y,Интеграция с участием Сергея Орлова_2 слот_y,Интеграция с участием ведущих_2 слот_y
71,NaN,NaN,нет,NaN,NaN,NaN,Варвара Каинова - v.kainova@vk.team,https://vk.com/vkvideo,https://drive.google.com/drive/folders/1BpIYrN...,vk video - кошмарный сон а4,...,NaN,NaN,NaN,NaN,NaN,NaN,2500000.0,NaN,NaN,NaN


> Переименуем столбцы, чтобы подвести их к формату DataFrame bufer

In [171]:
[col for col in vk_bloggers]

['kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза)',
 'стоимость интеграция 1 слот',
 'стоимость интеграция 2 слот',
 'стоимость продакт плейсмент',
 'стоимость преролл до 20 сек',
 'стоимость мидролл до 20 сек',
 'контакты менеджера по продажам',
 'ссылка',
 'статистика',
 'блогер',
 'Интеграция с участием актеров_1 слот_x',
 'Интеграция с участием комиков из выпуска_1 слот_x',
 'Пушной, Крастер, Старый_1 слот_x',
 'интеграция до  сек_1 слот_x',
 'Интеграция с участием актеров_2 слот_x',
 'Интеграция с участием комиков из выпуска_2 слот_x',
 'Пушной, Крастер, Старый_2 слот_x',
 'Интеграция с участием блогера_1 слот_y',
 'Интеграция с участием Сергея Орлова_1 слот_y',
 'Интеграция с участием ведущих_1 слот_y',
 'Мамикс_1 слот_y',
 'устник до  сек_1 слот_y',
 'Интеграция с участием блогера_2 слот_y',
 'Интеграция с участием Сергея Орлова_2 слот_y',
 'Интеграция с участием ведущих_2 слот_y']

In [172]:
vk_bloggers.rename(columns = {'kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза)':'среднее кол-во просмотров',
                              'контакты менеджера по продажам':'контакты менеджера',
                              'Интеграция с участием актеров_1 слот_x':'стоимость интеграция с участием актеров 1 слот',
                              'Интеграция с участием комиков из выпуска_1 слот_x':'стоимость интеграция с участием комиков из выпуска 1 слот',
                              'Пушной, Крастер, Старый_1 слот_x':'стоимость интеграция с участием Пушного, Крастера, Старого 1 слот',
                              'интеграция до  сек_1 слот_x':'стоимость интеграция до 60 секунд 1 слот',
                              'Интеграция с участием актеров_2 слот_x':'стоимость интеграция с участием актеров 2 слот',
                              'Интеграция с участием комиков из выпуска_2 слот_x':'стоимость интеграция с участием комиков из выпуска 2 слот',
                              'Пушной, Крастер, Старый_2 слот_x':'стоимость интеграция с участием Пушного, Крастера, Старого 2 слот',
                              'Интеграция с участием блогера_1 слот_y':'стоимость интеграция с участием блогера 1 слот',
                              'Интеграция с участием Сергея Орлова_1 слот_y':'стоимость интеграция с участием Сергея Орлова 1 слот',
                              'Интеграция с участием ведущих_1 слот_y':'стоимость интеграция с участием ведущих 1 слот',
                              'Мамикс_1 слот_y':'стоимость интеграция с участием Мамикса 1 слот',
                              'устник до  сек_1 слот_y':'стоимость устник до 30 сек 1 слот',
                              'Интеграция с участием блогера_2 слот_y':'стоимость интеграция с участием блогера 2 слот',
                              'Интеграция с участием Сергея Орлова_2 слот_y':'стоимость интеграция с участием Сергея Орлова 2 слот',
                              'Интеграция с участием ведущих_2 слот_y':'стоимость интеграция с участием ведущих 2 слот'}, inplace = True)

In [173]:
bufer_columns = ['среднее кол-во просмотров',
                 'стоимость интеграция 1 слот',
                 'стоимость интеграция 2 слот',
                 'стоимость продакт плейсмент',
                 'стоимость преролл до 20 сек',
                 'стоимость мидролл до 20 сек',
                 'контакты менеджера',
                 'ссылка',
                 'статистика',
                 'блогер']

In [174]:
ok_bloggers.columns = bufer_columns

### Восьмое объединение

In [175]:
bufer = merge_bufer_df(bufer, vk_bloggers, 'VK Видео')

Количество дубликатов: 0


In [176]:
# Заполним столбец "контакты менеджера" данными из столбца "VK Видео контакты менеджера"
bufer['контакты менеджера'] = bufer['контакты менеджера'].fillna(bufer['VK Видео контакты менеджера'])

# Удалить столбец "VK Видео контакты менеджера"
bufer.drop(['VK Видео контакты менеджера'], axis=1, inplace=True)

#Заполним значение налога
bufer['налог'].fillna('НДС (20%)', inplace=True)

### Девятое объединение

In [177]:
common_columns_vk_bloggers = bufer.columns.intersection(ok_bloggers.columns)
print(common_columns_vk_bloggers)

Index(['блогер', 'контакты менеджера'], dtype='object')


> У нас нет общих столбцов, так как к тем, что выше, мы добавим название соц сети ещё, но мы это предугадали в функции merge_bufer_df

In [178]:
bufer = merge_bufer_df(bufer, ok_bloggers, 'OK Шоу')

Количество дубликатов: 0


In [179]:
# Заполним столбец "контакты менеджера" данными из столбца "VK Видео контакты менеджера"
bufer['контакты менеджера'] = bufer['контакты менеджера'].fillna(bufer['OK Шоу контакты менеджера'])

# Удалить столбец "OK Шоу контакты менеджера"
bufer.drop(['OK Шоу контакты менеджера'], axis=1, inplace=True)

#Заполним значение налога
bufer['налог'].fillna('НДС (20%)', inplace=True)

In [180]:
bufer.tail(10)

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,VK Видео стоимость интеграция с участием ведущих 2 слот,OK Шоу среднее кол-во просмотров,OK Шоу стоимость интеграция 1 слот,OK Шоу стоимость интеграция 2 слот,OK Шоу стоимость продакт плейсмент,OK Шоу стоимость преролл до 20 сек,OK Шоу стоимость мидролл до 20 сек,OK Шоу ссылка,OK Шоу статистика,OK Шоу блогер
155,dancetory - дэнс или не дэнс,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156,big russian boss - авто.тюн,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157,ок video - в мире домашних животных,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5 000 000,р.1 150 000,по запросу,нет,р.800 000,нет,https://ok.ru/oknutye,None,ок video - в мире домашних животных
158,"ок video - ок, мама, поехали!",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5 000 000,р.700 000,по запросу,по запросу,нет,нет,https://ok.ru/travelok,None,"ок video - ок, мама, поехали!"
159,ок video - психология кино,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2 500 000,р.800 000,р.700 000,по запросу,р.500 000,р.400 000,https://ok.ru/travelok,None,ок video - психология кино
160,ок video - полтора спортсмена,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5 000 000,р.1 100 000,по запросу,по запросу,р.500 000,нет,https://ok.ru/oknutye,None,ок video - полтора спортсмена
161,ок video - шеф против любителя,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5 000 000,р.1 100 000,по запросу,р.800 000,р.500 000,нет,https://ok.ru/oknutye,None,ок video - шеф против любителя
162,ок video - кинокухня,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5 000 000,р.1 100 000,по запросу,по запросу,р.500 000,нет,https://ok.ru/oknutye,None,ок video - кинокухня
163,"ок video - шоу, где звезды играют в игры со св...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5 000 000,нет,нет,нет,р.1 500 000,р.1 200 000,https://ok.ru/oknutye,None,"ок video - шоу, где звезды играют в игры со св..."
164,ок video - ок на связи!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5 000 000,р.1 500 000,р.1 000 000,по запросу,р.800 000,р.600 000,https://ok.ru/oknutye,None,ок video - ок на связи!


## Подготовка данных по блогерам от BlogoSphere

### Подготовка таблицы blogosphere_youtube

In [181]:
blogosphere_youtube.head()

5,NaN,Название канала,Шоу,Подписчики,Среднее кол-во просмотров на видео,Статистика аудитории,Стандартные форматы,NaN,NaN,NaN,...,NaN,NaN,NaN,Пример рекламы 1,Пример рекламы 2,Пример рекламы 3,Среднее CPV,NaN,Ссылка,Статистика ссылка
6,NaN,NaN,NaN,NaN,NaN,NaN,Формат 1,NaN,Формат 2,NaN,...,NaN,Формат 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,Описание,Цена* от,Описание,Цена* от,...,Цена* от,Описание,Цена* от,NaN,NaN,NaN,Стандартный Формат 1,Стандартный Формат 2,NaN,NaN
8,Э,NaN,"Осторожно, Собчак",NaN,1 000 000,NaN,1 слот,1 310 000 ₽,2 слот,960 000 ₽,...,1 310 000 ₽,Product Placement,1 470 000 ₽,Eduson,Экоплант,Травелата,"1,31₽","0,96₽",NaN,NaN
9,NaN,Ксения Собчак,Осторожно: новости!,3 430 000,550 000,Подробнее,1 слот,760 000 ₽,2 Слот,510 000 ₽,...,NaN,NaN,NaN,Astons,CGPods,SkillFactory,"1,38₽","0,93₽",https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...
10,NaN,NaN,Красота требует КЭШ,NaN,800 000,NaN,1 слот,960 000 ₽,2 Слот,860 000 ₽,...,NaN,NaN,NaN,Золотое яблоко,4FORMS,FOREO,"1,20₽","1,08₽",NaN,NaN


> Переименуем сразу имена столбцов, чтобы нам было проще ориентироваться в данных

In [182]:
[col for col in bufer.columns if 'YouTube' in col]

['YouTube ссылка',
 'YouTube тематика',
 'YouTube количество подписчиков',
 'YouTube среднее кол-во просмотров видео',
 'YouTube среднее кол-во просмотров shorts',
 'YouTube среднее кол-во лайков',
 'YouTube среднее кол-во комментариев',
 'YouTube стоимость интеграция 1 слот',
 'YouTube стоимость продакт плейсмент',
 'YouTube стоимость преролл до 30 сек',
 'YouTube стоимость мидролл до 30 сек',
 'YouTube стоимость shorts до 1 мин',
 'YouTube статистика',
 'YouTube блогер',
 'YouTube стоимость интеграция 2 слот']

In [183]:
bufer_columns = ['Эксклюзивность',
                 'Название канала',
                 'Шоу',
                 'количество подписчиков',
                 'среднее кол-во просмотров',
                 'Статистика аудитории',
                 'Ф1 позиция',
                 'Ф1 Цена от',
                 'Ф2 позиция',
                 'Ф2 Цена от',
                 'Ф3 позиция',
                 'Ф3 Цена от',
                 'Ф4 позиция',
                 'Ф4 Цена от',
                 'Ф5 позиция',
                 'Ф5 Цена от',                 
                 'Пример рекламы 1',
                 'Пример рекламы 2',
                 'Пример рекламы 3',
                 'Среднее CPV формат 1',
                 'Среднее CPV формат 2',
                 'ссылка',
                 'Статистика ссылка']

In [184]:
blogosphere_youtube.columns = bufer_columns

> Сделаем предобработку DataFrame

In [185]:
key_word = ['Эксклюзивность', # Список ключевых слов для поиска столбцов и их удаления
            'Пример рекламы 1',
            'Пример рекламы 2',
            'Пример рекламы 3',
            'Среднее CPV формат 1',
            'Среднее CPV формат 2'] 

drop_word = '' # Список ключевых слов для их удаления из названия столбцов

upd_columns = ['Название канала', 'Шоу'] # Список столбцов с именами блогеров для приведения к единому формату

In [186]:
blogosphere_youtube = pretraining_df(blogosphere_youtube, key_word, drop_word, upd_columns)

In [187]:
blogosphere_youtube.head()

,название канала,шоу,количество подписчиков,среднее кол-во просмотров,статистика аудитории,ф1 позиция,ф1 цена от,ф2 позиция,ф2 цена от,ф3 позиция,ф3 цена от,ф4 позиция,ф4 цена от,ф5 позиция,ф5 цена от,ссылка,статистика ссылка
6,NaN,NaN,NaN,NaN,NaN,Формат 1,NaN,Формат 2,NaN,Формат 3,NaN,Формат 4,NaN,Формат 5,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,Описание,Цена* от,Описание,Цена* от,Описание,Цена* от,Описание,Цена* от,Описание,Цена* от,NaN,NaN
8,NaN,"осторожно, собчак",NaN,1 000 000,NaN,1 слот,1 310 000 ₽,2 слот,960 000 ₽,3 слот,860 000 ₽,Преролл,1 310 000 ₽,Product Placement,1 470 000 ₽,NaN,NaN
9,ксения собчак,осторожно: новости!,3 430 000,550 000,Подробнее,1 слот,760 000 ₽,2 Слот,510 000 ₽,Преролл,560 000 ₽,NaN,NaN,NaN,NaN,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...
10,NaN,красота требует кэш,NaN,800 000,NaN,1 слот,960 000 ₽,2 Слот,860 000 ₽,Преролл,660 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN


In [189]:
blogosphere_youtube[['название канала',
                     'шоу',
                     'количество подписчиков',
                     'среднее кол-во просмотров',
                     'статистика аудитории',
                     'ссылка',
                     'статистика ссылка']] = \
                    blogosphere_youtube[['название канала',
                                         'шоу',
                                         'количество подписчиков',
                                         'среднее кол-во просмотров',
                                         'статистика аудитории',
                                         'ссылка',
                                         'статистика ссылка']].fillna(method='ffill')

> Удалим первые две строки, так как в них указано неполное название столбцов и пока что в них актуальные пропуски, пока мы не запомнили "вверх"

In [190]:
blogosphere_youtube.dropna(subset=['шоу'], inplace=True)
blogosphere_youtube.reset_index(drop=True, inplace=True)

In [192]:
blogosphere_youtube[['название канала',
                     'шоу',
                     'количество подписчиков',
                     'среднее кол-во просмотров',
                     'статистика аудитории',
                     'ссылка',
                     'статистика ссылка']] = \
                    blogosphere_youtube[['название канала',
                                         'шоу',
                                         'количество подписчиков',
                                         'среднее кол-во просмотров',
                                         'статистика аудитории',
                                         'ссылка',
                                         'статистика ссылка']].fillna(method='backfill')

In [193]:
blogosphere_youtube.head()

,название канала,шоу,количество подписчиков,среднее кол-во просмотров,статистика аудитории,ф1 позиция,ф1 цена от,ф2 позиция,ф2 цена от,ф3 позиция,ф3 цена от,ф4 позиция,ф4 цена от,ф5 позиция,ф5 цена от,ссылка,статистика ссылка
0,ксения собчак,"осторожно, собчак",3 430 000,1 000 000,Подробнее,1 слот,1 310 000 ₽,2 слот,960 000 ₽,3 слот,860 000 ₽,Преролл,1 310 000 ₽,Product Placement,1 470 000 ₽,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...
1,ксения собчак,осторожно: новости!,3 430 000,550 000,Подробнее,1 слот,760 000 ₽,2 Слот,510 000 ₽,Преролл,560 000 ₽,NaN,NaN,NaN,NaN,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...
2,ксения собчак,красота требует кэш,3 430 000,800 000,Подробнее,1 слот,960 000 ₽,2 Слот,860 000 ₽,Преролл,660 000 ₽,NaN,NaN,NaN,NaN,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...
3,rogov live,обзоры,460 000,400 000,Подробнее,1 слот\nссылка после описания ролика,1 005 000 ₽,2 слот\nссылка после описания ролика,905 000 ₽,3 слот,905 000 ₽,Преролл,1 005 000 ₽,Мидролл,905 000 ₽,https://www.youtube.com/channel/UCefO_zFbQc6-z...,https://drive.google.com/drive/folders/1-PTy90...
4,без фильтров,интервью,20 000,100 000,Подробнее,1 слот,240 000 ₽,2 слот,210 000 ₽,NaN,NaN,Преролл,115 000 ₽,Мидролл,115 000 ₽,https://www.youtube.com/@bezfiltrovshow/,https://drive.google.com/drive/u/0/folders/1Gg...


> У нас есть ещё одна интересная особенность в данных: реклама в Shorts сделана как отдельная тематика видео, а не как стоимость. Сделаем из неё отдельный DataFrame, чтобы потом объединить его с основным DataFrame

In [194]:
blogosphere_youtube_shorts = blogosphere_youtube.loc[blogosphere_youtube['шоу'] == 'shorts']
blogosphere_youtube_shorts

,название канала,шоу,количество подписчиков,среднее кол-во просмотров,статистика аудитории,ф1 позиция,ф1 цена от,ф2 позиция,ф2 цена от,ф3 позиция,ф3 цена от,ф4 позиция,ф4 цена от,ф5 позиция,ф5 цена от,ссылка,статистика ссылка
19,александр пушной,shorts,1 780 000,200 000,Подробнее,Интеграция,300 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/c/PushnoyRU,https://drive.google.com/drive/folders/1-UqeBp...
42,дневник дизайнера,shorts,985 000,100 000,Подробнее,1 cлот,1 000 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/channel/UCbhIVqiD-Know...,https://drive.google.com/drive/folders/1-enyaA...
71,василий уткин,shorts,615 000,60 000,Подробнее,Ролик до 1 мин,84 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/channel/UCwrRor4IvrByH...,https://drive.google.com/drive/folders/11uM9Ql...
79,джарахов,shorts,3 740 000,60 000,Подробнее,1 слот,505 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/@dzharakhov/videos,https://drive.google.com/drive/folders/1x-28-y...
81,comigration,shorts,91 100,100 000,Подробнее,1 cлот,цена обсуждаема,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/@comigration,https://drive.google.com/drive/folders/13_EecF...
86,на шуме,shorts,219 000,200 000,Подробнее,1 слот,150 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://youtube.com/@nashume,https://drive.google.com/drive/folders/1jFhKc6...
89,nellifornication,shorts,2 500 000,1 000 000,Подробнее,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://youtube.com/@nashume,https://drive.google.com/drive/folders/1jFhKc6...


> Переименуем столбцы

In [195]:
blogosphere_youtube_shorts.rename(columns = {'ф1 цена от':'стоимость shorts до 1 мин',
                                             'среднее кол-во просмотров':'среднее кол-во просмотров shorts'
                                            }, inplace=True)

In [196]:
blogosphere_youtube_shorts.columns

Index(['название канала', 'шоу', 'количество подписчиков',
       'среднее кол-во просмотров shorts', 'статистика аудитории',
       'ф1 позиция', 'стоимость shorts до 1 мин', 'ф2 позиция', 'ф2 цена от',
       'ф3 позиция', 'ф3 цена от', 'ф4 позиция', 'ф4 цена от', 'ф5 позиция',
       'ф5 цена от', 'ссылка', 'статистика ссылка'],
      dtype='object')

In [197]:
columns_with_shorts = ['название канала',
                       'среднее кол-во просмотров shorts',
                       'стоимость shorts до 1 мин']

# Удаление всех столбцов, кроме тех, что в списке columns_to_keep
blogosphere_youtube_shorts = blogosphere_youtube_shorts[columns_with_shorts]
blogosphere_youtube_shorts

,название канала,среднее кол-во просмотров shorts,стоимость shorts до 1 мин
19,александр пушной,200 000,300 000 ₽
42,дневник дизайнера,100 000,1 000 000 ₽
71,василий уткин,60 000,84 000 ₽
79,джарахов,60 000,505 000 ₽
81,comigration,100 000,цена обсуждаема
86,на шуме,200 000,150 000 ₽
89,nellifornication,1 000 000,NaN


> Удалим данные строки из основного DataFrame

In [198]:
blogosphere_youtube = blogosphere_youtube.drop(blogosphere_youtube[blogosphere_youtube['шоу'] == 'shorts'].index)

#### Перейдём к основному DataFrame

> Сделаем новые столбцы для уникальных форматов, которые указаны в столбцах  'Ф1 позиция', 'Ф2 позиция', 'Ф3 позиция', 'Ф4 позиция',  'Ф5 позиция', указав также их цены из столбцов 'Ф1 Цена от', 'Ф2 Цена от', 'Ф3 Цена от', 'Ф4 Цена от', 'Ф5 Цена от'.   
> В некоторых новых столбцах получится по 1-5 цен, но так как это уникальный формат и он может встретиться в будущем, мы их оставим.  
> Некоторые позиции могут повторяться в разных столбцах, поэтому для разных столбцов с позициями мы будем указывать ключ с int, чтобы в дальнейшем объединить столбцы с одинакомыми позициями

> Так как у нас повторяются столцы и индексы, которые мы хотели бы использовать, то сделаем через цикл

In [199]:
def get_column_values(df: pd.DataFrame,
                      key: int,
                      column_with_position: str,
                      column_with_price: str,
                      column_list: str) -> pd.DataFrame:
    
    """
    АРГУМЕНТЫ ФУНКЦИИ:
    df - DataFrame, в котором делаем преобразования;
    key - Ключ в виде int, который нужно добавить к названию, 
          чтобы в дальнейшем объединить несколько столбцов с одинаковой позицией;
    column_with_position - Столбец, в котором указаны позиции;
    column_with_price - Столбец, в котором указаны цены на позиции;
    
    
    ОПИСАНИЕ ВЫПОЛНЕНИЯ ФУНКЦИИ С ПРИМЕРОМ:
    
    Создаём список с уникальными значениями позиций, приведя их к единому формату, используя .strip(), .str.lower(), .str.replace:
    spisok = avtormedia_youtube_actual['Спешл'].str.strip()\
                                               .str.lower()\
                                               .str.replace('/\n', ', ')\
                                               .str.replace('\n', ', ')\
                                               .str.replace('*', '')\
                                               .unique()
    
    #Большинство значений строк в столбце column_with_position - NaN, у которых нет определённых цен.
    #Мы удалим данные позиции из списка, чтобы не создавать для них столбцы и/или не получать ошибку
    spisok = spisok[~pd.isna(spisok)]
        
        
    # Цикл for проходит по списку уникальных позиций и вводит их названия:
    for i in range(len(unique_position)):
    
    # Делаем уникальный столбец, название которого соотвествует уникальной позиции, заполняя его значениями соответсвующей цены:
        df[f"{unique_position[i]}_{key}"] = df[column_with_price].\
        where(df[column_with_position] == unique_position[i], other='')
    
    
    Удаляем исходные столбцы, чтобы не было повтора:
    df.drop(columns=[column_with_price, column_with_position], axis=1, inplace= True)
    
    
    Возвращаем обновлённый DataFrame
    return df    
                
    """
    
    unique_position = df[column_with_position].str.strip()\
                                              .unique()
    
    unique_position = unique_position[~pd.isna(unique_position)]
    
    for i in range(len(unique_position)):
        new_column_name = f'{unique_position[i]}_{key}'
        df[new_column_name] = \
                df[column_with_price].where(df[column_with_position] == unique_position[i], other='')

            
        # Добавляем имема новых столбцов в список, чтобы в дальнейшем работать с ними
        column_list.append(new_column_name) 
        
        # Заменим пустую строку на NaN
        df[new_column_name] = df[new_column_name].replace('', np.nan)
            
    df.drop(columns=[column_with_price, column_with_position], axis=1, inplace=True)
    
    return df

In [200]:
column_list = []

for column_number in range(1, 6):
    position_column = f'ф{column_number} позиция'
    price_column = f'ф{column_number} цена от'
    blogosphere_youtube = get_column_values(blogosphere_youtube,
                                            column_number,
                                            position_column,
                                            price_column,
                                            column_list)

In [201]:
blogosphere_youtube.head()

,название канала,шоу,количество подписчиков,среднее кол-во просмотров,статистика аудитории,ссылка,статистика ссылка,1 слот_1,1 слот\nссылка после описания ролика_1,Мини-интеграция до 1 мин_1,...,Выпуск под клиента_3,Преролл_4,Мидролл_4,Преролл \nдо 20 сек_4,Выпуск под клиента_4,Product Placement_5,Мидролл_5,Спецвыпуск_5,Спонсорство_5,Product placement_5
0,ксения собчак,"осторожно, собчак",3 430 000,1 000 000,Подробнее,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...,1 310 000 ₽,NaN,NaN,...,NaN,1 310 000 ₽,NaN,NaN,NaN,1 470 000 ₽,NaN,NaN,NaN,NaN
1,ксения собчак,осторожно: новости!,3 430 000,550 000,Подробнее,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...,760 000 ₽,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ксения собчак,красота требует кэш,3 430 000,800 000,Подробнее,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...,960 000 ₽,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,rogov live,обзоры,460 000,400 000,Подробнее,https://www.youtube.com/channel/UCefO_zFbQc6-z...,https://drive.google.com/drive/folders/1-PTy90...,NaN,1 005 000 ₽,NaN,...,NaN,1 005 000 ₽,NaN,NaN,NaN,NaN,905 000 ₽,NaN,NaN,NaN
4,без фильтров,интервью,20 000,100 000,Подробнее,https://www.youtube.com/@bezfiltrovshow/,https://drive.google.com/drive/u/0/folders/1Gg...,NaN,NaN,NaN,...,NaN,115 000 ₽,NaN,NaN,NaN,NaN,115 000 ₽,NaN,NaN,NaN


> Посмотрим на список новых столбцов, предварительно отсортировав его

In [202]:
sorted_column_names = sorted(column_list)
sorted_column_names

['1 слот\nссылка после описания ролика_1',
 '1 слот_1',
 '2  слот_2',
 '2 Слот_2',
 '2 слот\nссылка после описания ролика_2',
 '2 слот_2',
 '3 слот_3',
 'Product Placement_2',
 'Product Placement_5',
 'Product placement_5',
 'Выпуск под клиента_3',
 'Выпуск под клиента_4',
 'Интеграция в стрим_1',
 'Мидролл_3',
 'Мидролл_4',
 'Мидролл_5',
 'Мини-интеграция до 1 мин_1',
 'Нативная интеграция_1',
 'Нативная интеграция_2',
 'Преролл \nдо 20 сек_4',
 'Преролл_3',
 'Преролл_4',
 'Спецвыпуск_5',
 'Спонсорство_5']

> Объединим столбцы с общими позициями. Сделаем это через словарь

In [ ]:
dict_new_columns = {'стоимость интеграция 1 слот':['1 слот\nссылка после описания ролика_1',
                                                    '1 слот_1'],
                    
                    'стоимость интеграция 2 слот':['2  слот_2',
                                                   '2 Слот_2',
                                                   '2 слот\nссылка после описания ролика_2',
                                                   '2 слот_2'],
                                        
                    'стоимость продакт плейсмент':['Product Placement_2',
                                                   'Product Placement_5',
                                                   'Product placement_5'],
                    
                    'стоимость выпуск под клиента':['Выпуск под клиента_3',
                                                    'Выпуск под клиента_4',
                                                    'Спецвыпуск_5'],
                    
                    'стоимость мидролл до 30 сек':['Мидролл_3',
                                                   'Мидролл_4',
                                                   'Мидролл_5'],
                    
                    'стоимость нативная интеграция':['Нативная интеграция_1',
                                                     'Нативная интеграция_2',],
                    
                    'стоимость преролл до 30 сек':['Преролл \nдо 20 сек_4',
                                                   'Преролл_3',
                                                   'Преролл_4']
                   }

In [ ]:
for new_column, old_column in dict_new_columns.items():
    for col in old_column:
        # Чтобы избежать преобразования значений NaN в строки "nan" при объединении строк,
        # мы будем использовать метод .fillna('') для замены значений NaN на пустые строки перед объединением
        blogosphere_youtube[col] = blogosphere_youtube[col].fillna('').astype(str)
    
    blogosphere_youtube[new_column] = blogosphere_youtube[old_column].agg(''.join, axis=1)  
    
    #Вернём обратно NaN
    blogosphere_youtube[new_column] = blogosphere_youtube[new_column].replace('', np.nan)

In [ ]:
blogosphere_youtube.drop(columns=['1 слот\nссылка после описания ролика_1',
                                  '1 слот_1',
                                  '2  слот_2',
                                  '2 Слот_2',
                                  '2 слот\nссылка после описания ролика_2',
                                  '2 слот_2',
                                  '3 слот_3',
                                  'Product Placement_2',
                                  'Product Placement_5',
                                  'Product placement_5',
                                  'Выпуск под клиента_3',
                                  'Выпуск под клиента_4',
                                  'Мидролл_3',
                                  'Мидролл_4',
                                  'Мидролл_5',
                                  'Нативная интеграция_1',
                                  'Нативная интеграция_2',
                                  'Преролл \nдо 20 сек_4',
                                  'Преролл_3',
                                  'Преролл_4',
                                  'Спецвыпуск_5'], axis=1, inplace=True)

In [ ]:
blogosphere_youtube.head()

> Подготовим названия столбцов к дальнейшей работе: объединим имя блогера и название шоу в один столбец "канал"

In [ ]:
[col for col in blogosphere_youtube.columns]

In [ ]:
[col for col in bufer.columns if 'YouTube' in col]

In [ ]:
blogosphere_youtube.rename(columns = {'1 слот для профильных клиентов_1':'стоимость интеграция для профильных клиентов 1 слот',
                                      '1 слот для непрофильных клиентов_1':'стоимость интеграция для непрофильных клиентов 1 слот',
                                      'Ролик до 1 мин_1':'стоимость отдельное видео до 1 мин',
                                      'Интеграция в стрим_1':'стоимость интеграция в стрим',
                                      '2 слот для профильных клиентов_2':'стоимость интеграция для профильных клиентов 2 слот',
                                      '2 слот для непрофильных клиентов_2':'стоимость интеграция для непрофильных клиентов 2 слот',
                                      '3 слот_3':'стоимость интеграция 3 слот',
                                      'Спонсорство_5':'стоимость спонсорство',
                                      'стоимость за интеграцию 1-ый слот':'стоимость интеграция 1 слот',
                                      '4 слот_1':'стоимость интеграция 4 слот',
                                      '5 слот_1':'стоимость интеграция 5 слот',
                                      'Мини-интеграция до 1 мин_1':'стоимость мини-интеграция до 1 мин',
                                     }, inplace=True)


> Присоединим столбец со стоимостью Shorts к единому DataFrame с помощью merge

In [ ]:
blogosphere_youtube = blogosphere_youtube.merge(blogosphere_youtube_shorts, on='название канала', how='left')
blogosphere_youtube

> Сделаем единый столбец с именем блогера, объединив столбцы 'название канала' и 'шоу'

In [ ]:
blogosphere_youtube['блогер'] = \
                blogosphere_youtube['название канала'] + ' - ' + blogosphere_youtube['шоу']

blogosphere_youtube.drop(columns=['статистика аудитории',
                                  'название канала',
                                  'шоу'], axis=1, inplace=True)

> Сделаем столбец "статистика" по примеру других DataFrame

In [ ]:
blogosphere_youtube.rename(columns = {'статистика ссылка':'статистика'}, inplace=True)

> У нас есть дубликат - обработаем его

In [ ]:
duplicates_blogosphere_youtube = blogosphere_youtube[blogosphere_youtube.duplicated('блогер', keep=False)]
duplicates_blogosphere_youtube

In [ ]:
filtered_blogosphere_youtube_1 = duplicates_blogosphere_youtube[duplicates_blogosphere_youtube.index % 2 != 0]
filtered_blogosphere_youtube_2 = duplicates_blogosphere_youtube[duplicates_blogosphere_youtube.index % 2 == 0]

In [ ]:
filtered_blogosphere_youtube_2 = filtered_blogosphere_youtube_2.dropna(axis=1)

In [ ]:
# Создаем паттерн для поиска строк, содержащих слова стоимость и блогер
pattern = r'стоимость|блогер'

# Только их и оставить в DataFrame filtered_blogosphere_youtube_2
filtered_blogosphere_youtube_2 = filtered_blogosphere_youtube_2.filter(regex=pattern, axis=1)

# Объединим новый DataFrame с основным
blogosphere_youtube = blogosphere_youtube.merge(filtered_blogosphere_youtube_2, on='блогер', how='outer')

# Удалим дубликат из обновного DataFrame
indexes_to_duplicates_blogosphere_youtube = filtered_blogosphere_youtube_2.index.tolist()
blogosphere_youtube.drop(indexes_to_duplicates_blogosphere_youtube, inplace=True)

> Проверяем результат

In [ ]:
duplicates_blogosphere_youtube = blogosphere_youtube[blogosphere_youtube.duplicated('блогер', keep=False)]
duplicates_blogosphere_youtube

In [ ]:
duplicates_blogosphere_youtube.columns

In [ ]:
def rename_endswith_xy(df):
    
    """
    Функция выполняет преобразования в DataFrame, объединяя столбцы, заканчивающиеся на "_x" и "_y".

    ПАРАМЕТРЫ:
        - df (pd.DataFrame): DataFrame, в котором выполняются преобразования.

    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
        - pd.DataFrame: Измененный DataFrame после выполнения преобразований.

    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
        blogosphere_youtube = rename_endswith_xy(blogosphere_youtube)
    """
    
    columns_x = df.columns[df.columns.str.endswith("_x")]

    # Получите список столбцов, которые заканчиваются на "_y"
    columns_y = df.columns[df.columns.str.endswith("_y")]

    # Объедините оба списка в один
    base_name = np.concatenate([columns_x, columns_y])

    base_names_with_x_y = []

    # Проходим по всем столбцам DataFrame
    for column in base_name:
        # Разбиваем название столбца по "_x" и "_y"
        parts = column.split('_')

        # Если у нас есть и "_x" и "_y" и их части до "_x" и "_y" совпадают
        base_names_with_x_y.append(parts[0])  

    # base_name = pd.Series(base_name)
    unique_base_names = pd.Series(base_names_with_x_y).unique()

    # unique_base_names[1]

    for i in range(len(unique_base_names)):
        df[f'{unique_base_names[i]}_x'] = df[f'{unique_base_names[i]}_x'].fillna('').astype(str)
        df[f'{unique_base_names[i]}_y'] = df[f'{unique_base_names[i]}_y'].fillna('').astype(str)

        df[f'{unique_base_names[i]}'] = \
            df[[f'{unique_base_names[i]}_x', f'{unique_base_names[i]}_y']].agg(''.join, axis=1)

        df.drop(columns=[f'{unique_base_names[i]}_x', f'{unique_base_names[i]}_y'], axis=1, inplace=True)

        df[f'{unique_base_names[i]}'] = df[f'{unique_base_names[i]}'].replace('', np.nan)
        
    return df

In [ ]:
# Проверяем
blogosphere_youtube.loc[blogosphere_youtube['блогер'] == 'без фильтров - интервью']

In [ ]:
blogosphere_youtube = rename_endswith_xy(blogosphere_youtube)

In [ ]:
# Проверяем
blogosphere_youtube.loc[blogosphere_youtube['блогер'] == 'без фильтров - интервью']

In [ ]:
# # Заполняем пропуски в столбцах списка list_merge пустыми строками
# list_merge = ['стоимость за преролл до 30 сек_x',
#               'стоимость за преролл до 30 сек_y']

# blogosphere_youtube[list_merge] = blogosphere_youtube[list_merge].fillna('')

# # Создаем новый столбец на основе объединенных значений
# blogosphere_youtube['стоимость за преролл до 30 сек'] = \
#     blogosphere_youtube[['стоимость за преролл до 30 сек_x',
#                          'стоимость за преролл до 30 сек_y']].agg(''.join, axis=1)

# # Удалим те столбцы, которые объединяли
# blogosphere_youtube.drop(columns=['стоимость за преролл до 30 сек_x',
#                                   'стоимость за преролл до 30 сек_y'], axis=1, inplace=True)

# # Переведём пустые строки в NaN
# blogosphere_youtube[['стоимость за преролл до 30 сек']] = \
#                                     blogosphere_youtube[['стоимость за преролл до 30 сек']].replace('', np.nan)


In [ ]:
blogosphere_youtube.head()

### Десятое объединение

In [ ]:
bufer = merge_bufer_df(bufer, blogosphere_youtube, 'YouTube')

In [ ]:
def show_duplicates(df: pd.DataFrame) -> str:
    
    """
    Удаляет дубликаты из DataFrame и выводит информацию о количестве дубликатов в столбце 'блогер'.
    
    ПАРАМЕТРЫ:
        df (pd.DataFrame): Исходный DataFrame.

    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
        Строка с информацией о дубликатах, отсортированных по столбцу 'блогер'.

    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
        duplicates_info = show_duplicates(my_dataframe)
        print(duplicates_info)
    """
    
    df = df.drop_duplicates()

    duplicates_df = df[df.duplicated('блогер', keep=False)]

    print("Количество дубликатов:", len(duplicates_df))

    return duplicates_df.sort_values(by='блогер')

In [ ]:
show_duplicates(bufer)

In [ ]:
стоп

In [ ]:
indices_to_remove = [244, 246]  # Индексы строк для удаления

bufer = bufer.drop(indices_to_remove)

### Подготовка таблицы blogosphere_tg_vk_zen

In [ ]:
blogosphere_tg_vk_zen.head()

In [ ]:
[col for col in blogosphere_tg_vk_zen.columns]

In [ ]:
bufer_columns_blogosphere_tg_vk_zen = ['Эксклюзивность',
                                       'Имя блогера',
                                       'Telegram блогер',
                                       'Telegram статистика',
                                       'Telegram количество подписчиков',
                                       'Telegram среднее кол-во просмотров поста',
                                       'Telegram стоимость фотопост',
                                       'TikTok блогер',
                                       'TikTok статистика',
                                       'TikTok количество подписчиков',
                                       'TikTok среднее кол-во просмотров',
                                       'TikTok стоимость видеопост',
                                       'VK группа блогер',
                                       'VK группа блогер статистика',
                                       'VK группа количество подписчиков',
                                       'VK группа среднее кол-во просмотров поста',
                                       'VK группа стоимость фотопост',
                                       'Дзен блогер',
                                       'Дзен статистика',
                                       'Дзен количество подписчиков',
                                       'Дзен среднее кол-во просмотров',
                                       'Дзен стоимость пост',
                                       'Telegram ссылка',
                                       'Telegram cтатистика',
                                       'TikTok ссылка',
                                       'TikTok статистика',
                                       'VK группа ссылка',
                                       'VK группа статистика',
                                       'Дзен ссылка',
                                       'Дзен статистика']

In [ ]:
blogosphere_tg_vk_zen.columns = bufer_columns_blogosphere_tg_vk_zen

In [ ]:
blogosphere_tg_vk_zen

In [ ]:
key_word = ['Эксклюзивность']# Список ключевых слов для поиска столбцов и их удаления
             
drop_word = '' # Список ключевых слов для их удаления из названия столбцов

upd_columns = ['Имя блогера',
               'Telegram блогер',
               'TikTok блогер',
               'VK группа блогер',
               'Дзен блогер'] # Список столбцов с именами блогеров для приведения к единому формату

In [ ]:
blogosphere_tg_vk_zen = pretraining_df(blogosphere_tg_vk_zen, key_word, drop_word, upd_columns)

> Назовём сразу столбцы как в едином DataFrame bufer, чтобы потом не делить данный DataFrame на части и не объединять их через функцию merge_bufer_df

In [ ]:
bufer_columns_blogosphere_tg_vk_zen = ['блогер',
                                       'Telegram блогер',
                                       'Telegram статистика',
                                       'Telegram количество подписчиков',
                                       'Telegram среднее кол-во просмотров поста',
                                       'Telegram стоимость фотопост',
                                       'TikTok блогер',
                                       'TikTok статистика',
                                       'TikTok количество подписчиков',
                                       'TikTok среднее кол-во просмотров',
                                       'TikTok стоимость видеопост',
                                       'VK группа блогер',
                                       'VK группа статистика',
                                       'VK группа количество подписчиков',
                                       'VK группа среднее кол-во просмотров поста',
                                       'VK группа стоимость фотопост',
                                       'Дзен блогер',
                                       'Дзен статистика',
                                       'Дзен количество подписчиков',
                                       'Дзен среднее кол-во просмотров',
                                       'Дзен стоимость пост',
                                       'Telegram ссылка',
                                       'Telegram cтатистика',
                                       'TikTok ссылка',
                                       'TikTok статистика',
                                       'VK группа ссылка',
                                       'VK группа статистика',
                                       'Дзен ссылка',
                                       'Дзен статистика']

In [ ]:
blogosphere_tg_vk_zen.columns = bufer_columns_blogosphere_tg_vk_zen

In [ ]:
blogosphere_tg_vk_zen.head()

> Удалим лишние пустые строки, в которых у нас нет информации из-за особенности DataFrame, в котором под таблицей указали запасные данные: удалить строки, в которых значения равны NaN как в столбце 'блогер', так и в столбце 'telegram блогер' как в самых заполненных столбцах

In [ ]:
blogosphere_tg_vk_zen = blogosphere_tg_vk_zen.dropna(subset=['блогер', 'Telegram блогер'], how='all')

# Отдельно удалим первую строку с NaN в столбец 'блогер', которая представляем собой название столбцов в виде строки
blogosphere_tg_vk_zen = blogosphere_tg_vk_zen.dropna(subset=['блогер'])

> Заполним значение столбца "имя блогера" с помощью метода .fillna(method='ffill')

In [ ]:
blogosphere_tg_vk_zen[['блогер']] = blogosphere_tg_vk_zen[['блогер']].fillna(method='ffill')

> Удалим лишние столбцы, где нет ссылки на статистику

In [ ]:
blogosphere_tg_vk_zen.drop(columns=['Telegram статистика',
                                    'TikTok статистика',
                                    'VK группа статистика',
                                    'Дзен статистика',
                                    'Telegram cтатистика'], axis=1, inplace=True)

In [ ]:
blogosphere_tg_vk_zen.head()

### Одинадцатое объединение

> У нас есть общие столбцы - на основе их и присоединим blogosphere_tg_vk_zen к bufer

In [ ]:
common_columns_blogosphere_tg_vk_zen = bufer.columns.intersection(blogosphere_tg_vk_zen.columns)
print(common_columns_blogosphere_tg_vk_zen)

> Так как мы сразу указали название соц. сети в названии столбцов, чтобы не делить на несколько DataFrame, то объединим не с помощью функции, а вручную с помощью merge

In [ ]:
bufer = bufer.merge(blogosphere_tg_vk_zen, \
                    left_on=common_columns_blogosphere_tg_vk_zen.tolist(), \
                    right_on=common_columns_blogosphere_tg_vk_zen.tolist(), \
                    how='outer')

In [ ]:
bufer.head()

In [ ]:
duplicates = bufer[bufer.duplicated('блогер', keep=False)]
duplicates

In [ ]:
#Заполним контактов менеджера
bufer['контакты менеджера'].fillna('sales@blogo-sphere.com', inplace=True)

#Заполним значение налога
bufer['налог'].fillna('НДС (20%)', inplace=True)

## Подготовка данных по блогерам от Insight People

**Имеющиеся DataFrame:**  

* insight_instagram
* insight_threads
* insight_telegram
* insight_vk_ls
* insight_vk_group
* insight_youtube
* insight_zen
* insight_likee
* insight_yappy
* insight_rutube
<!-- * insight_ok -->

> Предварительно сделаем предобработку DataFrame от Insight People

In [ ]:
insight_df = [insight_instagram,
              insight_threads,
              insight_telegram,
              insight_vk_ls,
              insight_vk_group,
              insight_youtube,
              insight_zen,
              insight_likee,
              insight_yappy,
              insight_rutube,]
#               insight_ok]

In [ ]:
key_word = ['CPU', 'ER', 'CPV']  # Список ключевых слов для поиска столбцов и их удаления
drop_word = ''  # Список ключевых слов для их удаления из названия столбцов

In [ ]:
for i in range(len(insight_df)):
    df = insight_df[i]  # Получаем текущий DataFrame по индексу
    
    upd_columns = []  # Пустой список столбцов с именами блогеров для приведения к единому формату

    if 'Фамилия' in df.columns:
        upd_columns.append('Фамилия')
    if 'Имя' in df.columns:
        upd_columns.append('Имя')
    if 'Имя блогера' in df.columns:
        upd_columns.append('Имя блогера')
    if 'Название канала' in df.columns:
        upd_columns.append('Название канала')
    if 'ФИО' in df.columns:
        upd_columns.append('ФИО')
    if 'Фамилия имя' in df.columns:
        upd_columns.append('Фамилия имя')

    pretraining_df(df, key_word, drop_word, upd_columns)

### Подготовка таблицы insight_instagram

In [ ]:
insight_instagram.head()

> Сделаем общий столбец с именем и фамилией блогера, и так как данное действие мы будем делать практически у всех DataFrame от Insight, то сделаем это функцией, а так как мы будем делать это для большинства DataFrame от Insight People, то сделаем это функцией

In [ ]:
[col for col in insight_instagram.columns]

In [ ]:
[col for col in bufer.columns if 'Instagram' in col]

In [ ]:
insight_instagram.rename(columns = {'фамилия имя':'блогер',
                                    'стоимость за фотопост':'стоимость фотопост',
                                    'охват пост':'среднее кол-во просмотров фотопоста',
                                    'охват сторис':'среднее кол-во просмотров 1 сторис',
                                    'охват серии сторис':'среднее кол-во просмотров серии сторис',
                                    'стоимость за сторис за 15 секунд':'стоимость 1 сторис до 15 сек',
                                    'стоимость за серию сторис до 60 секунд':'стоимость серия сторис до 60 сек',
                                    'стоимость за видеопост / reels':'стоимость reels / видеопост'
                                    }, inplace=True)

#### Проверка на дубликаты

> **ДОПИСАТЬ**

In [ ]:
def show_duplicates(df):
    
    """
    """
    
    df = df.drop_duplicates()

    duplicates_df = df[df.duplicated('блогер', keep=False)]

    print("Количество дубликатов:", len(duplicates_df))

    return duplicates_df.sort_values(by='блогер')

In [ ]:
show_duplicates(insight_instagram)

> В исходном DataFrame у нас есть дубликат, который отличается только значением в столбце "Instagram стоимость за серию сторис до 60 сек". Удалим один из них по индексу

In [ ]:
duplicates_insight_instagram = insight_instagram[insight_instagram.duplicated('блогер', keep=False)]
duplicates_insight_instagram

In [ ]:
indexes_to_duplicates_insight_instagram = duplicates_insight_instagram[duplicates_insight_instagram.index % 2 == 0]
indexes_to_duplicates_insight_instagram = indexes_to_duplicates_insight_instagram.index.tolist()
indexes_to_duplicates_insight_instagram

In [ ]:
insight_instagram.drop(indexes_to_duplicates_insight_instagram, inplace=True)

In [ ]:
show_duplicates(insight_instagram)

### Двенадцатое объединение

In [ ]:
bufer = merge_bufer_df(bufer, insight_instagram, 'Instagram')

In [ ]:
bufer.head()

### Подготовка таблицы insight_threads

In [ ]:
insight_threads.head()

In [ ]:
def name_familia(df: pd.DataFrame) -> pd.DataFrame:
    
    """
    Функция объединяет два столбца 'имя' и 'фамилия' нашего DataFrame df в единый столбец 'блогер'.
    Далее мы удаляем столбцы 'имя' и 'фамилия' из DataFrame.
    
    
    ПАРАМЕТРЫ:
    
    df (DataFrame) - DataFrame, в котором мы обновляем столбцы
    
 
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    
    df (DataFrame) - обновлённый DataFrame
    
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    
    insight_instagram = name_familia(insight_instagram)
    
    """
    
    df['блогер'] = df['фамилия'] + ' ' + df['имя']

    df.drop(columns=['имя', 'фамилия'], axis= 1, inplace=True)
    
    return df

In [ ]:
insight_threads = name_familia(insight_threads)

In [ ]:
insight_threads.head()

In [ ]:
insight_threads.rename(columns = {'стоимость за фотопубликацию':'стоимость фотопост',
                                  'стоимость за видеопубликацию':'стоимость видеопост'
                                 }, inplace=True)

#### Проверка на дубликаты

In [ ]:
show_duplicates(insight_threads)

> Опять есть дубликат - удалим тот, где меньше всего данных в столбце "тематика", так как в единственном столбце "количество подписчиков", где есть изменения, цифры практически не отличаются

In [ ]:
duplicates_insight_threads = insight_threads[insight_threads.duplicated('блогер', keep=False)]
duplicates_insight_threads

In [ ]:
indexes_to_duplicates_insight_threads = duplicates_insight_threads[duplicates_insight_threads.index % 2 != 0]
indexes_to_duplicates_insight_threads = indexes_to_duplicates_insight_threads.index.tolist()
indexes_to_duplicates_insight_threads

In [ ]:
insight_threads.drop(indexes_to_duplicates_insight_threads, inplace=True)

In [ ]:
show_duplicates(insight_threads)

### Тринадцатое объединение

> У нас нет общих столбцов, поэтому объединим по имени блогера

In [ ]:
bufer = merge_bufer_df(bufer, insight_threads, 'Threads')

### Подготовка таблицы insight_telegram

In [ ]:
insight_telegram.head()

> Переименуем столбцы под общий формат

In [ ]:
[col for col in insight_telegram.columns]

In [ ]:
[col for col in bufer.columns if 'Telegram' in col]

In [ ]:
insight_telegram.rename(columns = {'фамилия имя':'блогер',
                                   'средний охват публикации':'среднее кол-во просмотров поста',
                                   'кружок':'стоимость кружочек до 60 сек',
                                   'стоимость за фотопост':'стоимость фотопост',
                                   'стоимость за видеопост':'стоимость видеопост'
                                  }, inplace = True)

#### Проверка на дубликаты

In [ ]:
show_duplicates(insight_telegram)

> Дубликатов нет - может объединять

### Четырнадцатое объединение

> Исправим одну неточность в имени блогера

In [ ]:
insight_telegram.loc\
    [insight_telegram['блогер'] == 'сивцев станислав', 'блогер'] = 'станислав сивцев'

In [ ]:
bufer = merge_bufer_df(bufer, insight_telegram, 'Telegram')

In [ ]:
show_duplicates(bufer).head(6)

> У нас есть определённая проблема с DataFrame insight_telegram - почти половина каналов не заполнились с помощью merge, а сделались новыми строками - скорее всего это произошло из-за особого форматирования данных столбца "блогер", так как проверка ниже показывает, что данные равны по значению, и нам надо объединить данные строки в единую строку.

In [ ]:
# Проверка
test_dub = bufer.loc[bufer['блогер'] == 'ахметзянов айрат']
test_dub.reset_index()

In [ ]:
string1 = test_dub.iloc[0]['блогер']
string2 = test_dub.iloc[1]['блогер']

print(repr(string1))
print(repr(string2))

In [ ]:
# Сравним их
if repr(string1) == repr(string2):
    print("Строки идентичны")
else:
    print("Строки различаются")

> Отдельно стоит отметить, что даже если совпадают имена блогера, то бывают случаи, когда другие значения, в том числе по ценам, но чаще всего по столбцу "тематика", не совпадают, из-за чего мы не можем удалить дубликаты - учтём это в функции ниже

In [ ]:
duplicates_bufer = bufer[bufer.duplicated('блогер', keep=False)]
duplicates_bufer_sample = duplicates_bufer.loc[duplicates_bufer['блогер'] == 'бурмистрова татьяна']
duplicates_bufer_sample[['блогер', 'тематика']]

> Напишем функцию, которая будет заполненять пропущенные значения в DataFrame, объединенять уникальные значений в столбцах, а также удалять дубликаты строк и выполняет обработку столбцов на основе заданных правил, включая разделение значений и обновление 'тематики' для каждого блогера.

In [ ]:
def duplicates_stroka(df: pd.DataFrame,
                      col_to_check: str
                      columns_to_upd: str) -> pd.DataFrame:
    
    """
    Эта функция выполняет заполнение пропущенных значений в датафрейме, 
    объединение уникальных значений в столбцах, а также удаляет дубликаты строк и
    выполняет обработку столбцов на основе заданных правил, включая разделение значений и обновление 'тематики' для каждого блогера.
    
    
    ПАРАМЕТРЫ:
    df (DataFrame) - DataFrame, в котором мы будет делать преобразования
    col_to_check (str) - столбец, по которому мы будет смотреть дубликаты (чаще всего имя блогера)
    
    
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    df (DataFrame) - обновлённый DataFrame  
    
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    
    col_to_check = 'блогер'
    bufer = duplicates_bufer(bufer, col_to_check)
    """

    for col in df.columns:
        if col != col_to_check: # Пропускаем столбец, по которому выполняется проверка
            
            df['temp_fill'] = np.nan

# Создается временный столбец с именем 'temp_fill', который содержит последнее (по индексу)
# непустое значение из столбца col для каждой группы, сгруппированной по значениям в столбце col_to_check.
# Это выполняется с помощью bufer.groupby(col_to_check)[col].transform('last').
        
            df['temp_fill'] = df.groupby(col_to_check)[col].transform('last')  # Создание временного столбца

# Пропущенные значения в столбце col заполняются соответствующими значениями
# из временного столбца 'temp_fill' с использованием bufer[col].fillna(bufer['temp_fill']).
        
            df[col] = df[col].fillna(df['temp_fill'])  # Заполнение пропущенных значений

# Сбрасываются индексы в DataFrame bufer с помощью bufer.reset_index(drop=True). 
# Это делается для того, чтобы индексы начались с нуля после заполнения пропущенных значений.
            df = df.reset_index(drop=True)  # Сброс индексов

# Удаляется временный столбец 'temp_fill' с помощью bufer = bufer.drop(columns=['temp_fill']).    
            df = df.drop(columns=['temp_fill'])  # Удаление временного столбца

# # Удаляются дубликаты строк в DataFrame bufer с помощью bufer = bufer.drop_duplicates().    
#             df = df.drop_duplicates()
   
#     df = df.drop_duplicates()
    
    for column in df.columns:
        
        if 'тематика' in column:
            df[column] = df[column].astype(str)
            df[column] = df[column].fillna('')
            
            df[column] = df[column].str.strip()\
                                    .str.lower()\
                                    .str.replace('/', ', ')\
                                    .str.replace(r'\n', ', ')\
                                    .str.replace('*', '')\
                                    .str.replace(', ', ',')\
                                    .str.replace(',', ', ')
            df[column] = df[column].str.strip()
#             df[column] = df[column].str.split(', ')    
    
    for blogger in df[col_to_check].unique():
        blogger_rows = df[df[col_to_check] == blogger]
        unique_thematics = set()
        for column in blogger_rows.columns[blogger_rows.columns.str.contains(columns_to_upd)]:
            for value in blogger_rows[column].str.split(', ').dropna():
                unique_thematics.update(value)
        unique_thematics = ', '.join(sorted(unique_thematics))
        df.loc[df[col_to_check] == blogger, blogger_rows.columns[blogger_rows.columns.str.contains(columns_to_upd)]] = unique_thematics
    
    
    for column in df.columns:
        if columns_to_upd in column:    
            df[column] = df[column].str.lstrip('none, ')
#             df[column] = df[column].str.lstrip(', ')
            

    
    df = df.drop_duplicates()


    return df

In [ ]:
# До 
bufer.shape

> Сделаем преобразование

In [ ]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [ ]:
# После

bufer.shape

> Мы убрали почти 40 дубликатов, кроме тех, где есть отличия в ценах

> Ниже видно, что заполнились все столбцы и объединились правильно по блогеру, а также заполнились все столбцы, где есть слово "тематика" суммарным списком тематик

In [ ]:
show_duplicates(bufer).head(6)

In [ ]:
bufer.loc[bufer['блогер'] == 'бурмистрова татьяна']

### Подготовка таблицы insight_vk_ls

In [ ]:
insight_vk_ls.head()

In [ ]:
insight_vk_ls.rename(columns = {'фамилия имя':'блогер',
                                'средние просмотры поста':'среднее кол-во просмотров поста',
                                'средие просмотры клипа вк':'среднее кол-во просмотров вк клипа',
                                'стоимость за vk clip':'стоимость вк клип',
                                'стоимость за фотопост':'стоимость фотопост',
                                'стоимость за видеопост':'стоимость видеопост',
                                'передача прав на paid social , ( 1 месяц)':'стоимость передачи прав на 1 месяц'
                               }, inplace=True)

#### Проверка на дубликаты

In [ ]:
show_duplicates(insight_vk_ls)

### Пятнадцатое объединение

In [ ]:
bufer = merge_bufer_df(bufer, insight_vk_ls, 'VK личная страница')

In [ ]:
# До

bufer.shape

In [ ]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [ ]:
# После

bufer.shape

In [ ]:
show_duplicates(bufer).head(6)

In [ ]:
bufer[bufer['блогер'] == 'юлия ланге']

### Подготовка таблицы insight_vk_group

In [ ]:
insight_vk_group.head()

In [ ]:
insight_vk_group.rename(columns = {'фамилия имя':'блогер',
                                   'средние просмотры публикации':'среднее кол-во просмотров поста',
                                   'средние просмотры клипа вк':'среднее кол-во просмотров вк клипа',
                                   'стоимость за vk clip':'стоимость вк клип',
                                   'стоимость за фотопост':'стоимость фотопост',
                                   'стоимость за видеопост':'стоимость видеопост',
                                   'передача прав на paid social , ( 1 месяц)':'стоимость передачи прав на 1 месяц'
                                  }, inplace=True)

#### Проверка на дубликаты

In [ ]:
show_duplicates(insight_vk_group)

### Шестнадцатое объединение

In [ ]:
bufer = merge_bufer_df(bufer, insight_vk_group, 'VK группа')

In [ ]:
show_duplicates(bufer).head(6)

In [ ]:
# До

bufer.shape

In [ ]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [ ]:
# После

bufer.shape

In [ ]:
duplicates = bufer[bufer.duplicated('блогер', keep=False)]
duplicates.sort_values(by='блогер')

### Подготовка таблицы insight_youtube

> Название канала

In [ ]:
insight_youtube.head()

In [ ]:
[col for col in bufer.columns if 'YouTube' in col]

In [ ]:
insight_youtube.rename(columns = {'фамилия имя':'блогер',
                                  'просмотры видео среднее кол-во':'среднее кол-во просмотров видео',
                                  'просмотры shorts среднее кол-во':'среднее кол-во просмотров shorts',
                                  'стоимость преролла':'стоимость преролл до 30 сек',
                                  'стоимость интеграции (до 1 минуты)':'стоимость интеграция 1 слот',
                                  'стоимость shorts':'стоимость за shorts до 1 мин',
                                  'стоимость эксклюзивного ролика':'стоимость выпуск под клиента'
                                 }, inplace=True)

#### Проверка на дубликаты

In [ ]:
show_duplicates(insight_youtube)

> Два канала сделаны как разные строки по одному блогеру - исправим это: переименуем вторую строчку на название канала - 'биполярка сарычева'

In [ ]:
insight_youtube.loc\
    [insight_youtube['название канала'] == 'биполярка', 'блогер']  = 'биполярка сарычева'

In [ ]:
# duplicates_insight_youtube = insight_youtube[insight_youtube.duplicated('блогер', keep=False)]
# duplicates_insight_youtube

# indexes_insight_youtube_sarychev = duplicates_insight_youtube_sarychev.index.tolist()

# indexes_insight_youtube_sarychev

# insight_youtube.drop(indexes_insight_youtube_sarychev, inplace=True)

# columns_with_sarychev = ['количество подписчиков',
#                          'среднее кол-во просмотров',
#                          'среднее кол-во просмотров shorts',
#                          'стоимость за преролл до 30 сек',
#                          'стоимость за интеграцию до 1 мин',
#                          'стоимость за shorts до 1 мин',
#                          'стоимость за выпуск под клиента',
#                          'блогер']

# # Удаление всех столбцов, кроме тех, что в списке columns_with_sarychev
# duplicates_insight_youtube_sarychev = duplicates_insight_youtube_sarychev[columns_with_sarychev]
# duplicates_insight_youtube_sarychev

In [ ]:
show_duplicates(insight_youtube)

### Семнадцатое объединение

In [ ]:
bufer = merge_bufer_df(bufer, insight_youtube, 'YouTube')

In [ ]:
show_duplicates(bufer).head(6)

In [ ]:
# До

bufer.shape

In [ ]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [ ]:
# После

bufer.shape

In [ ]:
show_duplicates(bufer).head(6)

### Подготовка таблицы insight_zen

In [ ]:
insight_zen.head()

#### Обрежем DataFrame по высоте

> Так как мы загружаем данные с CSV, то мы видим не только данные, которые видят все, но и скрытые данные - обрежем данные, чтобы они соответствовали действительности

In [ ]:
def trim_df(df: pd.DataFrame, columns: str, number: int) -> pd.DataFrame:
    
    """
    Данная функция trim_df предназначена для обработки и фильтрации данных в DataFrame,
    в которой мы обрезаем DataFrame по определённому размеру числа number в столбце columns

    
    ПАРАМЕТРЫ:
    df (pd.DataFrame) - DataFrame, который нужно обработать и фильтровать..
    columns (str) - строка, представляющая название столбца в DataFrame, который должен быть числовой или можно его перевести
    который будет обрабатываться, то есть порог, по которому мы обрезаем DataFrame.
    number (int) - целое число, которое используется для фильтрации данных в указанном столбце.
 
 
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    df (pd.DataFrame) - обновлённый DataFrame.
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    insight_zen = trim_df(insight_zen, 'количество подписчиков', 999)
        
    """
    
    # Уберем неразрывные пробелы из значений столбца 'количество подписчиков'
    df[columns] = df[columns].str.replace('\xa0', '')
    df[columns] = df[columns].str.replace(' ', '')

    # Теперь преобразуем значения в числа с плавающей точкой
    df[columns] = df[columns].astype(float)

    #Обрежем DataFrame по-определённому размеру столбца columns.
    
    df = df[df[columns] >= number]
    
    #Переведём обратно в str, чтобы у нас не было проблем с объединением.

    df[columns] = df[columns].astype(str)
    
    return df

In [ ]:
insight_zen = trim_df(insight_zen, 'количество подписчиков', 999)

In [ ]:
[col for col in bufer.columns if 'Дзен' in col]

In [ ]:
[col for col in insight_zen.columns]

In [ ]:
insight_zen.rename(columns = {'фамилия':'блогер',
                              'категория':'тематика',
                              'средний охват публикации':'среднее кол-во просмотров',
                              'стоимость размещения за фотопост':'стоимость пост',
                              'стоимость размещения за видеопост':'стоимость видеопост'
                             }, inplace=True)

#### Проверка на дубликаты

> У нас есть дубликаты, но помимо столбца "тематика", также отличаются  столбцы с количествами, а также с городом

In [ ]:
show_duplicates(insight_zen)

> Обновим столбец "Тематика"

In [ ]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
insight_zen = duplicates_stroka(insight_zen, col_to_check, columns_to_upd)

In [ ]:
show_duplicates(insight_zen)

> Обновим столбец "Город"

In [ ]:
col_to_check = 'блогер'
columns_to_upd = 'город'
insight_zen = duplicates_stroka(insight_zen, col_to_check, columns_to_upd)

In [ ]:
show_duplicates(insight_zen)

> Удалим дубликаты, оставив только первое вхождение, так как чаще всего именно в них самые большие и видимо правильные значения

In [ ]:
insight_zen = insight_zen.drop_duplicates(subset='блогер', keep='first')

In [ ]:
show_duplicates(insight_zen)

### Восемнадцатое объединение

In [ ]:
bufer = merge_bufer_df(bufer, insight_zen, 'Дзен')

In [ ]:
show_duplicates(bufer).head(6)

In [ ]:
# До

bufer.shape

In [ ]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [ ]:
# После

bufer.shape

In [ ]:
show_duplicates(bufer).head(6)

### Подготовка таблицы insight_likee

In [ ]:
insight_likee.head()

In [ ]:
insight_likee = name_familia(insight_likee)

In [ ]:
[col for col in bufer.columns if 'TikTok' in col]

In [ ]:
[col for col in insight_likee.columns]

In [ ]:
insight_likee.rename(columns = {'средний просмотры публикации':'среднее кол-во просмотров',
                                'стоимость за видеопост (60 сек.)':'стоимость видеопост'
                               }, inplace=True)

#### Обрежем DataFrame по высоте

In [ ]:
insight_likee = trim_df(insight_likee, 'количество подписчиков', 27000)

In [ ]:
insight_likee.tail()

#### Проверка на дубликаты

In [ ]:
show_duplicates(insight_likee)

### Девятнадцатое объединение

In [ ]:
bufer = merge_bufer_df(bufer, insight_likee, 'Likee')

In [ ]:
# Посмотрим на размер DataFrame

bufer.shape

### Подготовка таблицы insight_yappy

In [ ]:
insight_yappy.head()

In [ ]:
insight_yappy = name_familia(insight_yappy)

In [ ]:
[col for col in bufer.columns if 'TikTok' in col]

In [ ]:
[col for col in insight_yappy.columns]

In [ ]:
insight_yappy.rename(columns = {'ссылка на профиль в yappy':'ссылка',
                                'стоимость за видеопост (60 сек.)':'стоимость видеопост',
                               }, inplace=True)

#### Обрежем DataFrame по высоте

In [ ]:
insight_yappy = trim_df(insight_yappy, 'количество подписчиков', 3000)

In [ ]:
insight_yappy.tail()

#### Проверка на дубликаты

In [ ]:
show_duplicates(insight_yappy)

### Двадцатое объединение

In [ ]:
bufer = merge_bufer_df(bufer, insight_yappy, 'Yappy')

### Подготовка таблицы insight_rutube

> У нас есть проблема с именем блогера: оно сделано как **ФИО**, а нам нужно **имя и фамилия**, чтобы правильно объединить по строкам - сделаем преобразования, чтобы подвести к правильному формату

In [ ]:
insight_rutube.head()

In [ ]:
def manipulate_text(text: str) -> str:
    words = text.split()  # Разделение строки на слова
    if len(words) >= 3:
        words.pop(2)  # Удаление третьего слова
    return ' '.join(words)  # Объединение слов обратно в строку

In [ ]:
insight_rutube['фио'] = insight_rutube['фио'].apply(manipulate_text).str.strip()

In [ ]:
[col for col in insight_rutube.columns]

In [ ]:
[col for col in insight_youtube.columns]

In [ ]:
insight_rutube.rename(columns = {'фио':'блогер',
                                 'средние просмотры за месяц':'среднее кол-во просмотров видео',
                                 'стоимость преролла':'стоимость преролл до 30 сек',
                                 'стоимость интеграции (до 1 минуты)':'стоимость интеграция 1 слот',
                                 'количество подписчиков':'тематика'
                                }, inplace=True)

#### Обрежем DataFrame по высоте

In [ ]:
insight_rutube.head()

In [ ]:
insight_rutube = trim_df(insight_rutube, 'среднее кол-во просмотров видео', 3299)
insight_rutube.tail()

In [ ]:
insight_rutube = insight_rutube.loc[0: 70]

In [ ]:
insight_rutube.tail(15)

#### Проверка на дубликаты

In [ ]:
show_duplicates(insight_rutube)

### Двадцать первое объединение

In [ ]:
bufer = merge_bufer_df(bufer, insight_rutube, 'RuTube')

### Подготовка таблицы insight_ok

In [ ]:
# insight_ok.head()

In [ ]:
# insight_ok = name_familia(insight_ok)

In [ ]:
# [col for col in bufer.columns if 'OK' in col]

In [ ]:
# [col for col in insight_ok.columns]

#### Обрежем DataFrame по высоте

In [ ]:
# insight_ok = trim_df(insight_ok, 'количество подписчиков', 53)

#### Проверка на дубликаты

In [ ]:
# show_duplicates(insight_ok)

### Двадцать второе объединение

In [ ]:
# bufer = merge_bufer_df(bufer, insight_ok, 'Одноклассники')

> Запустим функцию duplicates_stroka, чтобы обновить столбец bufer

In [ ]:
# col_to_check = 'блогер'
# columns_to_upd = 'тематика'
# bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

> Посмотрим на размер DataFrame bufer на данный момент

In [ ]:
# bufer.shape

> Заполним столбец контакты и налог

In [ ]:
#Заполним контактов менеджера
bufer['контакты менеджера'].fillna('info@insightpeople.ru', inplace=True)

#Заполним значение налога
bufer['налог'].fillna('НДС (20%)', inplace=True)

## Подготовка данных по блогерам от NextUp

In [ ]:
nextup_df = [nextup_instagram,
             nextup_telegram,
             nextup_youtube,
             nextup_twitch,
             nextup_vk]

In [ ]:
key_word = ['CPV', ]  # Список ключевых слов для поиска столбцов и их удаления
drop_word = ''  # Список ключевых слов для их удаления из названия столбцов
upd_columns = ['Имя блогера']

In [ ]:
for df in nextup_df:
    pretraining_df(df, key_word, drop_word, upd_columns)

### Подготовка таблицы nextup_instagram

In [ ]:
nextup_instagram.head()

In [ ]:
def nextup_col_upd(df: pd.DataFrame, nextup_columns_upd: list) -> pd.DataFrame:
    
    """
    
    ПАРАМЕТРЫ:
    df (pd.DataFrame) - .
    nextup_columns_upd (list) - .
 
 
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    df (pd.DataFrame) - DataFrame с обновлёнными столбцами nextup_columns_upd.
    
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    
    """
    

#     for col in df.columns:
#         df[nextup_columns_upd] = df[nextup_columns_upd].apply(lambda x: str(x).lower())\
#                                                         .str.replace('/\n', ', ')\
#                                                         .str.replace('\n', ', ')\
#                                                         .str.replace('*', '')\
#                                                         .str.strip()
        
    for col in nextup_columns_upd:
        df[col] = df[col].apply(lambda x: str(x).lower())\
                        .str.replace('\n', ', ')\
                        .str.replace('*', '')\
                        .str.strip()
        
    return df

In [ ]:
nextup_instagram_columns_upd = ['тематика', 'ца по городам', 'ца по возрасту', 'ца по полу']
nextup_instagram = nextup_col_upd(nextup_instagram, nextup_instagram_columns_upd)

In [ ]:
nextup_instagram.head()

> Удалим первые два столбца, которые не несут в себе полезной информации, а так как нам это надо будет делать ещё у нескольких DataFrame, то сделаем функцию

In [ ]:
def nextup_columns_to_drop(df: pd.DataFrame,
                           columns_to_drop: list,
                           statistika: bool) -> pd.DataFrame:
    
    """
    Функция удаляет определённые столбцы по индексу, которые мы передаём в листе columns_to_drop.
    Также у нас есть флаг statistika, и если стоит True, то также удаляется столбец "статистика"
    
    ПАРАМЕТРЫ:
    df (pd.DataFrame) - DataFrame, который мы редактируем.
    columns_to_drop: (list) - список с индексами столбцов для удаления.
    statistika (bool) - флаг для удаления столбца "Статистика".
 
 
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    df (pd.DataFrame) - обновлённый DataFrame.
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    columns_to_drop = [0, 1]
    nextup_instagram = nextup_columns_to_drop(nextup_instagram, columns_to_drop, True)
    
    
    """
    df = df.drop(df.columns[columns_to_drop], axis=1)
    
    if statistika:
        df.drop(columns=['статистика'], axis=1, inplace=True)
    
    return df  # Вы должны вернуть измененный DataFrame

In [ ]:
columns_to_drop = [0, 1]
nextup_instagram = nextup_columns_to_drop(nextup_instagram, columns_to_drop, True)

In [ ]:
nextup_instagram.head()

In [ ]:
[col for col in nextup_instagram.columns]

In [ ]:
[col for col in bufer.columns if 'Instagram' in col]

In [ ]:
nextup_instagram.rename(columns = {'имя блогера':'блогер',
                                   'кол-во подписчиков':'количество подписчиков',
                                   'охват фотопостов за 30 дней':'среднее кол-во просмотров фотопоста',
                                   'охват reels за 30 дней':'среднее кол-во просмотров reels / видеопоста',
                                   'охват stories за 14 дней':'среднее кол-во просмотров 1 сторис',
                                   'стоимость reels':'стоимость reels / видеопост',
                                   'стоимость серия stories (3 шт. по 15 сек.)':'стоимость серия из 3х сторис до 45 сек',
                                   'статистика ссылка':'статистика'
                                  }, inplace=True)

#### Проверка на дубликаты

In [ ]:
show_duplicates(nextup_instagram)

### Двадцать третье объединение

In [ ]:
bufer = merge_bufer_df(bufer, nextup_instagram, 'Instagram')

In [ ]:
show_duplicates(bufer).head(6)

> Обновим столбец "статистика"

In [ ]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [ ]:
show_duplicates(bufer).head(6)

> Лиза Барашик на эксклюзиве у агенства DIDENOK, поэтому мы должны оставить именно строчку с ними, то есть первую

In [ ]:
duplicates_bufer = bufer[bufer.duplicated('блогер', keep=False)]
duplicates_bufer

In [ ]:
max_index = duplicates_bufer.index.max()
bufer.drop(max_index, inplace=True)

In [ ]:
bufer[bufer['блогер'] == 'лиза барашик']

### Подготовка таблицы nextup_telegram

In [ ]:
nextup_telegram.head()

In [ ]:
nextup_telegram = nextup_telegram.dropna(subset=['имя блогера'])

In [ ]:
nextup_telegram.tail()

In [ ]:
columns_to_drop = [0, 1]
nextup_telegram = nextup_columns_to_drop(nextup_telegram, columns_to_drop, True)

In [ ]:
[col for col in nextup_telegram.columns]

In [ ]:
[col for col in bufer.columns if 'Telegram' in col]

In [ ]:
nextup_telegram.rename(columns = {'имя блогера':'блогер',
                                  'кол-во подписчиков':'количество подписчиков',
                                  'охват поста за 30 дней':'среднее кол-во просмотров поста',
                                  'стоимость текстовый пост (24 часа)':'стоимость текстовое сообщение без фото',
                                  'стоимость фотопост (24 часа)':'стоимость фотопост',
                                  'стоимость видеокружок (24 часа)':'стоимость кружочек до 60 сек',
                                  'статистика ссылка':'статистика'
                                 }, inplace=True)

#### Проверка на дубликаты

In [ ]:
show_duplicates(nextup_telegram)

### Двадцать четвёртое объединение

In [ ]:
bufer = merge_bufer_df(bufer, nextup_telegram, 'Telegram')

In [ ]:
show_duplicates(bufer).head(6)

In [ ]:
# До

bufer.shape

In [ ]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [ ]:
# После

bufer.shape

In [ ]:
show_duplicates(bufer)

In [ ]:
duplicates = bufer[bufer.duplicated('блогер', keep=False)]
duplicates

In [ ]:
telegram_columns = duplicates.filter(like='Telegram')
telegram_columns

> Оставим только первое значение данного блогера, так как блогер "лиза барашик" на эксклюзиве у DidenokTeam

In [ ]:
duplicates_bufer = bufer[bufer.duplicated('блогер', keep=False)]
max_index = duplicates.index.max()
bufer.drop(max_index, inplace=True)

In [ ]:
bufer[bufer['блогер'] == 'лиза барашик']

### Подготовка таблицы nextup_youtube

In [ ]:
nextup_youtube.head()

In [ ]:
columns_to_drop = [0, 1]
nextup_youtube = nextup_columns_to_drop(nextup_youtube, columns_to_drop, True)

In [ ]:
nextup_youtube_columns_upd = ['тематика', 'ца по возрасту', 'ца по полу', 'ца по странам']
nextup_youtube = nextup_col_upd(nextup_youtube, nextup_youtube_columns_upd)

In [ ]:
[col for col in nextup_youtube.columns]

In [ ]:
[col for col in bufer.columns if 'YouTube' in col]

In [ ]:
nextup_youtube.rename(columns = {'имя блогера':'блогер',
                                 'кол-во подписчиков':'количество подписчиков',
                                 'охват видео за 30 дней':'среднее кол-во просмотров видео',
                                 'стоимость интеграция / 1 слот':'стоимость интеграция 1 слот',
                                 'стоимость интеграция / 2 слот':'стоимость интеграция 2 слот',
                                 'стоимость преролл':'стоимость преролл до 30 сек',
                                 'статистика ссылка':'статистика'
                                }, inplace=True)

In [ ]:
nextup_youtube.head()

In [ ]:
columns_to_drop = [0]
nextup_youtube = nextup_columns_to_drop(nextup_youtube, columns_to_drop, False)

In [ ]:
nextup_youtube.head()

#### Проверка на дубликаты

In [ ]:
show_duplicates(nextup_youtube)

### Двадцать пятое объединение

In [ ]:
bufer = bufer.astype(str)

In [ ]:
bufer = merge_bufer_df(bufer, nextup_youtube, 'YouTube')

In [ ]:
show_duplicates(bufer).head(6)

In [ ]:
# До

bufer.shape

In [ ]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [ ]:
# После

bufer.shape

In [ ]:
duplicates = bufer[bufer.duplicated('блогер', keep=False)]
duplicates.sort_values(by='блогер')

> Удалим дубликаты, оставив только первое вхождение, так как чаще всего именно в них самые большие и видимо правильные значения

In [ ]:
bufer = bufer.drop_duplicates(subset=['блогер'], keep='first')

In [ ]:
show_duplicates(bufer).head(6)

### Подготовка таблицы nextup_twitch

In [ ]:
nextup_twitch.head()

In [ ]:
columns_to_drop = [0, 1]
nextup_twitch = nextup_columns_to_drop(nextup_twitch, columns_to_drop, True)

In [ ]:
nextup_twitch.head(1)

In [ ]:
[col for col in nextup_twitch.columns]

In [ ]:
[col for col in bufer.columns if 'Twitch' in col]

In [ ]:
nextup_twitch.rename(columns = {'имя блогера':'блогер',
                                'кол-во подписчиков':'количество подписчиков',
                                'средний онлайн на стриме':'среднее кол-во зрителей',
                                'охват стрима за 14 дней':'уникальные зрители (14 дней)',
                                'баннер + чат-бот + голосовая интеграция до 3 мин. на 1 стрим':'стоимость за баннер, чат-бот, голосовая интеграция до 3 мин. на 1 стрим',
                                'баннер + чат-бот + эксклюзивный стрим до 1 часа':'стоимость за баннер, чат-бот, эксклюзивный стрим до 1 часа',
                                'баннер + чат-бот на 1 месяц + голосовое упоминание 1 раз в неделю (1 месяц)':'стоимость за баннер, чат-бот на 1 месяц, голосовое упоминание 1 раз в неделю (1 месяц)',
                                'статистика ссылка':'статистика'
                               }, inplace=True)

#### Проверка на дубликаты

In [ ]:
show_duplicates(nextup_twitch)

### Двадцать шестое объединение

In [ ]:
bufer = merge_bufer_df(bufer, nextup_twitch, 'Twitch')

In [ ]:
show_duplicates(bufer)

In [ ]:
# До

bufer.shape

In [ ]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [ ]:
# После

bufer.shape

In [ ]:
show_duplicates(bufer)

> Удалим дубликаты, оставив только первое вхождение

In [ ]:
bufer = bufer.drop_duplicates(subset=['блогер'], keep='first')

### Подготовка таблицы nextup_vk

In [ ]:
nextup_vk.head()

In [ ]:
columns_to_drop = [0, 1]
nextup_vk = nextup_columns_to_drop(nextup_vk, columns_to_drop, True)

In [ ]:
nextup_vk_columns_upd = ['ца по городам', 'ца по возрасту', 'ца по полу']
nextup_vk = nextup_col_upd(nextup_vk, nextup_vk_columns_upd)

In [ ]:
[col for col in nextup_vk.columns]

In [ ]:
[col for col in bufer.columns if 'VK группа' in col]

In [ ]:
nextup_vk.rename(columns = {'имя блогера':'блогер',
                            'кол-во подписчиков':'количество подписчиков',
                            'охват поста за 30 дней':'среднее кол-во просмотров поста',
                            'охват vk клип за 30 дней':'среднее кол-во просмотров вк клипа',
                            'стоимость поста':'стоимость фотопост',
                            'стоимость vk клип':'стоимость вк клип',
                            'статистика ссылка':'статистика'
                           }, inplace=True)

In [ ]:
nextup_vk.head(1)

#### Проверка на дубликаты

In [ ]:
show_duplicates(nextup_vk)

### Двадцать седьмое объединение

In [ ]:
bufer = merge_bufer_df(bufer, nextup_vk, 'VK группа')

In [ ]:
show_duplicates(bufer).head(6)

In [ ]:
# До

bufer.shape

In [ ]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [ ]:
# После

bufer.shape

In [ ]:
show_duplicates(bufer)

> Удалим дубликаты, оставив только первое вхождение

In [ ]:
bufer = bufer.drop_duplicates(subset=['блогер'], keep='first')

In [ ]:
#Заполним контактов менеджера
bufer['контакты менеджера'].fillna('https://t.me/Oliva_Nick', inplace=True)

#Заполним значение налога
bufer['налог'].fillna('НДС (6%)', inplace=True)

> Посмотрим на нашу таблицу bufer

In [ ]:
bufer

## Подготовка данных по блогерам от AvtorMedia

> Предварительно посмотрим на содержание таблицы

In [ ]:
avtormedia_telegram.head(1)

In [ ]:
avtormedia_df = [avtormedia_youtube,
                 avtormedia_instagram,
                 avtormedia_telegram,
                 avtormedia_vk,
                 avtormedia_zen,
                 avtormedia_podcast]

In [ ]:
key_word = ['CPV']  # Список ключевых слов для поиска столбцов и их удаления
drop_word = ''  # Список ключевых слов для их удаления из названия столбцов
upd_columns = ['Блогер']

In [ ]:
# for df in avtormedia_df:
#     pretraining_df(df, key_word, drop_word, upd_columns)
    
    
for i in range(len(avtormedia_df)):
    df = avtormedia_df[i]  # Получаем текущий DataFrame по индексу
    
    upd_columns = []  # Пустой список столбцов с именами блогеров для приведения к единому формату

    if 'Блогер' in df.columns:
        upd_columns.append('Блогер')
    if 'Канал' in df.columns:
        upd_columns.append('Канал')

    pretraining_df(df, key_word, drop_word, upd_columns)

### Подготовка таблицы avtormedia_youtube

In [ ]:
avtormedia_youtube.head()

In [ ]:
[col for col in avtormedia_youtube.columns]

In [ ]:
bufer_columns = ['блогер',
                 'тематика',
                 'количество подписчиков',
                 'материалы и статистика',
                 'стоимость мидролл до 30 сек',
                 'среднее кол-во просмотров видео',
                 'стоимость интеграция 1 слот',
                 'стоимость преролл до 30 сек',
                 'стоимость интеграция 2 слот',
                 'стоимость спецпроект',
                 'среднее кол-во просмотров shorts',
                 'стоимость за shorts до 1 мин',
                 'спешл 1',
                 'цена 1',
                 'спешл 2',
                 'цена 2',
                 'спешл 3',
                 'цена 3',
                 'спешл 4',
                 'цена 4',
                 'спешл 5',
                 'цена 5',
                 'спешл 6',
                 'цена 6',
                 'спешл 7',
                 'цена 7',
                 'пакетное предложение 8',
                 'цена 8',
                 'охват 8',
                 'пакетное предложение 9',
                 'цена 9',
                 'охват 9',
                 'пакетное предложение 10',
                 'цена 10',
                 'охват 10',
                 'пакетное предложение 11',
                 'цена 11',
                 'охват 11',
                 'пакетное предложение 12',
                 'цена 12',
                 'охват 12',
                 'ссылка',
                 'статистика']

In [ ]:
[col for col in bufer.columns if 'YouTube' in col]

In [ ]:
avtormedia_youtube.columns = bufer_columns

In [ ]:
avtormedia_youtube.head()

In [ ]:
column_list_1 = []

for column_number in range(1, 8):
    position_column = f'спешл {column_number}'
    price_column = f'цена {column_number}'
    avtormedia_youtube = get_column_values(avtormedia_youtube,
                                           column_number,
                                           position_column, 
                                           price_column, 
                                           column_list_1)

In [ ]:
avtormedia_youtube.head()

In [ ]:
sorted_column_names_1 = sorted(column_list_1)
sorted_column_names_1

In [ ]:
def get_column_values_packet(df: pd.DataFrame,
                             key: int,
                             column_with_position: str,
                             column_with_price: str,
                             column_with_coverage: str,
                             column_list: list) -> pd.DataFrame:
    
    """
    АРГУМЕНТЫ ФУНКЦИИ:
        df (pd.DataFrame) - DataFrame, в котором делаем преобразования;
        key (int) - Ключ в виде int, который нужно добавить к названию, 
            чтобы в дальнейшем объединить несколько столбцов с одинаковой позицией;
        column_with_position (str) - Столбец, в котором указаны позиции;
        column_with_price (str) - Столбец, в котором указаны цены на позиции;
        column_with_coverage (str) - Столбец, в котором указан охват позиции;
        column_list (list) - 
    
    
    
    ОПИСАНИЕ ВЫПОЛНЕНИЯ ФУНКЦИИ С ПРИМЕРОМ:
    
        Создаём список с уникальными значениями позиций,
            приведя их к единому формату, используя .strip(), .str.lower(), .str.replace:
        spisok = avtormedia_youtube_actual['Спешл'].str.strip()\
                                                   .str.lower()\
                                                   .str.replace('/\n', ', ')\
                                                   .str.replace('\n', ', ')\
                                                   .str.replace('*', '')\
                                                   .unique()

        #Большинство значений строк в столбце column_with_position - NaN, у которых нет определённых цен.
        #Мы удалим данные позиции из списка, чтобы не создавать для них столбцы и/или не получать ошибку
        spisok = spisok[~pd.isna(spisok)]


        # Цикл for проходит по списку уникальных позиций и вводит их названия:
        for i in range(len(unique_position)):

        # Делаем уникальный столбец, название которого соотвествует уникальной позиции, заполняя его значениями соответсвующей цены:
            df[f"{unique_position[i]}_{key}"] = df[column_with_price].\
            where(df[column_with_position] == unique_position[i], other='')


        Удаляем исходные столбцы, чтобы не было повтора:
        df.drop(columns=[column_with_price, column_with_position], axis=1, inplace= True)


        Возвращаем обновлённый DataFrame
        return df    
                
    """

    unique_position = df[column_with_position].str.strip()\
                                              .unique()
    
    unique_position = unique_position[~pd.isna(unique_position)]

# Цикл for проходит по списку уникальных позиций и вводит их названия:
    for i in range(len(unique_position)):
        
        new_column_name = f'{unique_position[i]}_{key}'
        
# Делаем уникальный столбец, название которого соотвествует уникальной позиции, заполняя его значениями соответсвующей цены:    
        df[new_column_name] = \
                df[column_with_price].where(df[column_with_position] == unique_position[i], other='')
            
        df[f'охват {new_column_name}'] = \
                df[column_with_coverage].where(df[column_with_position] == unique_position[i], other='')
        
        
        # Добавляем имема новых столбцов в список, чтобы в дальнейшем работать с ними
        column_list.append(new_column_name) 
        
        # Заменим пустую строку на NaN
        df[new_column_name] = df[new_column_name].replace('', np.nan)
            
    df.drop(columns=[column_with_price, column_with_position], axis=1, inplace=True)
    
    return df

In [ ]:
column_list_2 = []

for column_number in range(8, 13):
    position_column = f'пакетное предложение {column_number}'
    price_column = f'цена {column_number}'
    coverage_column = f'охват {column_number}'
    avtormedia_youtube = get_column_values_packet(avtormedia_youtube,
                                                  column_number,
                                                  position_column,
                                                  price_column,
                                                  coverage_column,
                                                  column_list_2)

In [ ]:
avtormedia_youtube.head()

In [ ]:
sorted_column_names_2 = sorted(column_list_2)
sorted_column_names_2

#### Обработаем новые столбцы

##### Начнём со списка sorted_column_names_1

In [ ]:
dict_new_columns = {'стоимость усложненная интеграция':['Усложненная интеграция_2',
                                                        'Усложненная интеграция_3',
                                                        'Расширенная интеграция_1'],
                    
                    'стоимость пост в сообществе': ['Пост в сообществе_2',
                                                    'Пост в сообществе_3',
                                                    'Пост в сообществе_5'],
                    
                    'стоимость выездная интеграция':['Выездная интеграция_2',
                                                    'Выездная интеграция_3'],
                    
                   'стоимость интеграция с выбором темы видео':['Расширенная интеграция в ролик на тему по выбору клиента_1',
                                                                'Возможность выбрать тему ролика_6',
                                                                'Интеграция в ролик на тему по выбору клиента_2'],  
                    
                    'стоимость мидролл до 30 сек':['стоимость мидролл до 30 сек',
                                                   'Мидролл_1']
                    
                   }

In [ ]:
def agg_columns(df: pd.DataFrame, dictation: dict) -> pd.DataFrame:
    
    """
    
    
    ПАРАМЕТРЫ:
        df: pd.DataFrame - 
        dictation: dict
 
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    
    """
    
    for new_column, old_column in dictation.items():

        for col in old_column:
            # Чтобы избежать преобразования значений NaN в строки "nan" при объединении строк,
            # мы будем использовать метод .fillna('') для замены значений NaN на пустые строки перед объединением
            df[col] = df[col].fillna('').astype(str)

        df[new_column] = df[old_column].agg(''.join, axis=1)  

        #Вернём обратно NaN
        df[new_column] = df[new_column].replace('', np.nan)
        
#         # Удаляем столбцы из словаря
#         df.drop(columns=old_column, inplace=True)
        
    return df

In [ ]:
# avtormedia_youtube = agg_columns(avtormedia_youtube, dict_new_columns)

In [ ]:
for new_column, old_column in dict_new_columns.items():
    for col in old_column:
        # Чтобы избежать преобразования значений NaN в строки "nan" при объединении строк,
        # мы будем использовать метод .fillna('') для замены значений NaN на пустые строки перед объединением
        avtormedia_youtube[col] = avtormedia_youtube[col].fillna('').astype(str)
    
    avtormedia_youtube[new_column] = avtormedia_youtube[old_column].agg(''.join, axis=1)  
    
    #Вернём обратно NaN
    avtormedia_youtube[new_column] = avtormedia_youtube[new_column].replace('', np.nan)
    avtormedia_youtube[col] = avtormedia_youtube[col].astype(str)

In [ ]:
avtormedia_youtube.drop(columns=['Усложненная интеграция_2',
                                 'Усложненная интеграция_3',
                                 'Расширенная интеграция_1',
                                 'Пост в сообществе_2',
                                 'Пост в сообществе_3',
                                 'Пост в сообществе_5',
                                 'Выездная интеграция_2',
                                 'Выездная интеграция_3',
                                 'Расширенная интеграция в ролик на тему по выбору клиента_1',
                                 'Возможность выбрать тему ролика_6',
                                 'Интеграция в ролик на тему по выбору клиента_2',
                                 'Мидролл_1'], axis=1, inplace=True)

In [ ]:
avtormedia_youtube.head()

In [ ]:
[col for col in bufer.columns if 'YouTube' in col]

In [ ]:
avtormedia_youtube.rename(columns = {'Бегущая строка_1':'стоимость бегущая строка',
                                     'Мидролл_1':'стоимость мидролл до 30 сек',
                                     'Выездная интеграция в воскресный выпуск новостей "Чё Происходит"_5':'стоимость выездная интеграция в воскресный выпуск новостей "Чё Происходит"',
                                     'Выездной спецпроект_1':'стоимость выездной спецпроект',
                                     'Интеграция (3 слот)_2':'стоимость интеграция 3 слот',
                                     'Интеграция (4 слот)_3':'стоимость интеграция 4 слот',
                                     'Интеграция в воскресный выпуск новостей "Чё Происходит" (2 слот)_2':'стоимость интеграция в воскресный выпуск новостей "Чё Происходит" 2 слот',
                                     'Интеграция в воскресный выпуск новостей "Чё Происходит" 1 слот_4':'стоимость интеграция в воскресный выпуск новостей "Чё Происходит" 1 слот',
                                     'Интеграция в обзор_5':'стоимость интеграция в обзор',
                                     'Интеграция в подкаст «Сладкая жизнь» (1 слот)_3':'стоимость интеграция в подкаст «Сладкая жизнь» 1 слот',
                                     'Интеграция в подкаст «Сладкая жизнь» (2 слот)_6':'стоимость интеграция в подкаст «Сладкая жизнь» 2 слот',
                                     'Интеграция в разборы_4':'стоимость интеграция в разборы',
                                     'Интеграция в реакцию на трейлер_2':'стоимость интеграция в реакцию на трейлер',
                                     'Интеграция в рецепт_4':'стоимость интеграция в рецепт',
                                     'Интеграция в ролик с Линой Диановой (1 слот)_1':'стоимость интеграция в ролик с Линой Диановой 1 слот',
                                     'Интеграция в ролик с Линой Диановой (2 слот)_2':'стоимость интеграция в ролик с Линой Диановой 2 слот',
                                     'Интеграция в тревел-ролики_1':'стоимость интеграция в тревел-ролики',
                                     'Интеграция в формат "Криминальный ТикТок"_2':'стоимость интеграция в формат "Криминальный ТикТок"',
                                     'Интеграция в формат «Книга жалоб» (1 слот)_4':'стоимость интеграция в формат «Книга жалоб» 1 слот',
                                     'Интеграция в формат «Книга жалоб» (2 слот)_1':'стоимость интеграция в формат «Книга жалоб» 2 слот',
                                     'Интеграция в формат «Спешл»_5':'стоимость интеграция в формат «Спешл»',
                                     'Интеграция в формат Стиратель на YouTube_1':'стоимость интеграция в формат Стиратель на YouTube',
                                     'Интеграция в формат новости_3':'стоимость интеграция в формат новости',
                                     'Интеграция с участием Максима Каца_1':'стоимость интеграция с участием Максима Каца',
                                     'Повторное размещение реализованной интеграции_1':'стоимость повторное размещение реализованной интеграции',
                                     'Пре-ролл  в формат "Линия выгорания"_3':'стоимость преролл в формат "Линия выгорания"',
                                     'Пре-ролл в подкаст «Сладкая жизнь»_7':'стоимость преролл в подкаст «Сладкая жизнь»',
                                     'Пре-ролл в ролик с Линой Диановой_4':'стоимость преролл в ролик с Линой Диановой',
                                     'Пре-ролл в формат «Книга жалоб»_2':'стоимость преролл в формат «Книга жалоб»',
                                     'Преролл в обзор_3':'стоимость преролл в обзор',
                                     'Преролл в реакцию на трейлер или разборы_5':'стоимость преролл в реакцию на трейлер или разборы',
                                     'Преролл в рецепт_2':'стоимость преролл в рецепт',
                                     'Преролл в формат новости_1':'стоимость преролл в формат новости',
                                     'Продакт-плейсмент_4':'стоимость продакт плейсмент',
                                     'Ссылка в закрепленном комментарии и ссылка в описании до ката_4':'стоимость псылка в закрепленном комментарии и ссылка в описании до ката',
                                    }, inplace=True)

##### Обработаем список sorted_column_names_2

In [ ]:
sorted_column_names_2

In [ ]:
avtormedia_youtube.rename(columns = {'3 интеграции_8':'стоимость 3 интеграции',
                                     'Cпецпроект + фото-пост в Инст_8':'стоимость Cпецпроект и фотопост в Инстаграм',
                                     'Интеграция + фото-пост в Инст_9':'стоимость интеграция и фотопост в Инстаграм',
                                     'Интеграция в видео на YouTube + пост в сообществе YouTube + ссылка в закрепленном комментарии_10':'стоимость интеграция в видео на YouTube, пост в сообществе YouTube и ссылку в закрепленном комментарии',
                                     'Интеграция в видео на YouTube + пост в сообществе YouTube_11':'стоимость интеграция в видео на YouTube и пост в сообществе YouTube',
                                     'Интеграция в видео на YouTube + фото-пост в Инст_8':'стоимость интеграция в видео на YouTube и фотопост в Инстаграм',
                                     'Интеграция в видео на YouTube Алиса Лисова + интеграция в видео на YouTube Lisova_8':'стоимость интеграция в видео на YouTube Алиса Лисова и интеграцию в видео на YouTube Lisova',
                                     'Интеграция в обзор + фото в Инст_8':'стоимость интеграция в обзор и фотопост в Инстаграм',
                                     'Интеграция в рецепт + фото в Инст_9':'стоимость интеграция в рецепт и фотопост в Инстаграм',
                                     'Интеграция на YouTube + Сторис в Instagram_12':'стоимость интеграция на YouTube и сторис в Инстаграм',
                                     'Интеграция на YouTube канал Дюшес + сторис в Инст (серия из трех)_8':'стоимость интеграция на YouTube канал Дюшес и за серию из 3х сторис в Инстаграм',
                                     'Интеграция на Основном канале + Романов Говорит_8':'стоимость интеграция на Основном канале + Романов Говорит',
                                     'Спецпроект + Анонс конкурса_8':'стоимость спецпроект и анонс конкурса',
                                     'Спецпроект на YouTube + пост в Telegram_9':'стоимость спецпроект на YouTube и пост в Телеграм',
                                     'интеграция YouTube + Дзен_8':'стоимость интеграция YouTube и Zen'
                                    }, inplace=True)

In [ ]:
[col for col in avtormedia_youtube.columns if 'охват' in col]

> Отредактируем также название столбов с охватом пакетных предложений

In [ ]:
avtormedia_youtube.rename(columns = {'охват 3 интеграции_8':'охват позиции 3 интеграции',
                                     'охват Cпецпроект + фото-пост в Инст_8':'охват позиции Cпецпроект и фотопост в Инстаграм',
                                     'охват Интеграция на YouTube канал Дюшес + сторис в Инст (серия из трех)_8':'охват позиции интеграция на YouTube канал Дюшес и за серию из 3х сторис в Инстаграм',
                                     'охват Интеграция в видео на YouTube Алиса Лисова + интеграция в видео на YouTube Lisova_8':'охват позиции интеграция в видео на YouTube Алиса Лисова и интеграцию в видео на YouTube Lisova',
                                     'охват Интеграция в видео на YouTube + фото-пост в Инст_8':'охват позиции интеграция в видео на YouTube и фотопост в Инстаграм',
                                     'охват Спецпроект + Анонс конкурса_8':'охват позиции спецпроект и анонс конкурса',
                                     'охват Интеграция в обзор + фото в Инст_8':'охват позиции интеграция в обзор и фотопост в Инстаграм',
                                     'охват интеграция YouTube + Дзен_8':'охват позиции интеграция YouTube и Zen',
                                     'охват Интеграция на Основном канале + Романов Говорит_8':'охват позиции интеграция на Основном канале + Романов Говорит',
                                     'охват Интеграция + фото-пост в Инст_9':'охват позиции интеграция и фотопост в Инстаграм',
                                     'охват Спецпроект на YouTube + пост в Telegram_9':'охват позиции спецпроект на YouTube и пост в Телеграм',
                                     'охват Интеграция в рецепт + фото в Инст_9':'охват позиции интеграция в рецепт и фотопост в Инстаграм',
                                     'охват Интеграция в видео на YouTube + пост в сообществе YouTube + ссылка в закрепленном комментарии_10':'охват позиции интеграция в видео на YouTube, пост в сообществе YouTube и ссылку в закрепленном комментарии',
                                     'охват Интеграция в видео на YouTube + пост в сообществе YouTube_11':'охват позиции интеграция в видео на YouTube и пост в сообществе YouTube',
                                     'охват Интеграция на YouTube + Сторис в Instagram_12':'охват позиции интеграция на YouTube и сторис в Инстаграм'
                                    }, inplace=True)

In [ ]:
avtormedia_youtube.drop(columns=['материалы и статистика',
                                 'охват 8',
                                 'охват 9',
                                 'охват 10',
                                 'охват 11',
                                 'охват 12'], inplace=True)

In [ ]:
avtormedia_youtube.head()

> Отчистим пустые строки и сделаем их как NaN

In [ ]:
# # Получаем список столбцов, содержащих слово "охват"
# ohvat_columns = [col for col in avtormedia_youtube.columns if 'охват' in col]

# # Заменяем пустые значения в этих столбцах на NaN
# avtormedia_youtube[ohvat_columns] = avtormedia_youtube[ohvat_columns].replace('', np.nan)

#### Проверка на дубликаты

In [ ]:
show_duplicates(avtormedia_youtube)

### Двадцать восьмое объединение

In [ ]:
# # Преобразуйте все столбцы в строки (str)
# avtormedia_youtube = avtormedia_youtube.astype(str)
# bufer = bufer.astype(str)

In [ ]:
bufer = merge_bufer_df(bufer, avtormedia_youtube, 'YouTube')

In [ ]:
# Посмотрим на размер аудитории

bufer.shape

### Подготовка таблицы avtormedia_instagram

In [ ]:
avtormedia_instagram.head()

In [ ]:
[col for col in avtormedia_instagram.columns]

In [ ]:
[col for col in bufer.columns if 'Instagram' in col]

In [ ]:
avtormedia_instagram

In [ ]:
bufer_columns = ['блогер',
                 'тематика',
                 'количество подписчиков',
                 'материалы и статистика',
                 'среднее кол-во просмотров 1 сторис',
                 'стоимость 1 сторис',
                 'среднее кол-во просмотров фотопоста',
                 'стоимость фотопост',
                 'стоимость карусель',
                 'среднее кол-во просмотров reels / видеопоста',
                 'стоимость reels / видеопост',
                 'пакетное предложение 1',
                 'цена 1',
                 'охват 1',
                 'пакетное предложение 2',
                 'цена 2',
                 'охват 2',
                 'пакетное предложение 3',
                 'цена 3',
                 'охват 3',
                 'пакетное предложение 4',
                 'цена 4',
                 'охват 4',
                 'ссылка',
                 'статистика']

avtormedia_instagram.columns = bufer_columns

In [ ]:
# column_list_3 = []

# for column_number in range(1, 3):
#     position_column = f'спешл {column_number}'
#     price_column = f'цена {column_number}'
#     avtormedia_instagram = get_column_values(avtormedia_instagram,
#                                              column_number,
#                                              position_column,
#                                              price_column,
#                                              column_list_3)

In [ ]:
column_list_4 = []

for column_number in range(1, 5):
    position_column = f'пакетное предложение {column_number}'
    price_column = f'цена {column_number}'
    coverage_column = f'охват {column_number}'
    avtormedia_instagram = get_column_values_packet(avtormedia_instagram,
                                                    column_number,
                                                    position_column,
                                                    price_column,
                                                    coverage_column,
                                                    column_list_4)

##### Обработаем список sorted_column_names_3

In [ ]:
# sorted_column_names_3 = sorted(column_list_3)
# sorted_column_names_3

In [ ]:
# avtormedia_instagram.rename(columns = {'Рекламный день (сториз 7-10 экранов)_2':'стоимость рекламный день (сториз 7-10 экранов)',
#                                        'Сториз (5 экранов)_1':'стоимость сториз (5 экранов)'}, inplace = True)

##### Обработаем список sorted_column_names_4

In [ ]:
sorted_column_names_4 = sorted(column_list_4)
sorted_column_names_4

In [ ]:
dict_new_columns = {'стоимость фотопост в Инстаграм + пост в Телеграм': [ 'Фото-пост в Инст + пост в Telegram_1',
                                                                         'Фото-пост в Инст + пост в Telegram_3'],
                   
                   'охват позиции стоимость фотопост в Инстаграм + пост в Телеграм':['охват Фото-пост в Инст + пост в Telegram_1',
                                                                              'охват Фото-пост в Инст + пост в Telegram_3']}


# avtormedia_instagram = agg_columns(avtormedia_instagram, dict_new_columns)

In [ ]:
for new_column, old_column in dict_new_columns.items():
    for col in old_column:
        # Чтобы избежать преобразования значений NaN в строки "nan" при объединении строк,
        # мы будем использовать метод .fillna('') для замены значений NaN на пустые строки перед объединением
        avtormedia_instagram[col] = avtormedia_instagram[col].fillna('').astype(str)
    
    avtormedia_instagram[new_column] = avtormedia_instagram[old_column].agg(''.join, axis=1)  
    
    #Вернём обратно NaN
    avtormedia_instagram[new_column] = avtormedia_instagram[new_column].replace('', np.nan)
    avtormedia_instagram[col] = avtormedia_instagram[col].astype(str)

In [ ]:
avtormedia_instagram.rename(columns = {'Cторис (серия из трех) + Пост ТГ_4':'стоимость серия из 3х сторис и пост в Телеграм',
                                       'Интеграция в видео на YouTube + фото-пост в Инст_1':'стоимость интеграция в видео на Ютуб и фотопост в Инстаграм',
                                       'Интеграция в обзор + фото в Инст_2':'стоимость интеграция в обзор и фотопост в Инстаграм',
                                       'Интеграция в рецепт + фото в Инст_1':'стоимость интеграция в рецепт и фотопост в Инстаграм',
                                       'Интеграция на YouTube + Сторис в Instagram_2':'стоимость интеграция на Ютуб и сторис в Инстаграм',
                                       'Пост в Telegram (удаление через сутки) + Пост в Instagram_1':'стоимость пост в Телеграм (удаление через сутки) и пост в Инстаграм',
                                       'Пост в ТГ + сторис в ИГ_1':'стоимость пост в Телеграм и сторис в Инстаграм',
                                       'Фото в Инст + сторис в Инст (серия из трех) + пост в ВК_2':'стоимость фотопост в Инстаграм, серию из 3х сторис в Инстаграм и пост в ВК',
                                       'Фото в Инст + сторис в Инст (серия из трех)_1':'стоимость фотопост в Инстаграм + серию из 3х сторис в Инстаграм'
                                      }, inplace=True)

In [ ]:
avtormedia_instagram.columns

In [ ]:
avtormedia_instagram.drop(columns=['Фото-пост в Инст + пост в Telegram_1',
                                   'Фото-пост в Инст + пост в Telegram_3',
                                   'охват Фото-пост в Инст + пост в Telegram_1',
                                   'охват Фото-пост в Инст + пост в Telegram_3',
                                      
                                   'охват 1',
                                   'охват 2',
                                   'охват 3',
                                   'охват 4'], inplace=True)

In [ ]:
[col for col in avtormedia_instagram.columns if 'охват' in col]

In [ ]:
avtormedia_instagram.rename(columns = {'охват Пост в ТГ + сторис в ИГ_1':'охват позиции стоимость за пост в Телеграм и сторис в Инстаграм',
                                       'охват Интеграция в видео на YouTube + фото-пост в Инст_1':'охват позиции стоимость за интеграцию в видео на Ютуб и фотопост в Инстаграм',
                                       'охват Фото в Инст + сторис в Инст (серия из трех)_1':'охват позиции стоимость за фотопост в Инстаграм + серию из 3х сторис в Инстаграм',
                                       'охват Интеграция в рецепт + фото в Инст_1':'охват позиции стоимость за интеграцию в рецепт и фотопост в Инстаграм',
                                       'охват Пост в Telegram (удаление через сутки) + Пост в Instagram_1':'охват позиции стоимость за пост в Телеграм (удаление через сутки) и пост в Инстаграм',
                                       'охват Интеграция на YouTube + Сторис в Instagram_2':'охват позиции стоимость за интеграцию на Ютуб и сторис в Инстаграм',
                                       'охват Фото в Инст + сторис в Инст (серия из трех) + пост в ВК_2':'охват позиции стоимость за фотопост в Инстаграм, серию из 3х сторис в Инстаграм и пост в ВК',
                                       'охват Интеграция в обзор + фото в Инст_2':'охват позиции стоимость за интеграцию в обзор и фотопост в Инстаграм',
                                       'охват Cторис (серия из трех) + Пост ТГ_4':'охват позиции стоимость за серию из 3х сторис и пост в Телеграм'
                                      }, inplace = True)
                                   

> Отчистим пустые строки и сделаем их как NaN

In [ ]:
# Получаем список столбцов, содержащих слово "охват"
ohvat_columns = [col for col in avtormedia_instagram.columns if 'охват' in col]

# Заменяем пустые значения в этих столбцах на NaN
avtormedia_instagram[ohvat_columns] = avtormedia_instagram[ohvat_columns].replace('', np.nan)

#### Проверка на дубликаты

In [ ]:
show_duplicates(avtormedia_instagram)

### Двадцать девятое объединение

In [ ]:
bufer = merge_bufer_df(bufer, avtormedia_instagram, 'Instagram')

In [ ]:
show_duplicates(bufer).head(6)

In [ ]:
# До

bufer.shape

In [ ]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [ ]:
# После

bufer.shape

In [ ]:
show_duplicates(bufer)

### Подготовка таблицы avtormedia_telegram

In [ ]:
avtormedia_telegram.head(1)

In [ ]:
bufer_columns = ['блогер',
                 'тематика',
                 'количество подписчиков',
                 'материалы и статистика',
                 'среднее кол-во просмотров поста',
                 'стоимость видеопост',
                 'стоимость фотопост',
                 'спешл 1',
                 'цена 1',
                 'спешл 2',
                 'цена 2',
                 'пакетное предложение 3',
                 'цена 3',
                 'охват 3',
                 'пакетное предложение 4',
                 'цена 4',
                 'охват 4',
                 'ссылка',
                 'статистика']

avtormedia_telegram.columns = bufer_columns

In [ ]:
column_list_5 = []

for column_number in range(1, 3):
    position_column = f'спешл {column_number}'
    price_column = f'цена {column_number}'
    avtormedia_telegram = get_column_values(avtormedia_telegram,
                                            column_number,
                                            position_column,
                                            price_column,
                                            column_list_5)

In [ ]:
column_list_6 = []

for column_number in range(3, 5):
    position_column = f'пакетное предложение {column_number}'
    price_column = f'цена {column_number}'
    coverage_column = f'охват {column_number}'
    avtormedia_telegram = get_column_values_packet(avtormedia_telegram,
                                                   column_number,
                                                   position_column,
                                                   price_column,
                                                   coverage_column,
                                                   column_list_6)

##### Обработаем список sorted_column_names_5

In [ ]:
sorted_column_names_5 = sorted(column_list_5)
sorted_column_names_5

In [ ]:
dict_new_columns = {'стоимость кружочек до 60 сек': ['Кружок_1',
                                                     'Кружок_2']
                   }

avtormedia_telegram = agg_columns(avtormedia_telegram, dict_new_columns)

In [ ]:
avtormedia_telegram.drop(columns=['Кружок_1',
                                   'Кружок_2'], inplace=True)

In [ ]:
avtormedia_telegram.rename(columns = {'Авторский пост_1':'стоимость за авторский пост',
                                      'Пост без удаления_1':'стоимость за пост из удаления',
                                      'Пост 2/48_2':'стоимость за пост (2/48)',
                                      'Пост 3/72_1':'стоимость за пост (3/72)',
                                      'Сторис_1':'стоимость 1 сторис до 15 сек'}, inplace=True)

##### Обработаем список sorted_column_names_6

In [ ]:
sorted_column_names_6 = sorted(column_list_6)
sorted_column_names_6

In [ ]:
avtormedia_telegram.rename(columns = {'3 поста в TG_3':'стоимость 3 поста в Телеграм',
                                      '5 постов в Telegram (удаление через сутки)_3':'стоимость 5 постов в Телеграм (удаление через сутки)',
                                      'Cторис (серия из трех) + Пост ТГ_4':'стоимость серия из 3х сторис в Инстаграм и пост в Телеграм',
                                      'Пост в Telegram (удаление через сутки) + Пост в Instagram_4':'стоимость пост в Телеграм (удаление через сутки) и пост в Инстаграм',
                                      'Пост в ТГ + сторис в ИГ_3':'стоимость пост в Телеграм и сториз в Инстаграм',
                                      'Рекламное сообщение во все чаты "Вместе"_3':'стоимость рекламное сообщение во все чаты "Вместе"',
                                      'Спецпроект на YouTube + пост в Telegram_3':'стоимость спецпроект на Ютуб и пост в Телеграм',
                                      'Фото-пост в Инст + пост в Telegram_3':'стоимость фотопост в Инстаграм и пост в Телеграм'
                                     }, inplace=True)

In [ ]:
avtormedia_telegram.rename(columns = {'охват Фото-пост в Инст + пост в Telegram_3':'охват позиции стоимость за фотопост в Инстаграм и пост в Телеграм',
                                      'охват Спецпроект на YouTube + пост в Telegram_3':'охват позиции стоимость за спецпроект на Ютуб и пост в Телеграм',
                                      'охват Пост в ТГ + сторис в ИГ_3':'охват позиции стоимость за пост в Телеграм и сториз в Инстаграм',
                                      'охват 5 постов в Telegram (удаление через сутки)_3':'охват позиции стоимость за 5 постов в Телеграм (удаление через сутки)',
                                      'охват 3 поста в TG_3':'охват позиции стоимость за 3 поста в Телеграм',
                                      'охват Рекламное сообщение во все чаты "Вместе"_3':'охват позиции стоимость за рекламное сообщение во все чаты "Вместе"',
                                      'охват Пост в Telegram (удаление через сутки) + Пост в Instagram_4':'охват позиции стоимость за пост в Телеграм (удаление через сутки) и пост в Инстаграм',
                                      'охват Cторис (серия из трех) + Пост ТГ_4':'охват позиции стоимость за серию из 3х сторис в Инстаграм и пост в Телеграм'
                                     }, inplace=True)

In [ ]:
avtormedia_telegram.drop(columns=['материалы и статистика',
                                  'охват 3',
                                  'охват 4'], inplace=True)

> Отчистим пустые строки и сделаем их как NaN

In [ ]:
# Получаем список столбцов, содержащих слово "охват"
ohvat_columns = [col for col in avtormedia_telegram.columns if 'охват' in col]

# Заменяем пустые значения в этих столбцах на NaN
avtormedia_telegram[ohvat_columns] = avtormedia_telegram[ohvat_columns].replace('', np.nan)

#### Проверка на дубликаты

In [ ]:
show_duplicates(avtormedia_telegram)

### Тридцатое объединение

In [ ]:
bufer = merge_bufer_df(bufer, avtormedia_telegram, 'Telegram')

In [ ]:
# duplicates = bufer[bufer.duplicated('блогер', keep=False)]
# duplicates.sort_values(by='блогер').head(6)

show_duplicates(bufer).head(6)

In [ ]:
# До

bufer.shape

In [ ]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [ ]:
# После

bufer.shape

In [ ]:
show_duplicates(bufer)

### Подготовка таблицы avtormedia_vk

In [ ]:
avtormedia_vk.head()

In [ ]:
[col for col in bufer.columns if 'VK группа' in col]

In [ ]:
bufer_columns = ['блогер',
                 'тематика',
                 'количество подписчиков',
                 'материалы и статистика',
                 'среднее кол-во просмотров поста',
                 'стоимость вк клип',
                 'стоимость сторис',
                 'стоимость фотопост',
                 'пакетное предложение 1',
                 'цена 1',
                 'охват 1',
                 'ссылка',
                 'статистика']

avtormedia_vk.columns = bufer_columns

In [ ]:
column_list_7 = []

for column_number in range(1, 2):
    position_column = f'пакетное предложение {column_number}'
    price_column = f'цена {column_number}'
    coverage_column = f'охват {column_number}'
    avtormedia_vk = get_column_values_packet(avtormedia_vk,
                                             column_number,
                                             position_column,
                                             price_column,
                                             coverage_column,
                                             column_list_7)

##### Обработаем список sorted_column_names_7

In [ ]:
sorted_column_names_7 = sorted(column_list_7)
sorted_column_names_7

In [ ]:
avtormedia_vk.rename(columns = {'Фото в Инст + сторис в Инст (серия из трех) + пост в ВК_1':'стоимость фотопост в Инстаграм, серию из 3х сторис в Инстаграм, пост в ВК',
                                'Сайт + соцсети (публикации на Droider.ru, FB, Twitter, Вконтакте)_1':'стоимость сайт и соцсети (публикации на Droider.ru, FB, Твиттер, Вконтакте)',
                                'охват Фото в Инст + сторис в Инст (серия из трех) + пост в ВК_1':'охват позиции стоимость за фотопост в Инстаграм, серию из 3х сторис в Инстаграм, пост в ВК',
                                'охват Сайт + соцсети (публикации на Droider.ru, FB, Twitter, Вконтакте)_1':'охват позиции стоимость за сайт и соцсети (публикации на Droider.ru, FB, Твиттер, Вконтакте)',
                               }, inplace=True)

In [ ]:
[col for col in avtormedia_vk.columns]

avtormedia_vk.drop(columns=['материалы и статистика',
                            'охват 1'], inplace=True)

> Отчистим пустые строки и сделаем их как NaN

In [ ]:
# Получаем список столбцов, содержащих слово "охват"
ohvat_columns = [col for col in avtormedia_vk.columns if 'охват' in col]

# Заменяем пустые значения в этих столбцах на NaN
avtormedia_vk[ohvat_columns] = avtormedia_vk[ohvat_columns].replace('', np.nan)

#### Проверка на дубликаты

In [ ]:
show_duplicates(avtormedia_vk)

### Тридцать первое объединение

In [ ]:
bufer = merge_bufer_df(bufer, avtormedia_vk, 'VK группа')

In [ ]:
show_duplicates(bufer)

In [ ]:
# До

bufer.shape

In [ ]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [ ]:
# После

bufer.shape

In [ ]:
show_duplicates(bufer)

### Подготовка таблицы avtormedia_zen

In [ ]:
avtormedia_zen.head()

In [ ]:
[col for col in bufer.columns if 'Дзен' in col]

In [ ]:
bufer_columns = ['блогер',
                 'тематика',
                 'количество подписчиков',
                 'материалы и статистика',
                 'среднее кол-во просмотров',
                 'стоимость интеграция до 1 мин',
                 'стоимость пост',
                 'стоимость статья',
                 'стоимость за видеопост',
                 'спешл 1',
                 'цена 1',
                 'спешл 2',
                 'цена 2',
                 'пакетное предложение 3',
                 'цена 3',
                 'охват 3',
                 'ссылка',
                 'статистика']

avtormedia_zen.columns = bufer_columns

In [ ]:
column_list_8 = []

for column_number in range(1, 3):
    position_column = f'спешл {column_number}'
    price_column = f'цена {column_number}'
    avtormedia_zen = get_column_values(avtormedia_zen,
                                       column_number,
                                       position_column,
                                       price_column,
                                       column_list_8)

In [ ]:
column_list_9 = []

for column_number in range(3, 4):
    position_column = f'пакетное предложение {column_number}'
    price_column = f'цена {column_number}'
    coverage_column = f'охват {column_number}'
    avtormedia_zen = get_column_values_packet(avtormedia_zen,
                                              column_number,
                                              position_column,
                                              price_column,
                                              coverage_column,
                                              column_list_9)

##### Обработаем список sorted_column_names_8

In [ ]:
sorted_column_names_8 = sorted(column_list_8)
sorted_column_names_8

In [ ]:
avtormedia_zen.rename(columns = {'Выездная интеграция в видео_2':'стоимость за выездную интеграцию в видео',
                                 'Модерация комментариев_1':'стоимость за модерацию комментариев'
                                }, inplace=True)

##### Обработаем список sorted_column_names_9

In [ ]:
sorted_column_names_9 = sorted(column_list_9)
sorted_column_names_9

In [ ]:
avtormedia_zen.rename(columns = {'Статья в Яндекс.Дзен + фото-пост в Инст_3':'стоимость за статью в Яндекс.Дзен и фотопост в Инстаграм',
                                 'охват Статья в Яндекс.Дзен + фото-пост в Инст_3':'охват позиции стоимость за статью в Яндекс.Дзен и фотопост в Инстаграм'}, inplace = True)

In [ ]:
avtormedia_zen.drop(columns=['материалы и статистика',
                             'охват 3'], inplace=True)

> Отчистим пустые строки и сделаем их как NaN

In [ ]:
# Получаем список столбцов, содержащих слово "охват"
ohvat_columns = [col for col in avtormedia_zen.columns if 'охват' in col]

# Заменяем пустые значения в этих столбцах на NaN
avtormedia_zen[ohvat_columns] = avtormedia_zen[ohvat_columns].replace('', np.nan)

#### Проверка на дубликаты

In [ ]:
show_duplicates(avtormedia_zen)

### Тридцать второе объединение

In [ ]:
bufer = merge_bufer_df(bufer, avtormedia_zen, 'Дзен')

In [ ]:
show_duplicates(bufer)

In [ ]:
# До

bufer.shape

In [ ]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [ ]:
# После

bufer.shape

### Подготовка таблицы avtormedia_podcast

In [ ]:
avtormedia_podcast.head()

In [ ]:
[col for col in avtormedia_podcast.columns]

In [ ]:
bufer_columns = ['блогер',
                 'тематика',
                 'количество подписчиков',
                 'материалы и статистика',
                 'среднее кол-во прослушиваний',
                 'стоимость рекламная вставка',
                 'стоимость партнерская рубрика',
                 'ссылка',
                 'статистика']

avtormedia_podcast.columns = bufer_columns

In [ ]:
avtormedia_podcast.drop(columns='материалы и статистика', inplace=True)

### Тридцать третье объединение

In [ ]:
bufer = merge_bufer_df(bufer, avtormedia_podcast, 'Подкаст')

In [ ]:
# После

bufer.shape

In [ ]:
#Заполним контактов менеджера
bufer['контакты менеджера'].fillna('reklama@avtormedia.ru', inplace=True)

#Заполним значение налога
bufer['налог'].fillna('Стоимость указана с налогами', inplace=True)

## Подготовка данных по блогерам от PLATFORMA.COMMUNITY

In [ ]:
key_word = ['Вид контракта', 'CPV']  # Список ключевых слов для поиска столбцов и их удаления
drop_word = ', до учета налогов'  # Список ключевых слов для их удаления из названия столбцов
upd_columns = ['Канал']

platformacommunity_youtube = pretraining_df(platformacommunity_youtube, key_word, drop_word, upd_columns)

### Подготовка таблицы platformacommunity_youtube

In [ ]:
platformacommunity_youtube.head(10)

> Заполним имеющиеся пропуски "сверху вниз"

In [ ]:
platformacommunity_youtube[['канал', 'проект, ведущий, формат', 'подписчики', 'ссылка', 'статистика ссылка']] = \
            platformacommunity_youtube[['канал', 'проект, ведущий, формат', 'подписчики', 'ссылка', 'статистика ссылка']].fillna(method='ffill')

> Удалим не нужные нам столбцы

In [ ]:
platformacommunity_youtube.drop(columns=['статистика', 'пример'], inplace=True)

In [ ]:
platformacommunity_youtube.columns

> На данный момент мы не будем объединять столбцы 'канал' и 'проект, ведущий, формат', так как тогда мы не сможем правильно объединить

In [ ]:
# platformacommunity_youtube['блогер'] = platformacommunity_youtube.apply(
#     lambda row: row['канал'] + ' - ' + row['проект, ведущий, формат'] if len(row['проект, ведущий, формат']) < 40 else '',
#     axis=1
# )

In [ ]:
# platformacommunity_youtube['блогер'] = \
#                 platformacommunity_youtube['канал'] + ' - ' + platformacommunity_youtube['проект, ведущий, формат']

# platformacommunity_youtube.drop(columns=['статистика аудитории',
#                                          'проект, ведущий, формат'], axis=1, inplace= True)

In [ ]:
[col for col in bufer.columns if 'YouTube' in col]

In [ ]:
[col for col in platformacommunity_youtube.columns]

In [ ]:
bufer_columns = ['блогер',
                 'проект, ведущий, формат',
                 'шоу',
                 'количество подписчиков',
                 'среднее кол-во просмотров',
                 'стоимость преролл до 30 сек',
                 'стоимость мидролл до 30 сек',
                 'стоимость интеграция 1 слот',
                 'стоимость интеграция 2 слот',
                 'стоимость продакт плейсмент',
                 'стоимость пост в сообществе',
                 'ссылка',
                 'статистика']

In [ ]:
platformacommunity_youtube.columns = bufer_columns

### Тридцать четвёртое объединение

In [ ]:
platformacommunity_youtube.loc\
    [platformacommunity_youtube['блогер'] == 'ю', 'блогер']  = 'телеканал ю'

platformacommunity_youtube.loc\
    [platformacommunity_youtube['блогер'] == 'стс', 'блогер']  = 'телеканал стс'

# platformacommunity_youtube.loc\
#     [platformacommunity_youtube['блогер'] == 'ю', 'блогер']  = 'телеканал ю'

# platformacommunity_youtube.loc\
#     [platformacommunity_youtube['блогер'] == 'ю', 'блогер']  = 'телеканал ю'

In [ ]:
bufer = merge_bufer_df(bufer, platformacommunity_youtube, 'YouTube')

In [ ]:
show_duplicates(bufer)

### Подготовка таблицы platformacommunity_instagram

In [ ]:
platformacommunity_instagram.head()

In [ ]:
platformacommunity_instagram.drop(columns=['Статистика и инфо',
                                           'ATL активации, предоставление артиста в рекламные кампании'], inplace=True)

In [ ]:
platformacommunity_instagram['Персоналии'] = platformacommunity_instagram['Персоналии'].str.lower()

In [ ]:
platformacommunity_instagram.columns

In [ ]:
[col for col in bufer.columns if 'Instagram' in col]

In [ ]:
bufer_columns = ['блогер',
                 'ссылка',
                 'количество подписчиков',
                 'тематика',
                 'стоимость фотопост',
                 'стоимость 1 сторис',
                 'стоимость reels / видеопост']

platformacommunity_instagram.columns = bufer_columns

In [ ]:
platformacommunity_instagram.head()

In [ ]:
platformacommunity_instagram[platformacommunity_instagram['блогер'] == 'полина маришова']

In [ ]:
platformacommunity_instagram = platformacommunity_instagram.dropna(subset=['блогер'])

In [ ]:
platformacommunity_instagram.loc[platformacommunity_instagram['блогер'] == 'рита дакота', 'стоимость видеопост и серия сторис'] = 750000
platformacommunity_instagram.loc[platformacommunity_instagram['блогер'] == 'полина маришова', 'стоимость фотопост и 6-7 сторис'] = 600000

In [ ]:
platformacommunity_instagram['стоимость фотопост'] = \
platformacommunity_instagram['стоимость фотопост'].replace('Видео пост + серия сторис 750 000', np.nan)

platformacommunity_instagram['стоимость фотопост'] = \
            platformacommunity_instagram['стоимость фотопост'].replace('Только комплексный вид рекламы 6-7 сторис + пост 600 000', np.nan)

In [ ]:
platformacommunity_instagram

In [ ]:
show_duplicates(platformacommunity_instagram)

### Тридцать пятое объединение

In [ ]:
bufer = merge_bufer_df(bufer, platformacommunity_instagram, 'Instagram')

### Подготовка таблицы avtormedia_telegram

In [ ]:
platformacommunity_telegram.head()

In [ ]:
platformacommunity_telegram['Канал'] = platformacommunity_telegram['Канал'].str.lower()

In [ ]:
[col for col in bufer.columns if 'Telegram' in col]

In [ ]:
[col for col in platformacommunity_telegram.columns]

In [ ]:
bufer_columns = ['блогер',
                 'шоу',
                 'тематика',
                 'количество подписчиков',
                 'стоимость текстовый пост с закрепом',
                 'стоимость текстовый пост без закрепа',
                 'стоимость фотопост',
                 'стоимость видеопост',
                 'стоимость кружочек до 60 сек',
                 'ссылка']

platformacommunity_telegram.columns = bufer_columns

In [ ]:
platformacommunity_telegram.head()

### Тридцать шестое объединение

In [ ]:
bufer = merge_bufer_df(bufer, platformacommunity_telegram, 'Telegram')

In [ ]:
#Заполним контактов менеджера
bufer['контакты менеджера'].fillna('Нина Оуэн @ohnonina +7 901 761 4650', inplace=True)

#Заполним значение налога
bufer['налог'].fillna('НДС (20%)', inplace=True)

In [ ]:
bufer.tail()

## Подготовка данных единого DataFrame

### Очистка данных

In [ ]:
itog = bufer.replace({None: np.nan})
itog = bufer.replace('None', np.nan)

In [ ]:
itog

In [ ]:
[col for col in itog.columns]

In [ ]:
column_names_list = itog.columns.tolist()

In [ ]:
spisok_columns=['Instagram количество подписчиков',
                'Instagram среднее кол-во просмотров фотопоста',
                'Instagram среднее кол-во просмотров 1 сторис',
                'Instagram среднее кол-во просмотров reels / видеопоста',
 'Instagram среднее кол-во лайков',
 'Instagram среднее кол-во комментариев',
 'Instagram стоимость фотопост',
 'Instagram стоимость 1 сторис до 15 сек',
 'Instagram стоимость серия из 3х сторис до 45 сек',
 'Instagram стоимость reels / видеопост',

 'YouTube количество подписчиков',
 'YouTube среднее кол-во просмотров видео',
 'YouTube среднее кол-во просмотров shorts',
 'YouTube среднее кол-во лайков',
 'YouTube среднее кол-во комментариев',
 'YouTube стоимость интеграция 1 слот',
 'YouTube стоимость продакт плейсмент',
 'YouTube стоимость преролл до 30 сек',
 'YouTube стоимость мидролл до 30 сек',
 'YouTube стоимость shorts до 1 мин',

 'YouTube стоимость интеграция 2 слот',

 'Telegram количество подписчиков',
 'Telegram среднее кол-во просмотров поста',
 'Telegram средняя вовлечённость',
 'Telegram стоимость фотопост',
 'Telegram стоимость видеопост',
 'Telegram стоимость кружочек до 60 сек',
 'Telegram стоимость текстовое сообщение без фото',

 'Дзен Шоу количество подписчиков',
 'Дзен Шоу среднее кол-во просмотров ролика',
 'Дзен Шоу стоимость интеграция до 1 мин',
 'Дзен Шоу стоимость продакт плейсмент',
 'Дзен Шоу стоимость преролл до 30 сек',
 'Дзен Шоу стоимость мидролл до 30 сек',

 'Дзен Шоу стоимость интеграция 2 слот',

 'VK Видео количество подписчиков',
 'VK Видео среднее кол-во просмотров ролика',
 'VK Видео стоимость интеграция до 1 мин',
 'VK Видео стоимость продакт плейсмент',
 'VK Видео стоимость преролл до 30 сек',
 'VK Видео стоимость мидролл до 30 сек',

 'VK Видео стоимость интеграция 2 слот',
 'VK Видео стоимость за интеграцию до 1 мин 2 слот',

 'VK группа количество подписчиков',
 'VK группа среднее кол-во просмотров поста',
 'VK группа среднее кол-во просмотров вк клипа',
 'VK группа среднее кол-во лайков',
 'VK группа среднее кол-во комментариев',
 'VK группа стоимость фотопост',
 'VK группа стоимость видеопост',
 'VK группа стоимость вк клип',

 'Twitch количество подписчиков',
 'Twitch среднее кол-во зрителей',
 'Twitch уникальные зрители (30 дней)',
 'Twitch стоимость за баннер + ссылка в описании 1 мес',
 'Twitch стоимость оверлей логотипа товара/бренда в процессе трансляции',
 'Twitch стоимость нативное голосовое упоминание товара в трансляции',
 'Twitch стоимость ссылка в чат-боте на 1 стрим',
 'Twitch стоимость демонстрация/обзор товара на трансляции',

 'VK Видео среднее кол-во просмотров',
 'VK Видео стоимость интеграция 1 слот',
 'VK Видео стоимость преролл до 20 сек',
 'VK Видео стоимость мидролл до 20 сек',
 'VK Видео стоимость интеграция с участием актеров 1 слот',
 'VK Видео стоимость интеграция с участием комиков из выпуска 1 слот',
 'VK Видео стоимость интеграция с участием Пушного, Крастера, Старого 1 слот',
 'VK Видео стоимость интеграция до 60 секунд 1 слот',
 'VK Видео стоимость интеграция с участием актеров 2 слот',
 'VK Видео стоимость интеграция с участием комиков из выпуска 2 слот',
 'VK Видео стоимость интеграция с участием Пушного, Крастера, Старого 2 слот',
 'VK Видео стоимость интеграция с участием блогера 1 слот',
 'VK Видео стоимость интеграция с участием Сергея Орлова 1 слот',
 'VK Видео стоимость интеграция с участием ведущих 1 слот',
 'VK Видео стоимость интеграция с участием Мамикса 1 слот',
 'VK Видео стоимость устник до 30 сек 1 слот',
 'VK Видео стоимость интеграция с участием блогера 2 слот',
 'VK Видео стоимость интеграция с участием Сергея Орлова 2 слот',
 'VK Видео стоимость интеграция с участием ведущих 2 слот',
 'OK Шоу среднее кол-во просмотров',
 'OK Шоу стоимость интеграция 1 слот',
 'OK Шоу стоимость интеграция 2 слот',
 'OK Шоу стоимость продакт плейсмент',
 'OK Шоу стоимость преролл до 20 сек',
 'OK Шоу стоимость мидролл до 20 сек',

 'YouTube среднее кол-во просмотров',
 'YouTube стоимость интеграция для профильных клиентов 1 слот',
 'YouTube стоимость интеграция для непрофильных клиентов 1 слот',
 'YouTube стоимость мини-интеграция до 1 мин',
 'YouTube стоимость интеграция в стрим',
 'YouTube стоимость интеграция 4 слот',
 'YouTube стоимость интеграция 5 слот',
 'YouTube стоимость интеграция для профильных клиентов 2 слот',
 'YouTube стоимость интеграция для непрофильных клиентов 2 слот',
 'YouTube стоимость спонсорство',
 'YouTube стоимость интеграция 3 слот',
 'YouTube стоимость выпуск под клиента',
 'YouTube стоимость нативная интеграция',
 'TikTok количество подписчиков',
 'TikTok среднее кол-во просмотров',
 'TikTok стоимость видеопост',

 'Дзен количество подписчиков',
 'Дзен среднее кол-во просмотров',
 'Дзен стоимость пост',

 'Instagram среднее кол-во просмотров серии сторис',
 'Instagram стоимость серия сторис до 60 сек',

 'Threads количество подписчиков',
 'Threads стоимость фотопост',
 'Threads стоимость видеопост',

 'VK личная страница количество подписчиков',
 'VK личная страница среднее кол-во просмотров поста',
 'VK личная страница среднее кол-во просмотров вк клипа',
 'VK личная страница стоимость вк клип',
 'VK личная страница стоимость фотопост',
 'VK личная страница стоимость видеопост',
 'VK личная страница стоимость передачи прав на 1 месяц',
 'VK группа стоимость передачи прав на 1 месяц',

 'YouTube стоимость за shorts до 1 мин',

 'Дзен стоимость видеопост',

 'Likee количество подписчиков',
 'Likee среднее кол-во просмотров',
 'Likee стоимость видеопост',

 'Yappy количество подписчиков',
 'Yappy среднее кол-во просмотров',
 'Yappy стоимость видеопост',

 'RuTube среднее кол-во просмотров видео',
 'RuTube стоимость преролл до 30 сек',
 'RuTube стоимость интеграция 1 слот',

 'Twitch уникальные зрители (14 дней)',
 'Twitch стоимость за баннер, чат-бот, голосовая интеграция до 3 мин. на 1 стрим',
 'Twitch стоимость за баннер, чат-бот, эксклюзивный стрим до 1 часа',
 'Twitch стоимость за баннер, чат-бот на 1 месяц, голосовое упоминание 1 раз в неделю (1 месяц)',

 'YouTube стоимость спецпроект',
 'YouTube стоимость бегущая строка',
 'YouTube стоимость интеграция в тревел-ролики',
 'YouTube стоимость преролл в формат новости',
 'YouTube стоимость повторное размещение реализованной интеграции',
 'YouTube стоимость интеграция в ролик с Линой Диановой 1 слот',
 'YouTube стоимость выездной спецпроект',
 'YouTube стоимость интеграция с участием Максима Каца',
 'YouTube стоимость интеграция в формат Стиратель на YouTube',
 'YouTube стоимость интеграция в формат «Книга жалоб» 2 слот',
 'YouTube стоимость интеграция в воскресный выпуск новостей "Чё Происходит" 2 слот',
 'YouTube стоимость интеграция в реакцию на трейлер',
 'YouTube стоимость интеграция в ролик с Линой Диановой 2 слот',
 'YouTube стоимость преролл в рецепт',
 'YouTube стоимость интеграция в формат "Криминальный ТикТок"',
 'YouTube стоимость преролл в формат «Книга жалоб»',
 'YouTube стоимость интеграция в формат новости',
 'YouTube стоимость преролл в формат "Линия выгорания"',
 'YouTube стоимость преролл в обзор',
 'YouTube стоимость интеграция в подкаст «Сладкая жизнь» 1 слот',
 'YouTube стоимость интеграция в воскресный выпуск новостей "Чё Происходит" 1 слот',
 'YouTube стоимость интеграция в разборы',
 'YouTube стоимость псылка в закрепленном комментарии и ссылка в описании до ката',
 'YouTube стоимость преролл в ролик с Линой Диановой',
 'YouTube стоимость интеграция в рецепт',
 'YouTube стоимость интеграция в формат «Книга жалоб» 1 слот',
 'YouTube стоимость выездная интеграция в воскресный выпуск новостей "Чё Происходит"',
 'YouTube стоимость преролл в реакцию на трейлер или разборы',
 'YouTube стоимость интеграция в обзор',
 'YouTube стоимость интеграция в формат «Спешл»',
 'YouTube стоимость интеграция в подкаст «Сладкая жизнь» 2 слот',
 'YouTube стоимость преролл в подкаст «Сладкая жизнь»',
 'YouTube стоимость 3 интеграции',
 'YouTube охват позиции 3 интеграции',
 'YouTube стоимость Cпецпроект и фотопост в Инстаграм',
 'YouTube охват позиции Cпецпроект и фотопост в Инстаграм',
 'YouTube стоимость интеграция на YouTube канал Дюшес и за серию из 3х сторис в Инстаграм',
 'YouTube охват позиции интеграция на YouTube канал Дюшес и за серию из 3х сторис в Инстаграм',
 'YouTube стоимость интеграция в видео на YouTube Алиса Лисова и интеграцию в видео на YouTube Lisova',
 'YouTube охват позиции интеграция в видео на YouTube Алиса Лисова и интеграцию в видео на YouTube Lisova',
 'YouTube стоимость интеграция в видео на YouTube и фотопост в Инстаграм',
 'YouTube охват позиции интеграция в видео на YouTube и фотопост в Инстаграм',
 'YouTube стоимость спецпроект и анонс конкурса',
 'YouTube охват позиции спецпроект и анонс конкурса',
 'YouTube стоимость интеграция в обзор и фотопост в Инстаграм',
 'YouTube охват позиции интеграция в обзор и фотопост в Инстаграм',
 'YouTube стоимость интеграция YouTube и Zen',
 'YouTube охват позиции интеграция YouTube и Zen',
 'YouTube стоимость интеграция на Основном канале + Романов Говорит',
 'YouTube охват позиции интеграция на Основном канале + Романов Говорит',
 'YouTube стоимость интеграция и фотопост в Инстаграм',
 'YouTube охват позиции интеграция и фотопост в Инстаграм',
 'YouTube стоимость спецпроект на YouTube и пост в Телеграм',
 'YouTube охват позиции спецпроект на YouTube и пост в Телеграм',
 'YouTube стоимость интеграция в рецепт и фотопост в Инстаграм',
 'YouTube охват позиции интеграция в рецепт и фотопост в Инстаграм',
 'YouTube стоимость интеграция в видео на YouTube, пост в сообществе YouTube и ссылку в закрепленном комментарии',
 'YouTube охват позиции интеграция в видео на YouTube, пост в сообществе YouTube и ссылку в закрепленном комментарии',
 'YouTube стоимость интеграция в видео на YouTube и пост в сообществе YouTube',
 'YouTube охват позиции интеграция в видео на YouTube и пост в сообществе YouTube',
 'YouTube стоимость интеграция на YouTube и сторис в Инстаграм',
 'YouTube охват позиции интеграция на YouTube и сторис в Инстаграм',
 'YouTube стоимость усложненная интеграция',
 'YouTube стоимость пост в сообществе',
 'YouTube стоимость выездная интеграция',
 'YouTube стоимость интеграция с выбором темы видео',
 'Instagram стоимость 1 сторис',
 'Instagram стоимость карусель',
 'Instagram стоимость сториз (5 экранов)',
 'Instagram стоимость рекламный день (сториз 7-10 экранов)',
 'Instagram стоимость пост в Телеграм и сторис в Инстаграм',
 'Instagram охват позиции стоимость за пост в Телеграм и сторис в Инстаграм',
 'Instagram стоимость интеграция в видео на Ютуб и фотопост в Инстаграм',
 'Instagram охват позиции стоимость за интеграцию в видео на Ютуб и фотопост в Инстаграм',
 'Instagram стоимость фотопост в Инстаграм + серию из 3х сторис в Инстаграм',
 'Instagram охват позиции стоимость за фотопост в Инстаграм + серию из 3х сторис в Инстаграм',
 'Instagram стоимость интеграция в рецепт и фотопост в Инстаграм',
 'Instagram охват позиции стоимость за интеграцию в рецепт и фотопост в Инстаграм',
 'Instagram стоимость пост в Телеграм (удаление через сутки) и пост в Инстаграм',
 'Instagram охват позиции стоимость за пост в Телеграм (удаление через сутки) и пост в Инстаграм',
 'Instagram стоимость интеграция на Ютуб и сторис в Инстаграм',
 'Instagram охват позиции стоимость за интеграцию на Ютуб и сторис в Инстаграм',
 'Instagram стоимость фотопост в Инстаграм, серию из 3х сторис в Инстаграм и пост в ВК',
 'Instagram охват позиции стоимость за фотопост в Инстаграм, серию из 3х сторис в Инстаграм и пост в ВК',
 'Instagram стоимость интеграция в обзор и фотопост в Инстаграм',
 'Instagram охват позиции стоимость за интеграцию в обзор и фотопост в Инстаграм',
 'Instagram стоимость серия из 3х сторис и пост в Телеграм',
 'Instagram охват позиции стоимость за серию из 3х сторис и пост в Телеграм',
 'Instagram стоимость фотопост в Инстаграм + пост в Телеграм',
 'Instagram охват позиции стоимость фотопост в Инстаграм + пост в Телеграм',
 'Telegram стоимость за авторский пост',
 'Telegram стоимость за пост из удаления',
 'Telegram стоимость фотопост в Инстаграм и пост в Телеграм',
 'Telegram охват позиции стоимость за фотопост в Инстаграм и пост в Телеграм',
 'Telegram стоимость спецпроект на Ютуб и пост в Телеграм',
 'Telegram охват позиции стоимость за спецпроект на Ютуб и пост в Телеграм',
 'Telegram стоимость пост в Телеграм и сториз в Инстаграм',
 'Telegram охват позиции стоимость за пост в Телеграм и сториз в Инстаграм',
 'Telegram стоимость 5 постов в Телеграм (удаление через сутки)',
 'Telegram охват позиции стоимость за 5 постов в Телеграм (удаление через сутки)',
 'Telegram стоимость 3 поста в Телеграм',
 'Telegram охват позиции стоимость за 3 поста в Телеграм',
 'Telegram стоимость рекламное сообщение во все чаты "Вместе"',
 'Telegram охват позиции стоимость за рекламное сообщение во все чаты "Вместе"',
 'Telegram стоимость пост в Телеграм (удаление через сутки) и пост в Инстаграм',
 'Telegram охват позиции стоимость за пост в Телеграм (удаление через сутки) и пост в Инстаграм',
 'Telegram стоимость серия из 3х сторис в Инстаграм и пост в Телеграм',
 'Telegram охват позиции стоимость за серию из 3х сторис в Инстаграм и пост в Телеграм',
 'VK группа стоимость сторис',
 'VK группа стоимость фотопост в Инстаграм, серию из 3х сторис в Инстаграм, пост в ВК',
 'VK группа охват позиции стоимость за фотопост в Инстаграм, серию из 3х сторис в Инстаграм, пост в ВК',
 'VK группа стоимость сайт и соцсети (публикации на Droider.ru, FB, Твиттер, Вконтакте)',
 'VK группа охват позиции стоимость за сайт и соцсети (публикации на Droider.ru, FB, Твиттер, Вконтакте)',
 'Дзен стоимость интеграция до 1 мин',
 'Дзен стоимость статья',
 'Дзен стоимость за видеопост',

 'Дзен стоимость за модерацию комментариев',
 'Дзен стоимость за выездную интеграцию в видео',
 'Дзен стоимость за статью в Яндекс.Дзен и фотопост в Инстаграм',
 'Дзен охват позиции стоимость за статью в Яндекс.Дзен и фотопост в Инстаграм',

 'Подкаст количество подписчиков',
 'Подкаст среднее кол-во прослушиваний',
 'Подкаст стоимость рекламная вставка',
 'Подкаст стоимость партнерская рубрика',

 'Instagram стоимость видеопост и серия сторис',
 'Instagram стоимость фотопост и 6-7 сторис',

 'Telegram стоимость текстовый пост с закрепом',
 'Telegram стоимость текстовый пост без закрепа']

In [ ]:
difference1 = list(set(column_names_list).difference(spisok_columns))
difference1

In [ ]:
exclude_words = ['город',
                 'ссылка',
                 'статистика',
                 'блогер',
                 'тематика',
                 'ца по возрасту',
                 'проект, ведущий, формат',
                 'материалы и статистика',
                 'ца по полу',
                 'контакты менеджера',
                 'название канала',
                 'ца по городам',
                 'шоу',
                 'ца по странам',
                 'налог']

# Создаем список из названий столбцов, исключая те, которые содержат определённые слова
spisok_columns = [col for col in itog.columns if not any(word in col for word in exclude_words)]

In [ ]:
for columns in spisok_columns:
    itog[columns] = itog[columns].astype(str)
    itog[columns] = itog[columns].str.replace('р.', '', regex=True)
    itog[columns] = itog[columns].str.replace(r'(\d+),(\d+)', r'\1.\2', regex=True)
    itog[columns] = itog[columns].str.replace('[^\d.]', '', regex=True)
    itog[columns] = itog[columns].str.replace(' ', '')
    itog[columns] = itog[columns].replace('', np.nan)
    itog[columns] = itog[columns].astype(float)